# Ecofor veg analysis

In [ ]:
%matplotlib inline
import os, shapely, joblib, sys, subprocess, rasterio
# os.environ['USE_PYGEOS'] = '0'
from glob import glob
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from osgeo import ogr
from joblib import Parallel, delayed
import dask.dataframe as dd

from rasterstats import zonal_stats, gen_point_query
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, classification_report
from sklearn.model_selection import cross_val_predict, train_test_split

# gpd.options.io_engine = "pyogrio" # need to update to use this for faster i/o
# os.environ["PYOGRIO_USE_ARROW"] = 1

# sys.path.append(r'J:\users\stevenf\code\language\python')
# from sfgeo.raster_bounds import aoi_raster

In [ ]:
# run list of commands with concurrent threads
# TODO: look for better library to do this (joblib?) or make better non-blocking, reusable function with progress bar.
def cmd_concurrent(cmds, threads=1): 
    from subprocess import Popen
    from itertools import islice
    
    processes = (Popen(cmd, shell=True) for cmd in cmds)
    running_processes = list(islice(processes, threads))  # start new processes
    while running_processes:
        for i, process in enumerate(running_processes):
            if process.poll() is not None:  # the process has finished
                running_processes[i] = next(processes, None)  # start new process
                if running_processes[i] is None: # no new processes
                    del running_processes[i]
                    break
    return True

In [ ]:
def pretty_matrix(y_true, y_pred, normalize=True, outline_diag=True, labels=None):
    from sklearn.metrics import confusion_matrix
    if labels is None:
        names = np.unique(pd.concat([y_true, y_pred]))
    else:
        names=labels
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    fmt = "d"
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = "0.2f"
    cm = pd.DataFrame(cm, index = names, columns= names)
    cm.index.name = "Observed"
    cm.columns.name = "Predicted"
    fig, ax = plt.subplots(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap="viridis", ax=ax);
    
    if outline_diag:
        for i in range(len(cm)):
            ax.plot([i, i, i+1, i+1, i], [i, i+1, i+1, i, i], color="Red", linewidth=2)
   
    return fig

In [ ]:
def obs_pred_hexbin(y, x, folds=None, vmax=100):
    """y=true, x=pred, 
       k = Series of fold index in x and y used in K-fold cross-validation. Calculate mean error across folds if given.
    """
    fig, ax = plt.subplots(figsize=(4.5,3.75))   #(3, 2.5)
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=vmax)
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('counts')
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')
    
    if folds is not None:
        fdf = pd.DataFrame({'true':y, 'pred':x, 'fold':folds})
        folded = fdf.groupby('fold')
        r2 = folded.apply(lambda g: r2_score(g['true'], g['pred'])).mean()
        bias = folded.apply(lambda g: (g['pred'] - g['true']).mean()).mean()
        rmse = folded.apply(lambda g: mean_squared_error(g['true'], g['pred'])**0.5).mean()
    else:
        r2 = r2_score(y, x)
        bias = (x-y).mean()
        rmse = mean_squared_error(y, x)**0.5
    
    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(np.round(r2,2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(np.round(bias, 2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(np.round(rmse, 2)), transform=ax.transAxes, ha='right')
    ax.set(xlabel='Predicted', ylabel='Observed')
    return fig

In [ ]:
# pyramid and stats helper function
# If pyramids are generated for the tiles before creating VRT mosaics then gdalbuildvrt will recognize the 
# presence of the pyramids and add a line in the XML file to use them. They can then be used for doing 
# approximate statistics too.
def pyr_stats(path, nodata=None, run=True):
    """Set nodata (str of number or 'nan'). Calculate stats and pyramids for image at path (str)."""
    cmds = {'stats':[], 'pyr':[]}
    if nodata:
        cmd = 'rio edit-info --nodata ' + str(nodata) + ' ' + path
        result = subprocess.check_output(cmd)
    
    stats_cmd = 'gdalinfo -approx_stats --config GDAL_PAM_ENABLED TRUE ' + path
    if run:
        result = subprocess.check_output(stats_cmd)
    
    if not os.path.exists(path[:-4]+".ovr"):
        pyr_cmd = 'gdaladdo -ro --config COMPRESS_OVERVIEW ZSTD --config ZSTD_LEVEL 1 --config PREDICTOR 2 --config INTERLEAVE_OVERVIEW BAND --config GDAL_CACHEMAX 4096 ' + path
        if run:
            result = subprocess.check_output(pyr_cmd)
    
    return stats_cmd, pyr_cmd

# Temporary

In [ ]:
import fiona
fiona.supported_drivers['KML'] = 'rw'

In [ ]:
# quick creation of shapefile of examples
locationDict = {
  'Thornybush': {'lon': 31.180085534751232, 'lat': -24.449807514937508, 'zoom': 12, 'metric':'cover', 'year1':2016, 'year2':2020},
  'Tree plantations': {'lon': 30.748241, 'lat': -25.249696, 'zoom': 13, 'metric':'rh98', 'year1':2015, 'year2':2022},
  'Communal lands': {'lon': 30.9675782982199, 'lat': -24.49883259221857, 'zoom': 14, 'metric':'pai', 'year1':2017, 'year2':2021},
  'Citrus orchards': {'lon': 31.657928, 'lat': -25.441006, 'zoom': 14, 'metric':'pai', 'year1':2007, 'year2':2022},
  'Urbanization': {'lon': 31.691362, 'lat': -25.452824, 'zoom': 13, 'metric':'cover', 'year1':2007, 'year2':2015},
  'Woody encroachment': {'lon': 31.719479, 'lat': -25.051213, 'zoom': 14, 'metric':'fhd', 'year1':2009, 'year2':2019},
  'Prescribed fire': {'lon': 31.830821, 'lat': -24.46095, 'zoom': 13, 'metric':'cover', 'year1':2018, 'year2':2022},
}

df = pd.DataFrame(locationDict).T
df['name'] = df.index
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']), crs="EPSG:4326")
gdf = gdf.reset_index(drop=True)

In [ ]:
gdf.to_file(r"H:\ECOFOR\gedi\maps\v03\gedi_change_examps_v3.kml", driver="KML")

In [ ]:
from datetime import timedelta, datetime

def convert_partial_year(number):

    year = int(number)
    d = timedelta(days=(number - year)*365)
    day_one = datetime(year,1,1)
    date = d + day_one
    return date

{d:convert_partial_year(d) for d in [2010.0, 2015.8, 2020.5, 2022.1]}

In [ ]:
path = r"J:\data\lidar\_readMe\DataSources.xlsx"
df = pd.read_excel(path)

In [ ]:
df['Current drive'].value_counts()

In [ ]:
(df['Current drive']!='None').sum()

In [ ]:
path = r"J:\projects\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2AB_2019to2023_leafon_sampy500m.parquet"
df = gpd.read_parquet(path)

In [ ]:
print(df.columns.tolist())

In [ ]:
cols = ['shot_number', 'beam', 'algorithmrun_flag', 'degrade_flag', 'l2b_quality_flag', 'stale_return_flag', 'surface_flag',
        'solar_elevation', 'delta_time', 'sensitivity', 'lat_lowestmode', 'lon_lowestmode', 'elev_highestreturn', 'elev_lowestmode',
        'local_beam_elevation', 'fhd_normal', 'pgap_theta', 'pai', 'rhov', 'rhog', 'omega', 'cover', 'cover_z_0_5m', 'cover_z_5_10m',
        'cover_z_10_15m', 'cover_z_15_20m', 'cover_z_20_25m', 'pai_z0_5m', 'pai_z5_10m', 'pai_z10_15m', 'pai_z15_20m', 'pai_z20_25m',
        'pavd_z0_5m', 'pavd_z5_10m', 'pavd_z10_15m', 'pavd_z15_20m', 'pavd_z20_25m', 'rh90', 'rh95', 'rh96', 'rh97', 'rh98', 'rh99',
        'rh100', 'geometry', 'millis', 'year', 'rain_year', 'x', 'y', 'x_grid', 'y_grid']
sub = df[cols]

In [ ]:
sub.to_file(r"D:\GEDI_2AB_2019to2023_leafon_sampy500m_subcols.gpkg", driver="GPKG")

## Compare GEDI to 2018 CHM

In [ ]:
# Load CHM 1m GEDI extract and compare
path = r"C:\scratch\ecofor\gedi\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_chm1m.parquet"
df = gpd.read_parquet(path)
df = df.set_index('shot_number')

In [ ]:
df.rename(columns={i:'chm1m_'+i for i in ['mean', 'median', 'max']}, inplace=True)

In [ ]:
col_list = [('chm1m_mean','rh50'), ('chm1m_median','rh50'), ('chm1m_max','rh98')]

In [ ]:
# Plot the model results
fig, axes = plt.subplots(1, 3, figsize=(8, 1.75))

for i, (cols, ax) in enumerate(zip(col_list, axes)):
    subdf = df[list(cols)].dropna(axis=0)
    obscol, predcol = cols[0], cols[1]
    
    # remove gedi outliers for now
    subdf = subdf[subdf[predcol]<subdf[obscol].max()]
    
    # zero out negative CHM values for now
    subdf.loc[subdf[obscol]<0, obscol] = 0
    
    x, y = subdf[predcol], subdf[obscol]
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')
    
    r2 = r2_score(y, x)
    bias = (x-y).mean()
    rmse = mean_squared_error(y, x)**0.5
    
    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(r2.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')
    
    ax.set(xlabel=predcol, ylabel=obscol)
    # ax.set(title=ycol)
    # if i==0:
    #     ax.set(ylabel='Observed')
    # fig.supxlabel('Predicted', x=0.47, y=-0.16, ha='center', fontsize=10)

fig.subplots_adjust(wspace=0.4, hspace=0.5)
cb = fig.colorbar(hb, ax=axes, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
cb.set_label('Count')

# Thornybush data

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\Thornybush - NASA Tree Data with dates.xlsx"
df = pd.read_excel(path)

In [ ]:
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), crs=4326)

In [ ]:
df.to_file(r"J:\projects\ECOFOR\ancillary_data\Thornybush - NASA Tree Data with dates.gpkg", driver="GPKG", index=False)

# Field data
Prepare samples for field visits and wrangle collected data.

## GEDI field sample
Filter for points next to the roads that will be visited in each field season. Use stratified random sample to select points.

In [ ]:
# need to read a gpd first to initiallize fiona before turning on driver
gpd.io.file.fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

### January 2023

In [ ]:
# path = r"C:\scratch\ecofor\Kruger route.kml"
# route_names = ['Main route', 'Skukuza drives']

path = r"G:\My Drive\Work\ecofor\field_collection\Bushriver roads north.kml"
route_names = ["Bushriver roads north"]

# path = r"G:\My Drive\Work\ecofor\field_collection\bushriver roads south.kml"
# route_names = ["bushriver roads south"]

dfs = []
for name in route_names:
    df_route = gpd.read_file(path, layer=name)
    df_route = df_route[df_route['Name']==name]
    dfs.append(df_route)
rts = pd.concat(dfs)

In [ ]:
wgs84bufrect = rts.unary_union.envelope.buffer(0.01)
rts = rts.to_crs(epsg=32636)
rt = rts.unary_union
rtbuf = rt.buffer(1000)

In [ ]:
# Load gedi data and rough filter
cols = ['shot_number', 'beam', 'sensitivity', 'pai', 'cover', 'rh98', 'geometry']
df = gpd.read_parquet(r"C:\scratch\ecofor\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet", columns=cols)

sub = df[df.intersects(wgs84bufrect)]
sub = sub.to_crs(epsg=32636)

In [ ]:
# Route intersection and save
sdf = sub[sub.intersects(rtbuf)]
sdf = sdf.to_crs(epsg=4326)
sdf['strat'] = pd.cut(sdf['rh98'],[0,3,5,10,15,60], include_lowest=True)
sdf['strat']=sdf['strat'].astype(str)

sdf.to_file(r"C:\scratch\ecofor\GEDI_bushrivernorth.shp")#, driver="GPKG")

In [ ]:
cols = ['shot_number', 'comment', 'geometry']
sdf['covstr'] = (sdf['cover']*100).round().astype(int).astype(str)
sdf['rh98str'] = sdf['rh98'].round(2).astype(str)
sdf['comment'] = ("strat="+sdf['strat']+'   '+
                   "cov="+sdf['covstr']+'   '+
                   "rh98="+sdf['rh98str']+'   '+
                   "shot="+sdf['shot_number'])
sdf[cols].to_file(r"C:\scratch\ecofor\GEDI_bushriversouth_togpx.shp")

In [ ]:
# Stratified random samp by height
df = gpd.read_file(r"C:\scratch\ecofor\GEDI_rt220107.gpkg")
df['strat'] = pd.cut(df['rh98'],[0,3,5,10,15,60], include_lowest=True)
samp = df.groupby('strat', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 10), random_state=0))
samp['strat']=samp['strat'].astype(str)
samp.to_file(r"C:\scratch\ecofor\GEDI_rt220107_samp.gpkg", driver="GPKG")

In [ ]:
# Save to shapefile with subset of columns for gps
samp = gpd.read_file(r"C:\scratch\ecofor\GEDI_rt220107_samp.gpkg")
samp['covstr'] = (samp['cover']*100).round().astype(int).astype(str)
samp['rh98str'] = samp['rh98'].round(2).astype(str)
samp['comment'] = ("strat="+samp['strat']+'   '+
                   "cov="+samp['covstr']+'   '+
                   "rh98="+samp['rh98str']+'   '+
                   "shot="+samp['shot_number'])
samp.head()

In [ ]:
cols = ['shot_number', 'comment', 'geometry']
samp[cols].to_file(r"C:\scratch\ecofor\GEDI_rt220107_samp.shp")

In [ ]:
samp

### May 2023 v1  
First creation of points to be used in May 2023 campaign.

In [ ]:
# need to read a gpd first to initiallize fiona before turning on driver
# gpd.io.file.fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

# path = r"E:\ecofor\ancillary_data\SAPAD_OR_2021_Q3\SAPAD_OR_2021_Q3.shp"
# path = r"C:\scratch\ecofor\GEDI_skukuza230118.gpkg"
# path = r"C:\scratch\ecofor\gedi_pundaroute.gpkg"
path = r"J:\projects\ECOFOR\maps\ecofor\ecofor.gdb"
# polys = gpd.read_file(path, layer='pundamaria_poly')
polys = gpd.read_file(path, layer='hotosm_roads_kruger_buf500m_nodissolve')
# polys = polys[polys["WDPAID"] == 555570306] # Olifants west
# polys = polys[polys["CUR_NME"] == "Thornybush Nature Reserve"]
polys = polys.to_crs(epsg=32636)
polys.sindex;
# poly = polys.unary_union

In [ ]:
# Load gedi data and rough filter
cols = ['shot_number', 'beam', 'sensitivity', 'pai', 'cover', 'rh98', 'geometry']
df = gpd.read_parquet(r"C:\scratch\ecofor\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet", columns=cols)

df = df.to_crs(epsg=32636)
df.sindex;

In [ ]:
# spatial join with all individual polys faster than intersects or sjoin with unary_union poly
sdf = df.sjoin(polys, how='inner')
sdf = sdf.drop_duplicates('shot_number')

## simple but slow intersection
# sdf = df[df.intersects(poly)]

In [ ]:
# sdf = sdf.to_crs(epsg=4326)
sdf['strat'] = pd.cut(sdf['rh98'],[0,3,5,10,15,60], include_lowest=True)
sdf['strat'] = sdf['strat'].astype(str)

sdf.to_file(r"C:\scratch\ecofor\GEDI_hotosm_roadbuf500m_points.gpkg", driver="GPKG")

In [ ]:
# output as GPX
outpath = r"J:\projects\ECOFOR\field\gedi_may23\samples\GEDI_osmroads_buf500m.gpx"

# For GPX combine data into comment field
cols = ['shot_number', 'comment', 'geometry']
sdf['covstr'] = (sdf['cover']*100).round().astype(int).astype(str)
sdf['rh98str'] = sdf['rh98'].round(2).astype(str)
sdf['comment'] = ("strat="+sdf['strat']+'   '+
                   "cov="+sdf['covstr']+'   '+
                   "rh98="+sdf['rh98str']+'   '+
                   "shot="+sdf['shot_number'])



sdf['name'] = sdf['shot_number'].astype(str)
sdf['ele'] = 0.
sdf['magvar'] = 0.
sdf['time'] = pd.to_datetime('2019-08-02T14:17:50Z')
sdf['geoidheight'] = 0.
sdf['cmt'] = sdf['comment']
sub_cols = ['geometry', 'ele', 'time', 'magvar', 'geoidheight', 'name', 'cmt']
sdf[sub_cols].to_file(outpath, driver='GPX')

In [ ]:
# # Save as CSV for making trimble wpt file
# outpath = r"C:\scratch\ecofor\GEDI_pundamaria.csv"
# sdf['Latitude'], sdf['Longitude'] = sdf.geometry.y, sdf.geometry.x
# sdf[['shot_number', 'Latitude', 'Longitude', ]].to_csv(outpath, index=False)

### May 2023
Second version? Clip out a few areas David is visiting in May 2023 with UBC students.

In [ ]:
# Filter to roads in the greater Kruger area
roads_path = r"J:\projects\ECOFOR\ancillary_data\roads\hotosm_zaf_roads_gpkg\hotosm_zaf_roads.gpkg"
roads = gpd.read_file(roads_path)
roads.sindex;

aoi = gpd.read_file(r"J:\projects\ECOFOR\boundaries\GKSDP_Area_Prj\GKSDP_Area_Prj.shp")
aoi = aoi.to_crs(roads.crs)
aoi.sindex;

roads = roads[roads.intersects(aoi.loc[0, 'geometry'])]

roads = roads.to_crs(epsg=32636)

roads_buf = gpd.GeoDataFrame(geometry=roads.buffer(500))

In [ ]:
roads_buf.to_file(r"G:\temp\hotosm_zaf_roads_buf500m.gpkg", driver="GPKG")

In [ ]:
# Load gedi data
cols = ['shot_number', 'beam', 'sensitivity', 'pai', 'cover', 'rh98', 'geometry']
df = gpd.read_parquet(r"C:\scratch\ecofor\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet", columns=cols)

# Get points that intersect the buffered roads
df = df.to_crs(epsg=32636)
df.sindex;

In [ ]:
roads_buf_x = gpd.GeoDataFrame(geometry=[roads_buf.unary_union], crs=roads_buf.crs).explode().reset_index(drop=True)

In [ ]:
# Clip gedi points to buffered greater kruger roads 
# (takes 14 hours) probably still faster to do spatial join and drop duplicates
rdf = df.overlay(roads_buf_x, how='intersection')

In [ ]:
rdf['strat'] = pd.cut(rdf['rh98'],[0,3,5,10,15,60], include_lowest=True)
rdf['strat'] = rdf['strat'].astype(str)

In [ ]:
rdf.to_crs(epsg=4326).to_file(r"J:\projects\ECOFOR\field\gedi_may23\samples\GEDI_gkdp_osmroads_buf500m.gpkg", driver='GPKG')

In [ ]:
rdf.sindex;

In [ ]:
# Clip out AOIs
polys_path = r"J:\projects\ECOFOR\field\gedi_may23\samples\ubc_may23_aois.gpkg"
polys = gpd.read_file(polys_path)
polys = polys.to_crs(rdf.crs)
polys.sindex;

In [ ]:
pdf = rdf.sjoin(polys, how='inner')
pdf = pdf.drop_duplicates('shot_number')

In [ ]:
# pdf = pdf.drop(columns='index_right')
pdf.to_crs(epsg=4326).to_file(r"J:\projects\ECOFOR\field\gedi_may23\samples\GEDI_osmroads_ubcmay23aois_buf500m.gpkg", driver='GPKG')

In [ ]:
# groupby and sample for each aoi
grouped = pdf.groupby('name', as_index=False)
sampdf = grouped.apply(lambda x: x.sample(min(len(x), 10))).reset_index(drop=True)
sampdf = sampdf.to_crs(epsg=4326)
sampdf.to_file(r"J:\projects\ECOFOR\field\gedi_may23\samples\GEDI_osmroads_ubcmay23aois_buf500m_samp10.gpkg", driver='GPKG')

In [ ]:
# output as GPX
outpath = r"J:\projects\ECOFOR\field\gedi_may23\samples\GEDI_osmroads_ubcmay23aois_buf500m_samp10.gpx"

# For GPX combine data into comment field
cols = ['shot_number', 'comment', 'geometry']
sampdf['covstr'] = (sampdf['cover']*100).round().astype(int).astype(str)
sampdf['rh98str'] = sampdf['rh98'].round(2).astype(str)
sampdf['comment'] = ("strat="+sampdf['strat']+'   '+
                   "cov="+sampdf['covstr']+'   '+
                   "rh98="+sampdf['rh98str']+'   '+
                   "shot="+sampdf['shot_number'])



sampdf['name'] = sampdf['shot_number'].astype(str)
sampdf['ele'] = 0.
sampdf['magvar'] = 0.
sampdf['time'] = pd.to_datetime('2019-08-02T14:17:50Z')
sampdf['geoidheight'] = 0.
sampdf['cmt'] = sampdf['comment']
sub_cols = ['geometry', 'ele', 'time', 'magvar', 'geoidheight', 'name', 'cmt']
sampdf[sub_cols].to_file(outpath, driver='GPX')

In [ ]:
%%time
# load gedi data
cols = ['shot_number', 'beam', 'sensitivity', 'pai', 'cover', 'rh98', 'geometry']
gdf = gpd.read_parquet(r"C:\scratch\ecofor\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet")#, columns=cols)
gdf = gdf.rename(columns={'shot_number':'gedi'})

### May 2024
Get sample of GEDI shots for UBC students to visit in May 2024.

In [ ]:
# Load gedi data
cols = ['shot_number', 'beam', 'sensitivity', 'pai', 'cover', 'rh98', 'geometry']
df = gpd.read_parquet(r"H:\ECOFOR\gedi\extracted\GEDI_2AB_2019to2023_all.parquet", columns=cols)
df = df.reset_index()
df['shot_number'] = df['shot_number'].astype(str)
df = df.to_crs(epsg=32636)
df.sindex;

In [ ]:
# Clip out AOIs
polys_path = r"J:\projects\ECOFOR\field\gedi_may24\samples\ubc_may24_north_aois.gpkg" #r"J:\projects\ECOFOR\field\gedi_may24\samples\ubc_may24_aois.gpkg"
outpath = r"J:\projects\ECOFOR\field\gedi_may24\samples\GEDI_ubcmay24northaois.gdb" #r"J:\projects\ECOFOR\field\gedi_may24\samples\GEDI_ubcmay24aois.gdb"

polys = gpd.read_file(polys_path)
polys = polys.to_crs(df.crs)
polys.sindex;

pdf = df.sjoin(polys, how='inner')
pdf = pdf.loc[pdf.index.drop_duplicates()]

pdf = pdf.drop(columns='index_right')

pdf['strat'] = pd.cut(pdf['rh98'],[0,3,5,10,15,60], include_lowest=True)
pdf['strat'] = pdf['strat'].astype(str)

# Save
pdf.to_crs(epsg=4326).to_file(outpath, driver='OpenFileGDB') # saving to GDB because GPKG uses 64-bit objectid which is not compatible with "enable sync" in AGOL web layer

In [ ]:
# groupby and sample for each aoi
grouped = pdf.groupby('name', as_index=False)
sampdf = grouped.apply(lambda x: x.sample(min(len(x), 30))).reset_index(drop=True)
sampdf = sampdf.to_crs(epsg=4326)
sampdf.to_file(outpath[:-4]+"_samp30.gdb", driver='OpenFileGDB')

In [ ]:
# output as GPX

# For GPX combine data into comment field
cols = ['shot_number', 'comment', 'geometry']
sampdf['covstr'] = (sampdf['cover']*100).round().astype(int).astype(str)
sampdf['rh98str'] = sampdf['rh98'].round(2).astype(str)
sampdf['comment'] = ("strat="+sampdf['strat']+'   '+
                   "cov="+sampdf['covstr']+'   '+
                   "rh98="+sampdf['rh98str']+'   '+
                   "shot="+sampdf['shot_number'])

sampdf['name'] = sampdf['shot_number']
sampdf['ele'] = 0.
sampdf['magvar'] = 0.
sampdf['time'] = pd.to_datetime('2019-08-02T14:17:50Z')
sampdf['geoidheight'] = 0.
sampdf['cmt'] = sampdf['comment']
sub_cols = ['geometry', 'ele', 'time', 'magvar', 'geoidheight', 'name', 'cmt']
sampdf[sub_cols].to_file(outpath[:-5]+"_samp30.gpx", driver='GPX')

### May 2025
Get sample of GEDI shots for UBC students to visit in May 2024.

In [ ]:
# Load gedi data
cols = ['shot_number', 'beam', 'delta_time', 'l2b_quality_flag', 'sensitivity', 'pai', 'cover', 'rh98', 'geometry']
df = gpd.read_parquet(r"F:\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2AB_2019to2023.parquet", columns=cols)

df['shot_number'] = df['shot_number'].astype(str)

# Filter to quality shots, leaf-on, and reasonable height
df = df[df['l2b_quality_flag']==1] # quality filter
df['delta_time'] = pd.to_datetime(df['delta_time'])
df = df[df['rh98']<45] # Remove unreasonable points
# df['leaf_on'] = (df['delta_time'].dt.day_of_year < 121) | (df['delta_time'].dt.day_of_year > 305) # keep only leaf-on (Nov - Apr) as defined in Li 2023 (removes too many points)
df = df[(df['delta_time'].dt.day_of_year < 121) | (df['delta_time'].dt.day_of_year > 305)] # keep only leaf-on (Nov - Apr) as defined in Li 2023 (removes too many points)

df = df.to_crs(epsg=32636)
df.sindex;

In [ ]:
# Clip out AOIs
import fiona
fiona.supported_drivers['LIBKML'] = 'rw'

polys_path = r"J:\projects\ECOFOR\field\gedi_may25\GEDI point locations 2025 (1).kml"
outpath = r"J:\projects\ECOFOR\field\gedi_may25\samples\GEDI_ubcmay25_points.gdb"

polys = gpd.read_file(polys_path)

polys = polys[polys['geometry'].geom_type=='Polygon']
polys = polys.to_crs(df.crs)
polys.sindex;

pdf = df.sjoin(polys, how='inner')
pdf = pdf.loc[pdf.index.drop_duplicates()]

pdf = pdf.drop(columns='index_right')

pdf['strat'] = pd.cut(pdf['rh98'],[0,3,5,10,15,60], include_lowest=True)
pdf['strat'] = pdf['strat'].astype(str)

# Save
pdf.to_crs(epsg=4326).to_file(outpath, driver='OpenFileGDB') # saving to GDB because GPKG uses 64-bit objectid which is not compatible with "enable sync" in AGOL web layer

In [ ]:
# groupby and sample for each aoi
grouped = pdf.groupby('Name', as_index=False)
sampdf = grouped.apply(lambda x: x.sample(min(len(x), 30))).reset_index(drop=True)
sampdf = sampdf.to_crs(epsg=4326)
sampdf.to_file(outpath[:-4]+"_samp30.gdb", driver='OpenFileGDB')

In [ ]:
# output as GPX

# For GPX combine data into comment field
cols = ['shot_number', 'comment', 'geometry']
sampdf['covstr'] = (sampdf['cover']*100).round().astype(int).astype(str)
sampdf['rh98str'] = sampdf['rh98'].round(2).astype(str)
sampdf['comment'] = ("strat="+sampdf['strat']+'   '+
                   "cov="+sampdf['covstr']+'   '+
                   "rh98="+sampdf['rh98str']+'   '+
                   "shot="+sampdf['shot_number'])

sampdf['name'] = sampdf['shot_number']
sampdf['ele'] = 0.
sampdf['magvar'] = 0.
sampdf['time'] = pd.to_datetime('2019-08-02T14:17:50Z')
sampdf['geoidheight'] = 0.
sampdf['cmt'] = sampdf['comment']
sub_cols = ['geometry', 'ele', 'time', 'magvar', 'geoidheight', 'name', 'cmt']
sampdf[sub_cols].to_file(outpath[:-5]+"_samp30.gpx", driver='GPX')

## Organize collected data

### 2023 data prep

#### GEDI trees
Merge tree and plot data from May and Jan visits

In [ ]:
ddirs = [r"J:\projects\ECOFOR\field\gedi_jan23", r"J:\projects\ECOFOR\field\gedi_may23"]
dfs = []
for ddir in ddirs:
    pdf = pd.read_csv(os.path.join(ddir, "gedi_plot.csv"), dtype={'gedi':str})
    tdf = pd.read_csv(os.path.join(ddir, "gedi_trees.csv"))
    cols = tdf.columns.difference(pdf.columns.drop(['plot']))
    
    mdf = pd.merge(pdf, tdf[cols], on='plot', how='right')
    mdf['visit'] = ddir[-5:]
    dfs.append(mdf)
    
df = pd.concat(dfs)

df = df.sort_values(['visit', 'plot'])
df['plot_ix'] = pd.factorize(df['visit']+df['plot'].astype(str))[0]

In [ ]:
# Get quadrant

# def get_az_quadrant(az):
#     if np.isnan(az):
#         return None
#     else:
#         labels = {0:'ne', 1:'se', 2:'sw', 3:'nw'}
#         return labels[int(az//90)]

# df['quadrant'] = df['az'].apply(get_az_quadrant)

df['quadrant'] = df['Direction'].str.lower()
df['quadrant'] = df['quadrant'].fillna(df['num'].replace({1:'ne', 2:'se', 3:'sw', 4:'nw'}))

In [ ]:
# Save just the trees data
cols = ['plot_ix', 'quadrant', 'az', 'dist', 'hgt', 'species', 'live', 'pos', 'bole', 'notes',                              # plot and tree identifier, and tree values
        'visit', 'plot', 'gedi', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables

df[cols].to_csv(r"J:\projects\ECOFOR\field\field_trees_merged.csv", index=False)

In [ ]:
# merge in GEDI data
tdf = pd.merge(df, gdf, how='left', on='gedi')
tdf = gpd.GeoDataFrame(tdf, geometry='geometry', crs=gdf.crs)
tdf['date'] = tdf['date'].astype(str)
tdf['delta_time'] = tdf['delta_time'].astype(str)
tdf = tdf[cols + list(gdf.columns.drop('gedi'))]

In [ ]:
# Project tree coordinates but save plot coordinates
tdf['geo_utm36n'] = tdf['geometry'].to_crs(epsg = 32636)

def get_tree_coords(r):
    if np.isnan(r['az']) or np.isnan(r['dist']) or (r['geo_utm36n'] is None):
        return None
    rad = np.radians(r['az'])
    dx = r['dist'] * np.sin(rad)
    dy = r['dist']* np.cos(rad)
    return shapely.affinity.translate(r['geo_utm36n'], dx, dy)

tdf['tree_geo'] = tdf.apply(get_tree_coords, axis=1)
tdf['geometry'] = tdf['tree_geo'].set_crs(tdf['geo_utm36n'].crs).to_crs(tdf.crs)
tdf = tdf.drop(columns = ['geo_utm36n', 'tree_geo'])

In [ ]:
tdf.to_file(r"J:\projects\ECOFOR\field\gedi_all_merged.gpkg", layer="trees", driver="GPKG")

In [ ]:
tdf.drop(columns='geometry').to_csv(r"J:\projects\ECOFOR\field\gedi_trees_merged.csv", index=False)

#### GEDI cover

In [ ]:
ddirs = [r"J:\projects\ECOFOR\field\gedi_jan23", r"J:\projects\ECOFOR\field\gedi_may23"]
dfs = []
for ddir in ddirs:
    pdf = pd.read_csv(os.path.join(ddir, "gedi_plot.csv"), dtype={'gedi':str})
    cdf = pd.read_csv(os.path.join(ddir, "gedi_cover.csv"))
    cols = cdf.columns.difference(pdf.columns.drop(['plot']))
    
    mdf = pd.merge(pdf, cdf[cols], on='plot', how='right')
    mdf['visit'] = ddir[-5:]
    
    dfs.append(mdf)
    
df = pd.concat(dfs)

In [ ]:
df = df.sort_values(['visit', 'plot'])
df['plot_ix'] = pd.factorize(df['visit']+df['plot'].astype(str))[0]

In [ ]:
# Save just the cover data
cols = ['plot_ix', 'type', 'species', 'cover', 'notes',                                                                       # plot identifier and plot cover values
        'visit', 'plot', 'gedi', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables

df[cols].to_csv(r"J:\projects\ECOFOR\field\field_cover_merged.csv", index=False)

**Export overall cover with GEDI**

In [ ]:
# reshape overall cover only
df = df[df['type']=='overall']
wdf = df.pivot(index='plot_ix', columns='species', values='cover')
wdf['total'] = wdf.sum(axis=1)
plot_df = df.drop(columns=['type', 'species', 'cover', 'notes']).drop_duplicates('plot_ix')
wdf = pd.merge(wdf, plot_df, how='left', left_index=True, right_on='plot_ix')

In [ ]:
# merge in GEDI data
cdf = pd.merge(wdf, gdf, how='left', on='gedi')
cdf = gpd.GeoDataFrame(cdf, geometry='geometry', crs=gdf.crs)
cdf['date'] = cdf['date'].astype(str)
cdf['delta_time'] = cdf['delta_time'].astype(str)

cols = ['plot_ix', 'tree', 'shrub', 'herb', 'soil','litter', 'rock',  'other', 'total',                                                     # plot identifier and plot cover values
        'visit', 'plot', 'gedi', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables
cdf = cdf[cols + list(gdf.columns.drop('gedi'))]

In [ ]:
cdf.to_file(r"J:\projects\ECOFOR\field\gedi_all_merged.gpkg", layer="cover", driver="GPKG")

In [ ]:
cdf.drop(columns=['geometry']).to_csv(r"J:\projects\ECOFOR\field\gedi_cover_merged.csv", index=False)

#### Merged and simplified  
Merge tree and cover data and simplify it for use by UBC students

In [ ]:
# Extract quadrant with the tallest tree
tallest_ix = tdf.groupby('plot_ix')['hgt'].idxmax().dropna()
tcols = ['plot_ix', 'quadrant', 'az', 'dist', 'hgt', 'species', 'live', 'pos', 'bole', 'notes']
talldf = tdf.loc[tallest_ix, tcols]

# Cover columns
cols = ['plot_ix', 'tree', 'shrub', 'herb', 'soil','litter', 'rock',  'other', 'total',                                                     # plot identifier and plot cover values
        'cover', 'rh98', 'pai', 'elev_lowestmode', 'lat_lowestmode', 'lon_lowestmode', 'l2b_quality_flag', 'sensitivity',                   # GEDI variables
        'visit', 'plot', 'gedi', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables

simpdf = pd.merge(talldf, cdf[cols], on='plot_ix', how='right')

simpdf.to_csv(r"J:\projects\ECOFOR\field\gedi_trees_cover_simp.csv")

### 2024 data prep
Get metrics for GEDI field plots for David and Logan

In [ ]:
# Quick merge
path = r"C:\scratch\ECOFOR\gedi\GEDI_2AB_2019to2023.parquet"
df = gpd.read_parquet(path)
df['GEDI #'] = df['shot_number'].astype(np.uint64)

field_path = r"J:\projects\ECOFOR\field\gedi_may24\GEDI_CONS454samples_final.xlsx"
fdf = pd.read_excel(field_path)

cols = ['GEDI #', 'delta_time', 'rh98', 'lat_lowestmode', 'lon_lowestmode', 'elev_lowestmode', 'fhd_normal', 'pai', 'cover', 'rh98']
mdf = pd.merge(fdf, df[cols], on = 'GEDI #')
mdf.to_excel(field_path[:-5]+'_wGEDI.xlsx', sheet_name='PrimaryVeg', index=False)

### Combine 2023 and 2024 field data

#### GEDI trees
Merge tree and plot data from 2023 and 2024 visits

In [ ]:
# Load GEDI data
cols = ['shot_number', 'delta_time', 'cover', 'rh98', 'pai', 'elev_lowestmode', 'lat_lowestmode', 'lon_lowestmode', 'sensitivity', 'geometry']
gdf = gpd.read_parquet(r"J:\projects\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2AB_2019to2023.parquet", columns=cols)

In [ ]:
ddirs = [r"J:\projects\ECOFOR\field\gedi_jan23", r"J:\projects\ECOFOR\field\gedi_may23", r"J:\projects\ECOFOR\field\gedi_may24"]
dfs = []
for ddir in ddirs:
    pdf = pd.read_csv(os.path.join(ddir, "gedi_plot.csv"), dtype={'gedi':str, 'shot_number':str})
    tdf = pd.read_csv(os.path.join(ddir, "gedi_trees.csv"))
    cols = tdf.columns.difference(pdf.columns.drop(['plot']))
    visit = ddir[-5:]
    # need to account for multiple groups measuring same plot in merge
    if visit=='may24':
        cols = cols.tolist()+['group']
        mdf = pd.merge(pdf, tdf[cols], on=['group','plot'], how='right')
    else:
        mdf = pd.merge(pdf, tdf[cols], on='plot', how='right')
        mdf['group'] = 0
        mdf['shot_number'] = mdf['gedi'].astype(str)
    mdf['visit'] = visit
    dfs.append(mdf)
    
df = pd.concat(dfs)

df = df.sort_values(['visit', 'plot', 'group'])
df['plot_ix'] = pd.factorize(df['visit']+ df['plot'].astype(str) + df['group'].astype(str))[0]

In [ ]:
# Get quadrant

# def get_az_quadrant(az):
#     if np.isnan(az):
#         return None
#     else:
#         labels = {0:'ne', 1:'se', 2:'sw', 3:'nw'}
#         return labels[int(az//90)]

# df['quadrant'] = df['az'].apply(get_az_quadrant)

df['quadrant'] = df['Direction'].str.lower()
df['quadrant'] = df['quadrant'].fillna(df['num'].replace({1:'ne', 2:'se', 3:'sw', 4:'nw'}))

In [ ]:
# Save just the trees data
cols = ['plot_ix', 'visit', 'plot', 'group', 'quadrant', 'az', 'dist', 'hgt', 'species', 'live', 'pos', 'bole', 'notes',                              # plot and tree identifier, and tree values
         'shot_number', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables

df[cols].to_csv(r"J:\projects\ECOFOR\field\merged\field_trees_merged.csv", index=False)

In [ ]:
# merge in GEDI data
tdf = pd.merge(df, gdf, how='left', on='shot_number')
tdf = gpd.GeoDataFrame(tdf, geometry='geometry', crs=gdf.crs)
tdf['date'] = tdf['date'].astype(str)
tdf['delta_time'] = tdf['delta_time'].astype(str)
tdf = tdf[cols + list(gdf.columns.drop('shot_number'))]

In [ ]:
# Project tree coordinates but save plot coordinates
tdf['geo_utm36n'] = tdf['geometry'].to_crs(epsg = 32636)

def get_tree_coords(r):
    if np.isnan(r['az']) or np.isnan(r['dist']) or (r['geo_utm36n'] is None):
        return None
    rad = np.radians(r['az'])
    dx = r['dist'] * np.sin(rad)
    dy = r['dist']* np.cos(rad)
    return shapely.affinity.translate(r['geo_utm36n'], dx, dy)

tdf['tree_geo'] = tdf.apply(get_tree_coords, axis=1)
tdf['geometry'] = tdf['tree_geo'].set_crs(tdf['geo_utm36n'].crs).to_crs(tdf.crs)
tdf = tdf.drop(columns = ['geo_utm36n', 'tree_geo'])

In [ ]:
tdf.to_file(r"J:\projects\ECOFOR\field\merged\gedi_all_merged.gpkg", layer="trees", driver="GPKG")

In [ ]:
tdf.drop(columns='geometry').to_csv(r"J:\projects\ECOFOR\field\merged\gedi_trees_merged.csv", index=False)

#### GEDI cover

In [ ]:
ddirs = [r"J:\projects\ECOFOR\field\gedi_jan23", r"J:\projects\ECOFOR\field\gedi_may23", r"J:\projects\ECOFOR\field\gedi_may24"]
dfs = []
for ddir in ddirs:
    pdf = pd.read_csv(os.path.join(ddir, "gedi_plot.csv"), dtype={'gedi':str, 'shot_number':str})
    cdf = pd.read_csv(os.path.join(ddir, "gedi_cover.csv"))
    cols = cdf.columns.difference(pdf.columns.drop(['plot']))
    visit = ddir[-5:]
    # need to account for multiple groups measuring same plot in merge
    if visit=='may24':
        cols = cols.tolist()+['group']
        mdf = pd.merge(pdf, cdf[cols], on=['group','plot'], how='right')
    else:
        mdf = pd.merge(pdf, cdf[cols], on='plot', how='right')
        mdf['group'] = 0
        mdf['shot_number'] = mdf['gedi'].astype(str)
    mdf['visit'] = visit
    
    
    dfs.append(mdf)
    
df = pd.concat(dfs)

df = df.sort_values(['visit', 'plot', 'group'])
df['plot_ix'] = pd.factorize(df['visit'] + df['plot'].astype(str) + df['group'].astype(str))[0]

In [ ]:
# Save just the cover data
cols = ['plot_ix', 'visit', 'plot', 'group', 'type', 'species', 'cover', 'notes',                                                                       # plot identifier and plot cover values
        'shot_number', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables

df[cols].to_csv(r"J:\projects\ECOFOR\field\merged\field_cover_merged.csv", index=False)

#### Export overall cover with GEDI

In [ ]:
# reshape overall cover only
df = df[df['type']=='overall']

wdf = df.pivot(index=['plot_ix', 'group'], columns='species', values='cover')
wdf['total'] = wdf.sum(axis=1)
plot_df = df.drop(columns=['type', 'species', 'cover', 'notes']).drop_duplicates(['plot_ix', 'group']).reset_index()
wdf = pd.merge(wdf, plot_df, how='left', on=['plot_ix', 'group'])

In [ ]:
# merge in GEDI data
cdf = pd.merge(wdf, gdf, how='left', on='shot_number')
cdf = gpd.GeoDataFrame(cdf, geometry='geometry', crs=gdf.crs)
cdf['date'] = cdf['date'].astype(str)
cdf['delta_time'] = cdf['delta_time'].astype(str)

cols = ['plot_ix', 'visit', 'group', 'plot', 'tree', 'shrub', 'herb', 'soil','litter', 'rock',  'other', 'total',                                                     # plot identifier and plot cover values
        'shot_number', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables
cdf = cdf[cols + list(gdf.columns.drop('shot_number'))]

In [ ]:
cdf.to_file(r"J:\projects\ECOFOR\field\merged\gedi_all_merged.gpkg", layer="cover", driver="GPKG")

In [ ]:
cdf.drop(columns=['geometry']).to_csv(r"J:\projects\ECOFOR\field\merged\gedi_cover_merged.csv", index=False)

#### Merged and simplified  
Merge tree and cover data and simplify it for use by UBC students

In [ ]:
tdf = gpd.read_file(r"J:\projects\ECOFOR\field\merged\gedi_all_merged.gpkg", layer="trees")
cdf = gpd.read_file(r"J:\projects\ECOFOR\field\merged\gedi_all_merged.gpkg", layer="cover")

# Extract quadrant with the tallest tree
tallest_ix = tdf.groupby('plot_ix')['hgt'].idxmax().dropna()
tcols = ['plot_ix', 'quadrant', 'az', 'dist', 'hgt', 'species', 'live', 'pos', 'bole', 'notes']
talldf = tdf.loc[tallest_ix, tcols]

# Cover columns
cols = ['plot_ix', 'visit', 'plot', 'group', 'tree', 'shrub', 'herb', 'soil','litter', 'rock',  'other', 'total',                                                     # plot identifier and plot cover values
        'cover', 'rh98', 'pai', 'elev_lowestmode', 'lat_lowestmode', 'lon_lowestmode', 'sensitivity', 'delta_time',                  # GEDI variables
        'shot_number', 'gps', 'camera', 'photo_num1', 'photo_num2', 'recorder', 'heights', 'photos', 'distances', 'plot_notes']   # other plot variables

simpdf = pd.merge(talldf, cdf[cols], on='plot_ix', how='right')

# fill hgt with 0 if null because these are plots with no trees
simpdf.loc[simpdf['hgt'].isnull(), 'hgt'] = 0

simpdf.to_csv(r"J:\projects\ECOFOR\field\merged\gedi_trees_cover_simp.csv", index=False)

In [ ]:
pcols = ['plot_ix', 'visit', 'plot', 'group', 'shot_number', 'delta_time', 'gps', 'camera', 'plot_notes']
pdf = simpdf[pcols]
pdf['shot_number'] = pdf['shot_number'].astype(str)
# pdf.to_csv(r"J:\projects\ECOFOR\field\merged\plot_notes.csv", index=False)

# Need to IMPORT this into excel and tell it to NOT automatically detect data types in order to add the 
# exclude_plot columns without messing up the existing data types

## Compare field and RS data  
Compare the field measurements to GEDI footprints and predicted maps.

In [ ]:
def present_regplot(x, y, ax, lims=None, reg=True, oneone=True, **kwargs):
    from scipy.stats import linregress
    
    sns.regplot(x=x, y=y, ax=ax, **kwargs)

    if reg:
        # Regression
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        rmse = mean_squared_error(y, x*slope+intercept)**0.5
        eq = "y = " + str(np.round(slope,2)) + "x + " + str(np.round(intercept, 2))
        ax.text(0.05, 0.95, "Regression:", transform=ax.transAxes)
        ax.text(0.05, 0.89, eq, transform=ax.transAxes)
        ax.text(0.05, 0.83, "R$^2$= " + str(np.round(r_value**2, 2)), transform=ax.transAxes)
        ax.text(0.05, 0.77, "RMSE= "+str(np.round(rmse, 2)), transform=ax.transAxes)

#         # check that line is the same as from sns.reglot
#         samp = np.arange(x.min(), x.max(), 1)
#         ax.plot(samp, intercept + slope * samp, 'r')

    if oneone:
        if lims is None:
            lims = (0, np.nanmax(x.append(y))) #np.nanmin(x.append(y))
        ax.plot(lims, lims, '--k')
        ax.set(ylim=lims, xlim=lims)

        # add text for R2 and RMSE
        r2 = r2_score(y, x)
        rmse = mean_squared_error(y, x)**0.5
        bias = (x-y).mean()
        ax.text(0.97, 0.20, "1:1 stats:", transform=ax.transAxes, ha='right')
        ax.text(0.98, 0.13,"R$^2$= "+str(np.round(r2, 2)), transform=ax.transAxes, ha='right')
        ax.text(0.98, 0.07, "RMSE= "+str(np.round(rmse, 2)), transform=ax.transAxes, ha='right')
        ax.text(0.98, 0.01, "Bias= "+str(np.round(bias, 2)), transform=ax.transAxes, ha='right')

In [ ]:
# Plot max tree height compared to GEDI's RH98
path = r"J:\projects\ECOFOR\field\merged\gedi_trees_cover_simp.csv"
df = pd.read_csv(path, index_col='plot_ix')

# Load plot notes to drop bad plots
path = r"J:\projects\ECOFOR\field\merged\plot_notes.csv"
pdf = pd.read_csv(path, index_col='plot_ix')

df[['exclude_plot', 'exclude_reason']] = pdf[['exclude_plot', 'exclude_reason']]
df = df[~df['exclude_plot']]

In [ ]:
# Accuracy by field visit
pcols=df['visit'].unique()
# title_dict={'juoc':'Western Juniper', 'pimo': 'Single-leaf Pinyon Pine', 'juos':'Utah Juniper'}
fig, axes = plt.subplots(1, 3, figsize=(8.7, 3))
for i, (sp, ax) in enumerate(zip(pcols, axes.flat)):
    mask = df['visit']==sp
    x = df.loc[mask, 'rh98']
    y = df.loc[mask, 'hgt']
    present_regplot(x, y, ax, scatter_kws={'alpha':0.4})
    ax.set(xlabel='RH98',
           ylabel='Field max height',
           title=sp,
           xlim = (-1, 30),
           ylim= (-1, 30))
#     ax.text(0.97, 0.21, "1:1 stats:", transform=ax.transAxes, ha='right')
    if i!=0:
        ax.set_ylabel('')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
present_regplot(df['rh98'], df['hgt'], lims=(0,30), ax=ax)

# Airborne lidar

In [ ]:
import rasterio.features

## 2014 cover

In [ ]:
# extract lidar cover band from layer stack
path = r"K:\ECOFOR\lidar\2014\orig\S1_A_VH_VV_16_17_lidar.dat"
with rasterio.open(path) as src:
    arr = src.read(src.indexes[-1])
    crs = src.crs
    transform = src.transform

arr[(arr<0) | (arr>100)] = np.nan

new_prof = {'driver':'GTiff', 'dtype': 'float32', 'nodata': np.nan, 'width': 681, 'height': 3293,
            'count': 1, 'crs': crs, 'transform': transform,  'blockxsize': 256, 'blockysize': 256,
            'tiled': True, 'interleave': 'pixel', 'compress': 'lzw'}

with rasterio.open(r"K:\ECOFOR\lidar\2014\lidar_cover_2014.tif", 'w', **new_prof) as dst:
    dst.write(arr, 1)

In [ ]:
# Get bounds of 2012 airborne lidar data for clipping GEDI points in exactextractr script
path = r"K:\ECOFOR\lidar\2014\lidar_cover_2014.tif"
with rasterio.open(path) as src:
    arr = src.read(1)

msk = (arr >= 0) & (arr<=100)

shapes = rasterio.features.shapes(msk.astype(np.uint8), mask=msk, connectivity=4, transform = src.transform)

rows = []
for shp in shapes:
    row = {'value':shp[1], 'geometry':shapely.geometry.shape(shp[0])}
    rows.append(row)

df = gpd.GeoDataFrame(rows, crs=src.crs)

ldf = gpd.GeoDataFrame(geometry=[df.unary_union], crs=src.crs)
ldf.to_file(r"K:\ECOFOR\lidar\2014\lidar_cover_2014_bounds.gpkg", driver="GPKG")

In [ ]:
# Load data after run through exactextractR
path = "K:\ECOFOR\gedi\extracted_spectral_data\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_lidar14.gpkg"
df = gpd.read_file(path)
df['delta_time'] = pd.to_datetime(df['delta_time'])

df['month'] = df['delta_time'].dt.month

In [ ]:
# try only wet season
mask = df['month'].isin([11, 12, 1, 2, 3])

fig, ax = plt.subplots(figsize=(5,5))

sns.scatterplot(df.loc[mask, 'cover']*100, df.loc[mask, 'mean'], ax=ax)
ax.set(ylabel='2014 airborne cover %', xlabel='GEDI cover %')

## 2012 lidar points

### Prep ALS data
Notes
1. The vertical epsg may not be 100% correct but is the closest datum I could find with a code  
2. The Orion M200 collects 4 returns per pulse, but the data has no return number.  
3. There are a couple tiles missing in the southern river, but they're missing from both ground and veg points so it can be considered an excluded area.  

*The processing pipeline below was longer and more complicated than I remember from previous lidar work. Need to implement this in lidr or pdal to avoid error introduced by non-commercial lastools.*  
*Note that the data does contain noise and I didn't run lasnoise.*

**Merge points using existing ground classification**

In [ ]:
os.environ['USE_PYGEOS'] = '1'
import importlib
importlib.reload(gpd)

In [ ]:
# Merge ground points and label as ground

indir = r"K:\ECOFOR\lidar\2012\orig\Points\Points\Ground_Points"
outpath = r"K:\ECOFOR\lidar\2012\lidar_2012_ground.laz"
lastools = r"C:\Program Files\LAStools\bin"
cmd = lastools + r"\lasmerge -v -i " + indir + r"\*.txt -iparse xyzi -set_classification 2 -epsg 32736 -vertical_epsg 9279 -cpu64 -o " + outpath
print(cmd)

In [ ]:
# Merge vegetation points
paths = glob(r"K:\ECOFOR\lidar\2012\orig\Points\Points\Veg*\*.txt")
with open(r"K:\ECOFOR\lidar\2012\veg_lof.txt", 'w') as f:
    f.write("\n".join(paths))

lof_path = r"K:\ECOFOR\lidar\2012\veg_lof.txt"
outpath = r"K:\ECOFOR\lidar\2012\lidar_2012_veg.laz"
cmd = lastools + r"\lasmerge -v -lof " + lof_path + " -iparse xyzi -set_classification 1 -epsg 32736 -vertical_epsg 9279 -cpu64 -o " + outpath
print(cmd)

In [ ]:
# Merge ground and veg points
veg_path = r"K:\ECOFOR\lidar\2012\lidar_2012_veg.laz"
ground_path = r"K:\ECOFOR\lidar\2012\lidar_2012_ground.laz"
outpath = r"K:\ECOFOR\lidar\2012\lidar_2012_all.laz"
cmd = lastools + r"\lasmerge -v -i " + veg_path + " " + ground_path + r" -cpu64 -o " + outpath
print(cmd)

**Normalize points to height above ground**

In [ ]:
# Tile for lasheight
path = r"K:\ECOFOR\lidar\2012\lidar_2012_all.laz"
odir = r"K:\ECOFOR\lidar\2012\tiles"
cmd = lastools + "\lastile -i " + path + " -tile_size 5000 -olaz -odir " + odir
print(cmd)

In [ ]:
# Lasheight
iglob = r"K:\ECOFOR\lidar\2012\tiles\*.laz"
odir = r"K:\ECOFOR\lidar\2012\norm_tiles"
cmd = lastools + "\lasheight -i " + iglob + " -buffered 25 -replace_z -cores 6 -olaz -odir " + odir
print(cmd)

In [ ]:
# merge normalized points
iglob = r"K:\ECOFOR\lidar\2012\tiles_norm\*.laz"
outpath = r"K:\ECOFOR\lidar\2012\lidar_2012_all_norm.laz"
cmd = lastools + "\lasmerge -i " + iglob + " -cpu64 -o " + outpath
print(cmd)

**height in extra_bytes**  
gediSimulator readme recommends reclassifying all points within 60 cm of the ground as ground points for improved cover calculation.

In [ ]:
# Lasheight
iglob = r"K:\ECOFOR\lidar\2012\tiles\*.laz"
odir = r"K:\ECOFOR\lidar\2012\tiles_height"
cmd = lastools + "\lasheight -i " + iglob + " -buffered 25 -store_as_extra_bytes -cores 6 -olaz -odir " + odir
print(cmd)

### Compare discrete ALS metrics with GEDI

**Get canopy metrics for GEDI footprints**

In [ ]:
# Output boundary of points for clipping GEDI data
path = r"K:\ECOFOR\lidar\2012\lidar_2012_all.laz"
outpath = r"K:\ECOFOR\lidar\2012\lidar_2012_all_boundary.shp"
cmd = lastools + "\lasboundary -v -i "+ path + " -concavity 20 -disjoint_hull -o " + outpath
print(cmd)

In [ ]:
# Clip GEDI to the lidar data and buffer to create plots
boundary_path = r"K:\ECOFOR\lidar\2012\lidar_2012_all_boundary.shp"
gedi_path = r"K:\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet"
temp_path = r"K:\ECOFOR\gedi\extracted_spectral_data\GEDI_2012_lidar_temp.shp"

bdf = gpd.read_file(boundary_path)
gdf = gpd.read_parquet(gedi_path)

# remove small chunks
bdf['area'] = bdf.area/(1000*1000)
bdf = bdf[bdf['area']>1]

# Get points within lidar extent
gdf = gdf.to_crs(bdf.crs)
gdf = gdf[gdf.intersects(bdf.unary_union)]

# Buffer and get GEDI shots completely within lidar extent
gdf['geometry'] = gdf.buffer(12.5)
gdf = gdf[gdf.within(bdf.unary_union)]

gdf['delta_time'] = gdf['delta_time'].to_string()
gdf.to_file(temp_path, index=False)

In [ ]:
# Get canopy metrics (this isn't merging into one output but writing csvs in the input folder, but it's much faster than using a single input)
path = r"K:\ECOFOR\lidar\2012\tiles_norm\*.laz"
temp_path = r"K:\ECOFOR\gedi\extracted_spectral_data\GEDI_2012_lidar_temp.shp"
outpath = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_lascanopy.csv"
cmd = lastools + "\lascanopy -i " + path + " -lop " + temp_path + " shot_number -cover_cutoff 0.5 -height_cutoff 0 -p 50 75 95 98 -avg -max -dns -d 0.5 1 2 3 5 100 -cores 6 -o " + outpath
print(cmd)

In [ ]:
# merge the results into one
paths = glob(r"K:\ECOFOR\lidar\2012\tiles_norm\*.csv")
outpath = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_lascanopy.csv"
df = pd.concat([pd.read_csv(p) for p in paths])
df.to_csv(outpath, index=False)

**ALS discrete metrics comparison**

In [ ]:
lpath = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.csv"
gpath = r"K:\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet"

ldf = pd.read_csv(lpath)
ldf = ldf.rename(columns={'name':'shot_number'})
ldf = ldf[['shot_number', 'max', 'avg', 'p50', 'p75', 'p95', 'p98', 'd00', 'd01', 'd02', 'd03', 'd04', 'dns']]
ldf['d01_04'] = ldf['d01'] + ldf['d02'] + ldf['d02'] + ldf['d02'] # makes height cutoff 1 m
ldf['d01_

gdf = gpd.read_parquet(gpath)
gdf['shot_number'] = gdf['shot_number'].astype(np.uint64)
gdf['cover'] *= 100

df = gpd.GeoDataFrame(pd.merge(ldf, gdf, 'left',  on='shot_number'), crs=gdf.crs)

df['delta_time'] = pd.to_datetime(df['delta_time'])
df['month'] = df['delta_time'].dt.month

# mask = df['month'].isin([11, 12, 1, 2, 3]) # These are wet seasons months
mask = df['month'].isin([5,6,7,8,9])       # These are dry season months. Lidar data collected May 30 2012 so this is more appropriate.

In [ ]:
# Top of canopy height
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot('p98', 'rh98', data=df[mask], ax=ax)
ax.set(xlim=(0,40), ylim=(0,40))
ax.plot((0, 40), (0, 40), 'k--')

In [ ]:
# Cover with ALS height cutoff 50 cm
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot('cover', 'dns', data=df[mask], ax=ax)
ax.set(xlim=(0,100), ylim=(0,100))
ax.plot((0, 100), (0, 100), 'k--')

In [ ]:
# cover with als height cutoff 2 m
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot('cover', 'd01_02', data=df[mask], ax=ax)
ax.set(xlim=(0,100), ylim=(0,100))
ax.plot((0, 100), (0, 100), 'k--')

### Compare GEDI simulated waveforms with GEDI

**GEDI simulator waveforms**  
Prep ALS for the waveform simulator and run it to compare metrics

In [ ]:
# merge height and set <60cm high to class 2 for use in GEDI waveform simulator
iglob = r"K:\ECOFOR\lidar\2012\tiles_height\*.laz"
outpath = r"K:\ECOFOR\lidar\2012\lidar_2012_all_lt60grd.las"
cmd = lastools + "\lasmerge -i " + iglob + " -classify_attribute_below_as 0 0.6 2 -cpu64 -o " + outpath
print(cmd)

In [ ]:
# Get list of shots and files collected around the dry season when the 2012 ALS was collected (May 30)
gpath = r"K:\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet"
gdf = gpd.read_parquet(gpath)
gdf['shot_number'] = gdf['shot_number'].astype(np.uint64)
gdf['delta_time'] = pd.to_datetime(gdf['delta_time'])
gdf['month'] = gdf['delta_time'].dt.month
gdf['doy'] = gdf['delta_time'].dt.day_of_year
gdf['year'] = gdf['delta_time'].dt.year
gdf['ydoy'] = gdf['year'].astype(str) + gdf['doy'].astype(str).str.zfill(3)

# mask = df['month'].isin([11, 12, 1, 2, 3]) # These are wet seasons months
mask = gdf['month'].isin([5,6,7,8,9])       # These are dry season months. Lidar data collected May 30 2012 so this is more appropriate.
gdf = gdf[mask]

In [ ]:
# Get shots intersecting 2012 lidar
bdf = gpd.read_file(r"K:\ECOFOR\lidar\2012\lidar_2012_all_boundary.shp")
gdf = gdf.to_crs(bdf.crs)
gdf = gdf[gdf.intersects(bdf.unary_union)]

In [ ]:
# Get list of l1b granules to download
date_strings = gdf.loc[mask, 'ydoy'].unique()
paths = glob(r"K:\ECOFOR\gedi\gedi_data\02_download_files\01_GEDI_downloads_2B\*.h5")
l2b_granules = [p for p in paths for d in date_strings if d in p]
l2b_granules  # Manually filtered and downloaded corresponding L1B granules on EarthData Search

In [ ]:
# Output list of coordinates for running gediRAT and gediMetric without collocation
coord_strs = (gdf.geometry.x.astype(str) + ", " + gdf.geometry.y.astype(str) + ", " + gdf['shot_number'].astype(str)).tolist()
# coord_strs = coord_strs[:3]
with open(r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_coordslist.txt", "w") as f:
    f.write('\n'.join(coord_strs))

**Simulator Metrics without collocate**

In [ ]:
# Simulate waveforms for list of coordinates
lpath = "/mnt/k/ECOFOR/lidar/2012/lidar_2012_all_lt60grd.las"
coords_path = "/mnt/k/ECOFOR/lidar/2012/lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_coordslist.txt"
outpath = "/mnt/k/ECOFOR/lidar/2012/lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_sim.h5"
gedi_cmd = "gediRat -input " + lpath + " -listCoord " + coords_path + " -output " + outpath + " -hdf -ground"
sing_cmd = "singularity exec --bind /mnt/k gediSingularity "
cmd = sing_cmd + gedi_cmd 
print(cmd)

In [ ]:
# Metrics for simulated waveforms
simpath = "/mnt/k/ECOFOR/lidar/2012/lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_sim.h5"
outroot = "/mnt/k/ECOFOR/lidar/2012/lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_sim_canopy"

cmd = "gediMetric -input " + simpath + " -readHDFgedi -ground -outRoot " + outroot
sing_cmd = "singularity exec --bind /mnt/k gediSingularity "
print(sing_cmd + cmd)

**Collocate Waves Metrics**  
Run collocate waves with one granule at a time so the affine transform is done for each granule not all granules together. This assumes that GEDI shots near in time have similar geolocation error. If that's not true then collocate waves would need to be done per footprint.

In [ ]:
# Calculate expected offset between ALS and GEDI vertical datums

# Read simulated metrics from original shot location (no collocation)
sim_path = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_sim_canopy.metric.txt"
odf = pd.read_csv(sim_path, skiprows=1, header=None, delimiter=" ")

# Set column headers
with open(sim_path) as f:
    header = f.readline()
header = header[1:] # drop #
hlist = header.split(',')
hlist = ['_'.join(c.split(' ')[2:]) for c in hlist] # drop number and space
hlist = hlist[:-1] # drop end of line

odf.columns = hlist
odf = odf.rename(columns={'wave_ID':'shot_number'})

# Load prepared GEDI canopy table
gpath = r"K:\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet"
gdf = gpd.read_parquet(gpath)
gdf['shot_number'] = gdf['shot_number'].astype(np.uint64)

mdf = pd.merge(gdf, odf, how='inner', on='shot_number', suffixes=('', '_simN'))

dif = mdf['true_ground'] - mdf['elev_lowestmode'] 
print('offset=', np.median(dif))

In [ ]:
# run collocatewaves on all granules (do this in python session started in WSL)
# I think this will find one tranformation for each gedi file rather than each footprint
# Using -listGedi I think finds one transformation for all GEDI files together. Need to check with with Hancock.
import os, subprocess, shutil
from glob import glob
paths = glob("/mnt/k/ECOFOR/gedi/gedi_data/02_download_files/GEDI_downloads_1B/*.h5")
# paths = paths[1:2]
for gpath in paths:
    lpath = "/mnt/k/ECOFOR/lidar/2012/lidar_2012_all_lt60grd.las"
    outname = "lidar2012sim_" + os.path.basename(gpath)
    outpath = os.path.join("/mnt/k/ECOFOR/lidar/2012/gedi_collocatewaves", outname)
    if os.path.exists(outpath):
        print(outpath, 'already exists. Skipping.\n')
        continue

    bounds = "349954 7213558 402632 7370194"
    suggested_settings = "-solveCofG -geoError 30 5 -fixFsig -minDense 3 -minSense 0.9"
    gedi_cmd = "collocateWaves -als " + lpath + " -gedi " + gpath + " -writeWaves " + outpath + " -readHDFgedi -aEPSG 32736 -offset -13.84 -bounds " + bounds + " -checkCover -skipBeams 1234 " + suggested_settings
    sing_cmd = "singularity exec --bind /mnt/k gediSingularity " 
    cmd = sing_cmd + gedi_cmd 
    print(cmd)
    completed = subprocess.run(cmd.split(' '))
    shutil.move('teast.correl', outpath[:-3]+".correl")

In [ ]:
# Check the transformation and correlation of each granule
paths = glob(r"K:\ECOFOR\lidar\2012\gedi_collocatewaves\*.correl")
df = pd.concat([pd.read_csv(p, skiprows=1, header=None, delimiter=" ").assign(fname=os.path.basename(p)) for p in paths])

# Set column headers
with open(paths[0]) as f:
    header = f.readline().replace('\n', '')
hlist = header[1:].split(',') # drop # and split
hlist = ['_'.join(c.split(' ')[2:]) for c in hlist] # drop number and space
df.columns = hlist + ['fname']

df

Correlation should be >0.7 and delta_CofG should be close to zero. A big difference here could indicate a poor fit or something wrong with georegisration like using different vertical datums.  
Some files failed to produce correlation files.   

Drop lidar2012sim_GEDI01_B_2020125185739_O07897_01_T03482_02_005_01_V002 during analysis because of weird fit error.

In [ ]:
# Run canopy metrics on all granules
# Note gediMetric -inList not working, only runs first file
paths = glob("/mnt/k/ECOFOR/lidar/2012/gedi_collocatewaves/*.h5")
for path in paths:
    outroot = os.path.splitext(path)[0] + "_canopy"
    gedi_cmd = "gediMetric -input " + path + " -readHDFgedi -ground  -outRoot " + outroot
    sing_cmd = "singularity exec --bind /mnt/k gediSingularity "
    cmd = sing_cmd + gedi_cmd 
    print(cmd)
    completed = subprocess.run(cmd.split(' '))

In [ ]:
# Read all collocated simulated metrics data and merge
sim_paths = glob(r"K:\ECOFOR\lidar\2012\gedi_collocatewaves\*metric.txt")
outpath = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_collocate_canopy.metric.csv"

sdf = pd.concat([pd.read_csv(p, skiprows=1, header=None, delimiter=" ") for p in sim_paths])

# Set column headers
with open(sim_paths[0]) as f:
    header = f.readline()
header = header[1:] # drop #
hlist = header.split(',')
hlist = ['_'.join(c.split(' ')[2:]) for c in hlist] # drop number and space
hlist = hlist[:-1] # drop end of line

sdf.columns = hlist

# split wave_id into constituent parts
sdf['beam'] = sdf['wave_ID'].str.split('.').str[1]
sdf['shot_number'] = sdf['wave_ID'].str.split('.').str[2].astype(np.uint64)
sdf['x'] = sdf['wave_ID'].str.split('.').str[3]
sdf['y'] = sdf['wave_ID'].str.split('.').str[4]

# modify metrics
sdf['cover'] *= 100
sdf['ALS_cover'] *= 100

sdf.to_csv(outpath, index=False)

In [ ]:
# Load and plot simulated waveform data
sys.path.append(r"J:\users\stevenf\code\fork\gedisimulator")
from gediHandler import gediData

gdat = gediData(r"K:\ECOFOR\lidar\2012\gedi_collocatewaves\lidar2012sim_GEDI01_B_2019174002232_O02984_01_T02212_02_005_01_V002.h5")

def plotSimWave(gdat, i):
    '''Plot waveforms from a simulated GEDI file'''
    fig, ax = plt.subplots(figsize=(4,4))
    # make z profile
    gdat.res=(gdat.Z0[i]-gdat.ZN[i])/(gdat.nBins-1)
    gdat.z=np.linspace(gdat.Z0[i],gdat.ZN[i],num=gdat.nBins)

    # determine noise for scaling ground return
    reflScale,meanN,stdev=gdat.meanNoise(i,statsLen=10)
    # find bounds
    minX,maxX=gdat.findBounds(meanN,stdev,i)
    # plot it
    #plt.plot(gdat.wave[i],gdat.z,label='Waveform')
    #plt.plot(gdat.gWave[i]*reflScale+meanN,z,label='Ground')
    ax.fill_betweenx(gdat.z,gdat.wave[i],meanN)
    #plt.legend()
    #plt.xlim(left=0)
    ax.set(ylim=(minX,maxX), ylabel='Elevation (m)')
    #plt.xlabel('DN')
#     ax.ylabel('Elevation (m)')
    return ax

ax = plotSimWave(gdat, 18)
ax.set(ylim=(210, 220))

**Plot comparison**

In [ ]:
# Load prepared GEDI canopy table
gpath = r"K:\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31.parquet"

gdf = gpd.read_parquet(gpath)
gdf['shot_number'] = gdf['shot_number'].astype(np.uint64)
gdf['cover'] *= 100

In [ ]:
# Read simulated metrics from collocate waves
cdf = pd.read_csv(r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_collocate_canopy.metric.csv")
cdf['FHD_bfix'] = cdf['FHD']-2

# Read simulated metrics from original shot location (no collocation)
sim_path = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_sim_canopy.metric.txt"
ndf = pd.read_csv(sim_path, skiprows=1, header=None, delimiter=" ")

# Set column headers
with open(sim_path) as f:
    header = f.readline()
header = header[1:] # drop #
hlist = header.split(',')
hlist = ['_'.join(c.split(' ')[2:]) for c in hlist] # drop number and space
hlist = hlist[:-1] # drop end of line

ndf.columns = hlist
ndf = ndf.rename(columns={'wave_ID':'shot_number'})

# modify metrics
ndf['cover'] *= 100
ndf['ALS_cover'] *= 100
ndf['FHD_bfix'] = ndf['FHD']-2

In [ ]:
# Discrete lidar metrics (but not at the colocated positions)
lpath = r"K:\ECOFOR\lidar\2012\lidar12_GEDI_2A_2B_merged_filtered_2019-01-01_2022-12-31_lascanopy.csv"

ldf = pd.read_csv(lpath)
ldf = ldf.rename(columns={'name':'shot_number'})
ldf = ldf[['shot_number', 'max', 'avg', 'p50', 'p75', 'p95', 'p98', 'd00', 'd01', 'd02', 'd03', 'd04', 'dns']]
# Density with different height cutoffs
ldf['d05m'] = ldf[['d00', 'd01', 'd02', 'd03', 'd04']].sum(axis=1)  # makes height cutoff 0.5 m
ldf['d1m'] = ldf[['d01', 'd02', 'd03', 'd04']].sum(axis=1)  # makes height cutoff 1 m
ldf['d2m'] = ldf[['d02', 'd03', 'd04']].sum(axis=1)  # makes height cutoff 2 m
ldf['d3m'] = ldf[['d03', 'd04']].sum(axis=1)  # makes height cutoff 3 m
ldf['d5m'] = ldf['d04']  # makes height cutoff 5 m

In [ ]:
# Merge sources
mdf = pd.merge(gdf.add_prefix('gedi_'), ndf.add_prefix('nsim_'), how='inner', left_on='gedi_shot_number', right_on='nsim_shot_number')
mdf = pd.merge(mdf, cdf.add_prefix('csim_'), how='left', left_on='gedi_shot_number', right_on='csim_shot_number')
mdf = pd.merge(mdf, ldf.add_prefix('als_'), how='left', left_on='gedi_shot_number', right_on='als_shot_number')

In [ ]:
def op_plot(xcol, ycol, hexplot=True, vmax=10):
    fig, ax = plt.subplots(figsize=(4,4))
    sub = mdf[[xcol, ycol]].dropna()
    x, y = sub[xcol], sub[ycol]
    if hexplot:
        hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=vmax)
    else:
        sns.scatterplot(x=x, y=y, ax=ax)
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')
    axmin, axmax = sub.min().min(), sub.max().max()
    ax.set(xlabel=xcol, ylabel=ycol,
           xlim=(axmin, axmax), ylim=(axmin, axmax))
    
    r2 = r2_score(y, x)
    bias = (x-y).mean()
    rmse = mean_squared_error(y, x)**0.5
    
    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(r2.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')
    return fig

In [ ]:
# comparisons
# xcol, ycol = 'gedi_cover', 'simn_cover'
# xcol, ycol = 'gedi_cover', 'simn_ALS_cover'
# xcol, ycol = 'als_dns', 'nsim_cover'

# xcol, ycol = 'gedi_rh95', 'simn_rhGauss_95'

xcol, ycol = 'gedi_fhd_normal', 'nsim_FHD_bfix'

fig = op_plot(xcol, ycol, hexplot=False, vmax=100)

# Model GEDI Canopy Structure  
Use Landsat data to construct models of GEDI canopy height and structure metrics that can be used for mapping wall-to-wall.

In [ ]:
# Load and prep data
path = r"J:\projects\ECOFOR\gedi\extracted\GEDI_2AB_2019to2023_leafon_sampy500m_all.parquet"
df = gpd.read_parquet(path)

outbasedir = r"J:\projects\ECOFOR\gedi\models" #r"D:\ECOFOR\gedi\models" #r"C:\scratch\ECOFOR\gedi\models" #r"J:\projects\ECOFOR\corli_elephantimpacts" #
ver = "v10"
outdir = os.path.join(outbasedir, ver)
os.makedirs(outdir, exist_ok=True)
outbasename = os.path.splitext(os.path.basename(path))[0]

In [ ]:
# # Spatial filter for points intersecting the new study area for v6
# aoi_path = r"J:\projects\ECOFOR\boundaries\gknp_utm36n_v2.gpkg"
# aoi = gpd.read_file(aoi_path)[['geometry']]
# # df = df.to_crs(aoi.crs) # should already be the same
# df.sindex
# df = gpd.tools.sjoin(df, aoi, how='left', predicate='intersects') # spatial join much faster than .intersects which takes 6 minutes.
# df.dropna(subset=['index_right'], inplace=True)
# df.drop('index_right', axis=1, inplace=True)

In [ ]:
# # Spatial filter the points from the buffer aoi to those intersecting only the study area
# # TODO: Takes 37 seconds. Do this in GEDI prep next time.
# aoi_path = r"J:\projects\ECOFOR\boundaries\greaterkruger_utm36n.gpkg"
# aoi = gpd.read_file(aoi_path)[['geometry']]
# # df = df.to_crs(aoi.crs) # should already be the same
# df.sindex
# df = gpd.tools.sjoin(df, aoi, how='left', predicate='intersects') # spatial join much faster than .intersects which takes 6 minutes.
# df.dropna(subset=['index_right'], inplace=True)
# df.drop('index_right', axis=1, inplace=True)

In [ ]:
# # Add various radar vegetation indices for PALSAR 
# # Equations taken from table 1 of Hu et al., 2024.
# df['palsar_rc'] = df['palsar_HV'] / df['palsar_HH'] # cross-polarization ratio
# df['palsar_rvihh'] = 4*df['palsar_HV'] / (df['palsar_HH'] + df['palsar_HV'])
# df['palsar_rfdi'] = (df['palsar_HH'] - df['palsar_HV']) / (df['palsar_HH'] + df['palsar_HV'])

In [ ]:
cols = df.columns

Xcols = list(cols[cols.str.startswith('lt')]) # LandTrendr
bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'] #'ca', 
Xcols += [col for col in cols for src in ['hls', 'l30'] for band in bands if col.startswith(src) and (band in col)] # using CCDC
Xcols += ['palsar_HV', 'palsar_HH', 'palsar_angle'] #, 'palsar_rc', 'palsar_rvihh', 'palsar_rfdi'] 
Xcols += list(cols[cols.str.startswith('topo')])
Xcols += list(cols[cols.str.startswith('soil')])
# Xcols += list(cols[cols.str.startswith('ps')]) # PlanetScope
# Xcols += list(cols[cols.str.startswith('climate')])
# Xcols = list(cols[cols.str.startswith('dry') | cols.str.startswith('wet')])
# Xcols = list(cols[cols.str.startswith('dry')]) + ['HH', 'HV', 'angle']


# df['cover_gt5m'] = df['cover_z_5_10m']
# df['cover_lt5m'] = df['cover'] - df['cover_gt5m']

Ycols = ['cover', 'rh98', 'fhd_normal', 'pai']
# Ycols = ['cover', 'cover_gt5m', 'cover_lt5m']
meta_cols = ['delta_time', 'year', 'rain_year', 'elev_lowestmode', 'geometry']

# Remove NAN's in any cols (same data for each model)
mdf = df.dropna(subset=Xcols+Ycols)

# Filter unreasonable RH98 (none anyway)
mdf = mdf[mdf['rh98']<45]

# # Take a random sample for efficiency
# ## TODO: Run learning curves to see if there's a good number to keep and then use the rest of the data for testing
# mdf = mdf.sample(100000, random_state=0)

# Shuffle the data for use in cross-validation
mdf = mdf.sample(frac=1, random_state=0)

In [ ]:
Xcols_dict = {
    # 'lt': [col for col in Xcols if col.startswith('lt')],
    # 'ccdcl30': [col for col in Xcols if col.startswith('l30')],
    # 'ccdchls': [col for col in Xcols if col.startswith('hls')],
     'p': ['palsar_HV', 'palsar_HH', 'palsar_angle'],#, 'palsar_rc', 'palsar_rvihh', 'palsar_rfdi'],
# #     'ps': [col for col in Xcols if col.startswith('ps')],
    # 's-t': [col for col in Xcols for dstr in [ 'topo', 'soil'] if col.startswith(dstr)],
    # 'lt-p': [col for col in Xcols if col.startswith('lt')] + ['palsar_HV', 'palsar_HH', 'palsar_angle'],
    # 'ccdcl30-p': [col for col in Xcols if col.startswith('l30')] + ['palsar_HV', 'palsar_HH', 'palsar_angle'],
    # 'ccdchls-p': [col for col in Xcols if col.startswith('hls')] + ['palsar_HV', 'palsar_HH', 'palsar_angle'],
# #     'p-ps': [col for col in Xcols if col.startswith('ps')] + ['palsar_HV', 'palsar_HH', 'palsar_angle'],
    'lt-p-s-t': [col for col in Xcols for dstr in ['lt', 'palsar', 'topo', 'soil'] if col.startswith(dstr)],
    # 'ccdcl30-p-s-t': [col for col in Xcols for dstr in ['l30', 'palsar', 'topo', 'soil'] if col.startswith(dstr)],
    # 'ccdchls-p-s-t': [col for col in Xcols for dstr in ['hls', 'palsar', 'topo', 'soil'] if col.startswith(dstr)],
    # 'p-s-t': [col for col in Xcols for dstr in ['palsar', 'topo', 'soil'] if col.startswith(dstr)],
}

Xsets = Xcols_dict.keys()

## Random Forest

### Learning Curves  
Construct learning curves for a few metrics to get a sense of how many data points are needed to construct accurate models and the training/prediction time tradeoff.  

TODO: try to get random forest learning curves to work with out-of-bag accuracy and skip using CV because it's slower and not necessary and plain training accuracy is useless for RF. See the GLRI-TCC learning curve code for starting.

### Model with different predictor sets

In [ ]:
# Structures to hold predictions, feature importances, and model objects
pdf = mdf[meta_cols+Ycols].copy()
imp_dict = {}
model_dict = {}

for Xset, Xcols in Xcols_dict.items():
    # Run models
    X = mdf[Xcols]
    for ycol in Ycols:
        y = mdf[ycol]
        
        
        # # Get CV predictions
        # rf = RandomForestRegressor(n_estimators=200, max_features='sqrt', oob_score=False, random_state=0, n_jobs=20)
        # cv_preds = cross_val_predict(rf, X, y, cv=10, n_jobs=1)
        # pdf['pred_'+Xset+'_'+ycol] = pd.Series(cv_preds, index=y.index)
        
        # Get OOB predictions
        rf = RandomForestRegressor(n_estimators=100, max_features='sqrt', oob_score=True, random_state=0, n_jobs=20)
        rf.fit(X,y)
        pdf['pred_'+Xset+'_'+ycol] = pd.Series(rf.oob_prediction_, index=y.index)

        # Get feature importances of every tree
        imps = [tree.feature_importances_ for tree in rf.estimators_]
        imps = pd.DataFrame(imps, columns=X.columns)
        imp_dict[Xset+'_'+ycol] = imps

        # keep the model and training data for the model
        model_dict[Xset+'_'+ycol] = rf 

# merge feature importances
imp_merged = pd.concat(imp_dict, axis=1)

In [ ]:
# # Test use of Nearest Neighbors instead of RF for v07
# from sklearn import neighbors
# from sklearn.model_selection import cross_val_predict #train_test_split

# # Use top 10 RF model vars for the NN model
# imp_path = r"J:\projects\ECOFOR\gedi\models\v04\GEDI_2AB_2019to2023_leafon_sampy500m_all_imps_v04.csv"
# imps = pd.read_csv(imp_path, header=[0,1])
# imps = imps.mean(axis=0)

# # Structures to hold predictions, feature importances, and model objects
# pdf = mdf[meta_cols+Ycols].copy()
# imp_dict = {}
# model_dict = {}

# for Xset, Xcols in Xcols_dict.items():    
#     for ycol in Ycols:
#         # Get subset of variables to use
#         Xselected = imps[(Xset+"_"+ycol, slice(None))]
#         Xselected = Xselected.sort_values(ascending=False)[:10].index
        
#         X = mdf[Xselected]
#         y = mdf[ycol]
#         nn = neighbors.KNeighborsRegressor(n_neighbors=2, algorithm="auto", leaf_size=30, p=2, metric='minkowski', n_jobs=1)
        
#         # Get CV predictions
#         cv_preds = cross_val_predict(nn, X, y, cv=10, n_jobs=-1)
#         pdf['pred_'+Xset+'_'+ycol] = pd.Series(cv_preds, index=y.index)

#         # redo model with all data
#         nn.fit(X, y)
#         model_dict[Xset+'_'+ycol] = nn 

In [ ]:
# # Plot rh98 and cover for v04_ltpa for jody
# fig, axes = plt.subplots(1, 2, figsize=(5.7, 2))

# Ycols_sub = {'rh98':'RH98', 'cover':'Cover'}
# for j, (ycol, ax) in enumerate(zip(Ycols_sub.keys(), axes)):
#     x, y = pdf['pred_'+Xset+'_'+ycol], pdf[ycol]
#     hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
#     ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

#     r2 = r2_score(y, x)
#     bias = (x-y).mean()
#     rmse = mean_squared_error(y, x)**0.5

#     # add text
#     ax.text(0.99, 0.22, "R$^2$= " + str(r2.round(2)), transform=ax.transAxes, ha='right')
#     ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
#     ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')

#     ax.set(title=Ycols_sub[ycol])
#     if j==0:
#         ax.set(ylabel='Observed')
#     fig.supxlabel('Predicted', x=0.47, y=-0.1, ha='center', fontsize=10)

# fig.subplots_adjust(wspace=0.3, hspace=0.5)
# cb = fig.colorbar(hb, ax=axes, shrink=True, aspect=30, pad=0.02) #cax=cax, aspect=)#
# cb.set_label('Count')

In [ ]:
# Plot the model results
nrows = len(Xcols_dict.keys())
ncols = len(Ycols)
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, 2.4*nrows))

if nrows==1:
    axes = np.expand_dims(axes, axis=0)

for i, (Xset, axrow) in enumerate(zip(Xsets, axes)):
    for j, (ycol, ax) in enumerate(zip(Ycols, axrow)):
        x, y = pdf['pred_'+Xset+'_'+ycol], pdf[ycol]
        hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
        ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

        r2 = r2_score(y, x)
        bias = (x-y).mean()
        rmse = mean_squared_error(y, x)**0.5

        # add text
        ax.text(0.99, 0.22, "R$^2$= " + str(np.round(r2, 2)), transform=ax.transAxes, ha='right')
        ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
        ax.text(0.99, 0.02,  "RMSE= " + str(np.round(rmse, 2)), transform=ax.transAxes, ha='right')

        ax.set(title=Xset+'_'+ycol)
        if j==0:
            ax.set(ylabel='Observed')
        fig.supxlabel('Predicted', x=0.47, y=0, ha='center', fontsize=10)

fig.subplots_adjust(wspace=0.3, hspace=0.5)
cb = fig.colorbar(hb, ax=axes, shrink=True, aspect=30, pad=0.02) #cax=cax, aspect=)#
cb.set_label('Count')

In [ ]:
# Compare distributions of the predictions and observations
from scipy.stats import ks_2samp, t

nrows = len(Xcols_dict.keys())
fig, axes = plt.subplots(nrows, 4, figsize=(10, 2.4*nrows))

if nrows==1:
    axes = np.expand_dims(axes, axis=0)

for i, (Xset, axrow) in enumerate(zip(Xsets, axes)):
    for j, (ycol, ax) in enumerate(zip(Ycols, axrow)):
        xcol = 'pred_'+Xset+'_'+ycol
        xydf = pd.melt(pdf[[xcol, ycol]])
        sns.ecdfplot(xydf, x='value', hue='variable', ax=ax, legend=False)

        # ks_2samp silently gives wrong values if nan's included, so make sure they're removed
        mask = pdf[[xcol, ycol]].notna().all(axis=1)
        ks, pval = ks_2samp(pdf.loc[mask, xcol], pdf.loc[mask, ycol])
        ax.text(0.95, 0.1, f"KS = {np.round(ks,2)}", ha='right', transform=ax.transAxes)

        ax.set(ylabel=None, title=Xset+'_'+ycol)
        
orange_line = mpl.lines.Line2D([0], [0], color='orange', lw=2)
blue_line = mpl.lines.Line2D([0], [0], color='blue', lw=2)
fig.legend([orange_line, blue_line], ['obs', 'pred'], loc='upper center', ncol=2)

fig.tight_layout()

In [ ]:
# Combine and save predictions, importances, and model objects

# TODO: Set these data types on load next time
pdf['delta_time'] = pdf['delta_time'].astype(str)

# oob_path = os.path.join(outdir, outbasename+"_oob_"+ver+".gpkg")
# pdf.to_file(oob_path, driver='GPKG', index=True)

oob_path = os.path.join(outdir, outbasename+"_oob_"+ver+".parquet")
pdf.to_parquet(oob_path, index=True)

imp_path = os.path.join(outdir, outbasename+"_imps_"+ver+".csv")
imp_merged.to_csv(imp_path, index=False)

for dset, model in model_dict.items():
    model_path = os.path.join(outdir, outbasename + "_" + dset + "_" + ver + ".joblib")
    joblib.dump(model, model_path)

### Temporal cross-validation

In [ ]:
# Setup for TCV
tcv_path = os.path.join(outdir, outbasename+"_tcv_"+ver+".parquet")

years = list(mdf['rain_year'].unique())
years.sort()
meta_cols = ['delta_time', 'year', 'rain_year']
pdf = mdf[meta_cols+Ycols].copy()
stats = pd.DataFrame(columns=['Xset', 'metric', 'year', 'n', 'r2', 'rmse', 'bias'])

In [ ]:
# Perform temporal cross-validation on all Xsets
for Xset, Xcols in Xcols_dict.items():
    for ycol in Ycols:
        print(Xset, ycol)
        for year in years:
            ddf = mdf[Xcols+[ycol, 'rain_year']].dropna()
            train, test = ddf[ddf['rain_year']!=year], mdf[mdf['rain_year']==year]
            Xtrain, ytrain = train[Xcols], train[ycol]
            Xtest, ytest = test[Xcols], test[ycol]
            rf = RandomForestRegressor(n_estimators=100, max_features='sqrt', oob_score=False, random_state=0, n_jobs=20)
            rf = rf.fit(Xtrain, ytrain)
            pred = rf.predict(Xtest)
            pdf.loc[ytest.index, Xset+'_'+ycol] = pd.Series(pred, index=ytest.index)

            # accuracy stats
            # TODO: consider assigning non-forest as 0, like use of irr_area in irrigation_ks.ipynb
            obs = ytest.copy()

            ix = len(stats)
            stats.loc[ix, 'metric'] = ycol 
            stats.loc[ix, 'Xset'] = Xset
            stats.loc[ix, 'year'] = year
            stats.loc[ix, 'n'] = obs.size
            stats.loc[ix, 'r2'] = r2_score(obs, pred)
            stats.loc[ix, 'rmse'] = mean_squared_error(obs, pred)**0.5
            stats.loc[ix, 'bias'] = bias = (pred-obs).mean()

            # model_dict[dsetyr+'tcv'] = rf
stats

In [ ]:
# Save
pdf.to_parquet(tcv_path, index=True)
stats.to_csv(os.path.splitext(tcv_path)[0]+"_stats.csv", index=False)

### Bias correction of selected model

#### BC1
Use BC1 in Zhang and Lu 2012 to reduce compression to the mean in the selected model.

In [ ]:
# Setup 
Xset = 'lt-p-s-t' # selected model predictor set
Xcols = Xcols_dict[Xset]

train, test = train_test_split(mdf, test_size=0.3, random_state=42)
pdf = test[meta_cols+Ycols].copy()
model_dict = {}

In [ ]:
# Run bias corrected random forest on each GEDI metric
for ycol in Ycols:
    Xtrain, ytrain = train[Xcols], train[ycol]
    Xtest, ytest = test[Xcols], test[ycol]
    
    # Random forest 1 with OOB predictions
    rf = RandomForestRegressor(n_estimators=100, max_features='sqrt', oob_score=True, random_state=0, n_jobs=20)
    rf.fit(Xtrain,ytrain)
    ypred = pd.Series(rf.oob_prediction_, index=ytrain.index, name='ypred')

    # Random forest 2 of residuals
    resids = ytrain - ypred
    rtrain = pd.concat([Xtrain, ypred], axis=1)
    rf_resid = RandomForestRegressor(n_estimators=100, max_features='sqrt', oob_score=True, random_state=0, n_jobs=20)
    rf_resid.fit(rtrain, resids)

    # Apply both RFs to test data and sum pred + resid
    ypred_test = pd.Series(rf.predict(Xtest), index=ytest.index, name='ypred')
    rtest = pd.concat([Xtest, ypred_test], axis=1)
    resids_test = pd.Series(rf_resid.predict(rtest), index=rtest.index, name='resid')
    bc_test = ypred_test + resids_test

    # Save test results
    pdf["pred_"+ycol] = ypred_test
    pdf["pred_resid_"+ycol] = resids_test
    pdf["pred_bc_"+ycol] = bc_test
    model_dict[ycol] = rf
    model_dict[ycol+"_resid"] = rf_resid

In [ ]:
# Combine and save predictions, importances, and model objects

# TODO: Set these data types on load next time
pdf['delta_time'] = pdf['delta_time'].astype(str)

# oob_path = os.path.join(outdir, outbasename+"_oob_"+ver+".gpkg")
# pdf.to_file(oob_path, driver='GPKG', index=True)

bc_path = os.path.join(outdir, outbasename+"_bc_"+ver+".parquet")
pdf.to_parquet(bc_path, index=True)

for mname, model in model_dict.items():
    model_path = os.path.join(outdir, outbasename + "_" + mname + "_bc_" + ver + ".joblib")
    joblib.dump(model, model_path)

#### EDM
Use empirical distribution modeling from ___.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# --- Core EDM Implementation ---
def empirical_distribution_matching(X_train, Y_train, X_test, random_state=42):
    """
    Implements the Empirical Distribution Matching (EDM) method for post-processing
    Random Forest predictions based on OOB scores.

    The method finds a mapping function f such that f(Y_OOB) matches the empirical
    distribution of the true observations (Y_train). This function is then applied
    to the raw test set predictions (Y_test_pred_raw).

    f(Y_test_pred_raw) = F_Y_train^-1( F_Y_OOB(Y_test_pred_raw) )

    Args:
        X_train (np.ndarray): Training features.
        Y_train (np.ndarray): True training observations.
        X_test (np.ndarray): Test features.
        random_state (int): Random seed for the Random Forest model.

    Returns:
        tuple: (Y_test_pred_corrected, Y_test_pred_raw, rf_model)
    """

    # 1. Train Random Forest and obtain OOB predictions
    # oob_score=True is essential for the EDM method
    rf = RandomForestRegressor(
        n_estimators=100,
        oob_score=True,
        random_state=random_state,
        max_features='sqrt',
        n_jobs=-1
    )
    rf.fit(X_train, Y_train)

    # Check for OOB predictions availability
    if not hasattr(rf, 'oob_prediction_') or rf.oob_prediction_ is None:
        raise ValueError(
            "RandomForestRegressor failed to produce OOB predictions. "
            "Ensure 'oob_score=True' is set and n_estimators is large enough."
        )

    Y_OOB = rf.oob_prediction_
    Y_obs = Y_train  # True observations (Y)

    # Filter out NaNs if any (can happen with small datasets/estimators, though unlikely here)
    valid_indices = ~np.isnan(Y_OOB)
    Y_OOB_valid = Y_OOB[valid_indices]
    Y_obs_valid = Y_obs[valid_indices]

    # 2. Compute the ECDF of OOB predictions (F_Y_OOB)
    # We sort the OOB predictions and define their corresponding cumulative probabilities.
    Y_OOB_sorted = np.sort(Y_OOB_valid)
    n_oob = len(Y_OOB_sorted)
    # ECDF probabilities are defined at [1/N, 2/N, ..., 1]
    p_oob = np.linspace(1 / n_oob, 1.0, n_oob)

    # 3. Compute the Inverse ECDF (Quantile Function) of Observations (F_Y_obs^-1)
    # We sort the observations and define their corresponding cumulative probabilities.
    Y_obs_sorted = np.sort(Y_obs_valid)
    n_obs = len(Y_obs_sorted)
    # Probabilities for interpolation (P = F_Y_obs(Y_obs_sorted))
    p_obs = np.linspace(1 / n_obs, 1.0, n_obs)

    # 4. Apply the EDM transformation
    Y_test_pred_raw = rf.predict(X_test)

    # Step A: Find the ECDF value (probability 'p') for each raw test prediction
    # This is p = F_Y_OOB(Y_test_pred_raw).
    # We interpolate from (Y_OOB_sorted, p_oob) to Y_test_pred_raw.
    prob_values = np.interp(
        Y_test_pred_raw,  # Values to evaluate (raw test predictions)
        Y_OOB_sorted,     # Known x-coordinates (sorted OOB predictions)
        p_oob,            # Known y-coordinates (ECDF probabilities)
        left=0.0,         # Clip probability to 0.0 below min
        right=1.0         # Clip probability to 1.0 above max
    )

    # Step B: Apply the Inverse ECDF of observations (Quantile function)
    # This is Y_test_pred_corrected = F_Y_obs^-1(prob_values).
    # We interpolate from (p_obs, Y_obs_sorted) to the probability values 'prob_values'.
    Y_test_pred_corrected = np.interp(
        prob_values,      # Values to evaluate (the probability 'p' from Step A)
        p_obs,            # Known x-coordinates (ECDF probabilities of observations)
        Y_obs_sorted,     # Known y-coordinates (sorted observations/quantiles)
        left=Y_obs_sorted[0],
        right=Y_obs_sorted[-1]
    )

    return Y_test_pred_corrected, Y_test_pred_raw, rf

In [ ]:
# Setup 
Xset = 'lt-p-s-t' # selected model predictor set
Xcols = Xcols_dict[Xset]

train, test = train_test_split(mdf, test_size=0.3, random_state=42)
pdf = test[meta_cols+Ycols].copy()
model_dict = {}

In [ ]:
for ycol in Ycols:
    Xtrain, ytrain = train[Xcols], train[ycol]
    Xtest, ytest = test[Xcols], test[ycol]

    # Execute the EDM method
    bc_test, ypred_test, rf = empirical_distribution_matching(
        Xtrain, ytrain, Xtest
    )
    
    # Save test results
    pdf["pred_"+ycol] = ypred_test
    pdf["pred_bc_"+ycol] = bc_test
    model_dict[ycol] = rf

    # 2. Evaluate Performance

    # Evaluate the raw (uncalibrated) predictions
    mse_raw = mean_squared_error(ytest, ypred_test)

    # Evaluate the corrected (EDM-calibrated) predictions
    mse_corrected = mean_squared_error(ytest, bc_test)
    
    print("-"*20)
    print(f"Metric: {ycol}\n")
    print(f"Model OOB Score (R^2): {rf.oob_score_:.4f}")
    print("\nPrediction Statistics:")
    print(f"  True Test Mean: {np.mean(ytest):.4f}")
    print(f"  Raw Pred Mean:  {np.mean(ypred_test):.4f} (Note the difference/bias)")
    print(f"  EDM Pred Mean:  {np.mean(bc_test):.4f} (Should be closer to True Mean)")

    print("\nError Metrics (Mean Squared Error):")
    print(f"  1. Raw RF Prediction MSE:    {mse_raw:.4f}")
    print(f"  2. EDM Corrected Prediction MSE: {mse_corrected:.4f}")

    improvement = ((mse_raw - mse_corrected) / mse_raw) * 100
    print(f"\nResult: EDM improved the MSE by {improvement:.2f}%")
    print("-" * 50)

    # Quick check on the corrected distribution matching
    print("Quantile Check (Should be similar for ytest and bc_test):")
    quantiles = [0.05, 0.5, 0.95]
    print(f"  {quantiles[0]*100}% Quantile:")
    print(f"    True Observations: {np.quantile(ytest, quantiles[0]):.4f}")
    print(f"    Raw Predictions:   {np.quantile(ypred_test, quantiles[0]):.4f}")
    print(f"    EDM Corrected:     {np.quantile(bc_test, quantiles[0]):.4f}")
    print(f"  {quantiles[1]*100}% Quantile (Median):")
    print(f"    True Observations: {np.quantile(ytest, quantiles[1]):.4f}")
    print(f"    Raw Predictions:   {np.quantile(ypred_test, quantiles[1]):.4f}")
    print(f"    EDM Corrected:     {np.quantile(bc_test, quantiles[1]):.4f}")
    print(f"  {quantiles[2]*100}% Quantile:")
    print(f"    True Observations: {np.quantile(ytest, quantiles[2]):.4f}")
    print(f"    Raw Predictions:   {np.quantile(ypred_test, quantiles[2]):.4f}")
    print(f"    EDM Corrected:     {np.quantile(bc_test, quantiles[2]):.4f}")

In [ ]:
outdir = 'J:\\projects\\ECOFOR\\gedi\\models\\v08EDM'

In [ ]:
ver = "v08EDM"

In [ ]:
# Combine and save predictions, importances, and model objects

# TODO: Set these data types on load next time
pdf['delta_time'] = pdf['delta_time'].astype(str)

# oob_path = os.path.join(outdir, outbasename+"_oob_"+ver+".gpkg")
# pdf.to_file(oob_path, driver='GPKG', index=True)

bc_path = os.path.join(outdir, outbasename+"_bc_"+ver+".parquet")
pdf.to_parquet(bc_path, index=True)

for mname, model in model_dict.items():
    model_path = os.path.join(outdir, outbasename + "_" + mname + "_bc_" + ver + ".joblib")
    joblib.dump(model, model_path)

# Maps

## Pyramids

In [ ]:
# Calculating pyramids and stats for all maps

# gdaladdo -ro --config COMPRESS_OVERVIEW ZSTD --config ZSTD_LEVEL_OVERVIEW 1 --config PREDICTOR_OVERVIEW 2 --config INTERLEAVE_OVERVIEW BAND --config GDAL_NUM_THREADS 16 --config GDAL_CACHEMAX 4096 path

# helper functions
def pyr_stats(path, run=True):
    """Set nodata (str of number or 'nan'). Calculate stats and pyramids for image at path (str)."""
    cmds = {'stats':[], 'pyr':[]}
    
    stats_cmd = 'gdalinfo -approx_stats ' + path
    if run:
        result = subprocess.check_output(stats_cmd)
    
    if not os.path.exists(path+".ovr"):
        pyr_cmd = 'gdaladdo -ro --config COMPRESS_OVERVIEW ZSTD --config ZSTD_LEVEL 1 --config PREDICTOR 2 --config INTERLEAVE_OVERVIEW BAND --config GDAL_NUM_THREADS 6 --config GDAL_CACHEMAX 4096 ' + path
        if run:
            result = subprocess.check_output(pyr_cmd)
    else:
        pyr_cmd = "ECHO " + path + " completed"
    
    return stats_cmd, pyr_cmd

stat_cmds, pyr_cmds = [], []
paths = glob(r"D:\ECOFOR\gedi\maps\v04_ltpa2\*\*.tif")
for path in paths:
    stat_cmd, pyr_cmd = pyr_stats(path, run=False)
    stat_cmds.append(stat_cmd)
    pyr_cmds.append(pyr_cmd)

cmd_concurrent(stat_cmds, threads=16)
cmd_concurrent(pyr_cmds, threads=8)

## Change

In [ ]:
# Calculate difference between two years for each metric
basedir = r"J:\projects\ECOFOR\gedi\maps\v08\lt-p-s-t"
y1 = "2017"
y2 = "2021"
pct_chg = False #True
mask_y1_lt = 5 # mask out areas where y1 is less than x

outdir = os.path.join(basedir, 'change')
metrics = ["cover_gt5m"] #"rh98", "cover"] #, "pai", "fhd",, 

os.makedirs(outdir, exist_ok=True)

for metric in metrics:
    path1 = os.path.join(basedir, metric, metric+"_"+y1+".tif")
    path2 = os.path.join(basedir, metric, metric+"_"+y2+".tif")
    pct_str = "_pct" if pct_chg else ""
    mask_str = "_y1gte"+str(mask_y1_lt) if mask_y1_lt else ""
    outpath = os.path.join(outdir, metric+"_"+y2+"minus"+y1+pct_str+mask_str+".tif")
    
    with rasterio.open(path1) as src:
        arr1 = src.read(1)
        profile = src.profile
    
    with rasterio.open(path2) as src:
        arr2 = src.read(1)
        
    if mask_y1_lt:
        arr1[arr1<mask_y1_lt] = np.nan

    chg = np.subtract(arr2, arr1, dtype=np.float32)
    chg[np.isnan(arr1) | np.isnan(arr2)] = np.nan
    
    
    if pct_chg:
        arr1[arr1==0] = 0.01
        chg = chg / arr1 * 100
    
    # change if using integer layers
    # chg = np.subtract(arr2, arr1, dtype=np.int16)
    # chg[(arr1==profile['nodata']) | (arr2==profile['nodata'])] = -32768
    # profile['nodata'] = -32768
    # profile['dtype'] = np.int16

    with rasterio.open(outpath, 'w', **profile) as dst:
        dst.write(chg, 1)

In [ ]:
stat_cmds, pyr_cmds = [], []
paths = glob(r"J:\projects\ECOFOR\gedi\maps\v08\lt-p-s-t\change\*.tif")
for path in paths:
    stat_cmd, pyr_cmd = pyr_stats(path, run=False)
    stat_cmds.append(stat_cmd)
    pyr_cmds.append(pyr_cmd)

In [ ]:
cmd_concurrent(stat_cmds, threads=10)
cmd_concurrent(pyr_cmds, threads=3)

## Upload to GEE  
Rescale each of the datasets to an 8-bit integer, and save the scale and offset. Then upload to GEE.  


In [ ]:
basedir = r"D:\ECOFOR\gedi\maps\v08\lt-p-s-t"
metrics = next(os.walk(basedir))[1]

for metric in metrics:
    paths = glob(os.path.join(basedir, metric, "*.tif"))
    outdir = os.path.join(basedir, metric, 'as_uint8')
    os.makedirs(outdir, exist_ok=True)
    
    # load each array and get the min & max for a universal scaling
    # TODO: could load all at once and then process to avoid reading twice
    mins, maxs = [], []
    for path in paths:
        with rasterio.open(path) as src:
            arr = src.read(1, masked=True)
            mins.append(arr.min())
            maxs.append(arr.max())
    low = np.array(mins).min()
    high = np.array(maxs).max()
    
    scale = 254.0 / (high - low)  # setting 254 as max and 255 as nodata
    offset = -low * scale
    
    for path in paths:
        with rasterio.open(path) as src:
            arr = src.read(1, masked=True)
            prof = src.profile
        
        arr = (arr * scale + offset).round().astype(np.uint8)
        arr = arr.filled(255)
        
        outpath = os.path.join(outdir, os.path.basename(path))
        prof.update({'dtype':np.uint8, 'nodata':255})
        with rasterio.open(outpath, 'w', **prof) as dst:
            dst.write(arr, 1)
            
    text = ("scale = " + str(scale) + "\n" + 
            "offset = " + str(offset) + "\n" + 
            "orig = (value - offset) / scale")
            
    with open(os.path.join(outdir, "readme.txt"), 'w') as f:
        f.writelines(text)

In [ ]:
# Create image collections in legacy storage for each metric
for metric in metrics:
    indir = os.path.join(r"D:\ECOFOR\gedi\maps\v08\lt-p-s-t", metric, "as_uint8")
    asset_id = "projects/earthengine-legacy/assets/users/stevenf/ecofor/" + metric + "_v8_uint8"
    cmd = "earthengine create collection " + asset_id
    stdout = subprocess.check_output(cmd)
    
    # Get scale and offset to write in the image and collection properties
    # TODO: next time process metric groups so reading the readme doesn't need to be repeated
    readme = os.path.join(indir, "readme.txt")
    with open(readme) as f:
        lines = f.readlines()

    for l in lines:
        if l.startswith('scale'):
            scale = float(l.split('= ')[1])
        if l.startswith('offset'):
            offset = float(l.split('= ')[1])
        if l.startswith('orig'):
            rescaling_str = l
    
    # set properties on the collection
    cmd = ('earthengine asset set'+
       ' -p scale='+str(scale) +
       ' -p offset='+str(offset) +
       ' -p description="' + rescaling_str + '"' +
       ' ' + asset_id)
    print(cmd)
    stdout = subprocess.check_output(cmd)

In [ ]:
# Upload to GEE
# Note: Must manually upload all images together to the gedi_v8 cloud storage folder first. No subfolders.


# Get files in GCS bucket
gsutil = r"C:\Users\stevenf\AppData\Local\Google\Cloud SDK\google-cloud-sdk\bin\gsutil.cmd"
stdout = subprocess.check_output(gsutil + " ls gs://earthengine-228722/gedi_v8")
uris = stdout.decode("utf-8").split("\n")[1:-1]

import json

# Upload with manifest
for uri in uris:
    name = os.path.basename(uri)[:-4]
    metric = name.split('_')[0]
    indir = os.path.join(basedir, metric, "as_uint8")
    
    # Get scale and offset to write in the image and collection properties
    readme = os.path.join(indir, "readme.txt")
    with open(readme) as f:
        lines = f.readlines()

    for l in lines:
        if l.startswith('scale'):
            scale = float(l.split('= ')[1])
        if l.startswith('offset'):
            offset = float(l.split('= ')[1])
        if l.startswith('orig'):
            rescaling_str = l

    # manifest
    manifest_path = os.path.join(indir, "manifest.json")
    manifest = {
      "name": "projects/earthengine-legacy/assets/users/stevenf/ecofor/" + metric + "_v8_uint8/"+name,
      "tilesets": [
        {
          "sources": [
            {
              "uris": [
                uri
              ]
            }
          ]
        }
      ],
      "missingData": {
         "values": [255]
      },
      "startTime": name.split("_")[1]+"-01-01T00:00:00Z",
      "endTime": name.split("_")[1]+"-12-31T00:00:00Z",
      "properties":{
          "scale":scale,
          "offset":offset
      }
    }

    # write manifest to JSON
    with open(manifest_path, 'w') as f:
        json.dump(manifest, f)
    
    # upload to earth engine
    cmd = "earthengine upload image --manifest " + manifest_path
    stdout = subprocess.check_output(cmd)

In [ ]:
# # Old
# # Cover as integer for manual upload to GEE
# paths = glob(r"C:\scratch\ECOFOR\gedi\maps\v03\cover\*.tif")
# # paths = paths[:1]

# for path in paths:
#     outdir, outname = os.path.split(path)
#     outpath = os.path.join(outdir, "as_int", outname)
#     with rasterio.open(path) as src:
#         arr = src.read(1)
#         profile = src.profile
    
#     nodata = np.isnan(arr)
#     arr = np.uint8(np.round(arr*100))
#     arr[nodata] = 255
    
#     profile['dtype'] = np.uint8
#     profile['nodata'] = 255
    
#     with rasterio.open(outpath, 'w', **profile) as dst:
#         dst.write(arr, 1)

## Reproject for DAAC
The ORNL DAAC wants the data in UTM 36 S intead of N.

In [ ]:
paths = glob(r"J:\projects\ECOFOR\deliverables\ORNL_DAAC_202505\v08\lt-p-s-t\*\*.tif")
outdir = r"J:\projects\ECOFOR\deliverables\ORNL_DAAC_202506" 

for path in paths:
    outpath = os.path.join(outdir, os.path.basename(path))

    cmd = "gdalwarp -t_srs EPSG:32736 -co COMPRESS=LZW " + path + " " + outpath
    result = subprocess.check_output(cmd)

## Prep SAE tables  
Prepare data necessary for doing small area estimation in R

In [ ]:
# Get population of pixels for all years
aois_path = r"J:\projects\ECOFOR\gedi\sae\sae_aois.gpkg"
basedir = r"D:\ECOFOR\gedi\maps\v08\lt-p-s-t"
outpath = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_pop.parquet"

rast_paths = [p for p in glob(os.path.join(basedir, "*/*.tif")) if "change" not  in p]

def get_fdf(fdict):
    fprop = fdict['properties']
    arr = fprop['mini_raster_array'].ravel()
    arr = arr[~arr.mask].data
    aoi = fprop['name']
    return pd.DataFrame({'val':arr, 'aoi':aoi})

def get_aoi_vals(aois_path, rast_path):
    zstats = zonal_stats(aois_path, rast_path, stats="count", raster_out=True, geojson_out=True)
    fname = os.path.basename(rast_path)[:-4]
    metric, year = fname.split('_')
    fdfs = [get_fdf(f) for f in zstats]
    df = pd.concat(fdfs, axis=0)
    df['metric'] = metric
    df['year'] = int(year)
    return df

rast_dfs = Parallel(n_jobs=10)(delayed(get_aoi_vals)(aois_path, rast_path) for rast_path in rast_paths)

df = pd.concat(rast_dfs, axis=0)

# pivoting this way takes 3ish minutes and lots of ram
df['ix'] = df.index
dfw = df.pivot(index=['aoi', 'year', 'ix'], columns='metric', values='val').reset_index()
dfw['domain'] = dfw['aoi']+'_'+dfw['year'].astype(str)

dfw.to_parquet(outpath)
del dfw

In [ ]:
# Get GEDI pred/obs samples in domains
oob_path = r"J:\projects\ECOFOR\gedi\models\v08\GEDI_2AB_2019to2023_leafon_sampy500m_all_v08.parquet"
aois_path = r"J:\projects\ECOFOR\gedi\sae\sae_aois.gpkg"
outpath = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_samp_v08EDM.gpkg"

df = gpd.read_parquet(oob_path)
aois = gpd.read_file(aois_path)

sdf = df.sjoin(aois[['name', 'geometry']], how='inner')
sdf = sdf.drop(columns=['index_right'])
sdf = sdf.rename(columns={'name':'aoi'})
sdf['domain'] = sdf['aoi']+'_'+sdf['rain_year'].astype(str)

sdf.to_file(outpath, driver="GPKG")

In [ ]:
# Get all GEDI footprints in domains
path = r"J:\projects\ECOFOR\gedi\gedi_data\04_gedi_filtered_data_shp\GEDI_2AB_2019to2023.parquet"
aois_path = r"J:\projects\ECOFOR\gedi\sae\sae_aois.gpkg"
outpath = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_all.gpkg"

df = gpd.read_parquet(path)
aois = gpd.read_file(aois_path)

# Filter to points that will be used (leaf-on only)
df['delta_time'] = pd.to_datetime(df['delta_time'])
df = df[df['rh98']<45] # Remove unreasonable points
df = df[(df['delta_time'].dt.day_of_year < 121) | (df['delta_time'].dt.day_of_year > 305)] # keep only leaf-on (Nov - Apr) as defined in Li 2023

# Rain year is defined as the year beginning with the start of the dry season (121-273) and the following wet season (274-120)
# e.g., rain year 2018 is May 1, 2018 - April 30 2019
df['year'] = df['delta_time'].dt.year
df['rain_year'] = df['year'].copy()
df.loc[df['delta_time'].dt.day_of_year < 121, 'rain_year'] += -1 

df = df.to_crs(aois.crs)

sdf = df.sjoin(aois[['name', 'geometry']], how='inner')
sdf = sdf.drop(columns=['index_right'])
sdf = sdf.rename(columns={'name':'aoi'})

sdf['domain'] = sdf['aoi']+'_'+sdf['rain_year'].astype(str)

sdf.to_file(outpath, driver="GPKG")

## Compare SAE to zonal stats
Run standard zonal stats as a comparison to the SAE results obtained from R.

In [ ]:
path = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_pop.parquet"
df = pd.read_parquet(path)
means = df.groupby('domain').agg({'aoi':'first', 'year':'first', 'cover':'mean', 'fhd':'mean', 'pai':'mean', 'rh98':'mean'})

path = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20250312.csv"
mb = pd.read_csv(path)
mb = mb.pivot(index='domain', columns='metric', values='mean')

path = r"J:\projects\ECOFOR\gedi\sae\direct_gedi_estimates_20250415.csv"
db = pd.read_csv(path)
db = db[db['samptype']=='srs']
db = db.pivot(index='domain', columns='metric', values='mean')

mdf = pd.concat([means, mb, db], keys=['zonal', 'mb','db'], axis=1)

In [ ]:
mdf[mdf[('zonal','aoi')]=='plantation_a']

## Climate sensitivity  
Evaluate the sensitivity of the predicted metrics to climate, and in particular the drought in 2015/2016. This drought was during the rainy season of 2015/16 so October 2015 to April 2016. There was a recovery in the wet season of 2016. https://doi.org/10.2989/10220119.2020.1718755

Compare 2015 (2015 dry + 2015/16) to 2016 for areas with no apparent vegetation change.

In [ ]:
# Filter VCA sites for intersection with fires between 2009 and 2017
fire_paths = [
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2009fires.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2010fires.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2011fires.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2012fires.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2013fires.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2014fires.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2014firesUTM.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2015firesUTM.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2016firesUTM.shp',
    'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2017firesUTM.shp'
]

fire_df = pd.concat([gpd.read_file(p).to_crs(epsg=32736) for p in fire_paths])

burned = fire_df.unary_union

In [ ]:
# Prep VCA points and convert to boxes for photo-interp
path = r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_sites.gpkg"
df = gpd.read_file(path)
df = df.to_crs(epsg=fire_df.crs.to_epsg())

# Drop points intersecting burns, or with a big difference in coordiantes or their is no geometry
df = df[~df.intersects(burned)]
df = df[~(df["coords_dif"]>10) & (~df["geometry"].isnull())]

# Shuffle to get better spatial distribution when examining first X in list
df = df.sample(frac=1, random_state=42) 

# Snap to nearest pixel
rast_path = r"J:\projects\ECOFOR\gedi\maps\v08\lt-p-s-t\cover\cover_2015.tif"

with rasterio.open(rast_path) as src:
    src_crs = src.crs
df = df.to_crs(epsg=src_crs.to_epsg())

def snap_points(gdf_points, raster_filepath):
    with rasterio.open(raster_filepath) as src:
        transform = src.transform
        res = src.res[0]   
    coords = np.array([(p.x, p.y) for p in gdf_points.geometry])
    rows, cols = rasterio.transform.rowcol(transform, coords[:, 0], coords[:, 1])
    x_snapped, y_snapped = rasterio.transform.xy(transform, rows, cols, offset='center')
    snapped_geometries = [shapely.geometry.Point(x, y) for x, y in zip(x_snapped, y_snapped)]
    return snapped_geometries

df["geometry"] = snap_points(df, rast_path)

# df.to_file(r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped.gpkg", driver="GPKG")
# df.to_file(r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped.shp")

# # Create boxes for visualization in Google earth
# df["geometry"] = df.buffer(15, cap_style="square")
# df.to_file(r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_box.shp")

In [ ]:
# Convert the version with PI completed to a shapefile for upload to GEE
path = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi.gpkg"
df = gpd.read_file(path)
df.to_file(os.path.splitext(path)[0]+".shp")

In [ ]:
# Extract cover and LandTrendr for points
path = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi.gpkg"
outpath = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi_extract2.gpkg"

df = gpd.read_file(path)

indirs = {
    "ltwet":r"J:\projects\ECOFOR\lt\wet",
    "ltdry":r"J:\projects\ECOFOR\lt\dry",
    "cover":r"J:\projects\ECOFOR\gedi\maps\v08\lt-p-s-t\cover"
}

layers = {}
for key, indir in indirs.items():
    ext = "vrt" if key.startswith("lt") else "tif"
    paths = glob(os.path.join(indir, "*."+ext))
    for path in paths:
        year = path[-8:-4]
#         if int(year) < 2007:
#             continue
        layers[key+"_"+year] = path
        
# Only points with a classification for extraction
df = df[~df["change"].isnull()]

# Extract layers for each point
for name, path in layers.items():
    with rasterio.open(path) as src:
        bands = src.descriptions
        
    if len(bands)<2:
        bands = [name.split("_")[0]]
    
    def extract_vals(band, band_name):
        vals = list(gen_point_query(df['geometry'], path, band=band+1, interpolate='nearest'))
        return pd.Series(vals, index=df.index, name=name+'_'+band_name)

    val_series = Parallel(n_jobs=8)(delayed(extract_vals)(band, band_name) for band, band_name in enumerate(bands))
    valdf = pd.concat(val_series, axis=1)
    df = pd.merge(df, valdf, 'left', left_index=True, right_index=True)

df.to_file(outpath, driver="GPKG")

In [ ]:
# load earth engine functions
sys.path.append(r'J:\users\stevenf\code\utils\pee')
import ee
import landsat as lxtools
import time_series

ee.Initialize()

# Extract original landsat wet season NDVI composites for points (before LandTrendr)
fc = ee.FeatureCollection("projects/earthengine-legacy/assets/users/stevenf/ecofor/vca_unburned09to17_snapped_pi")
fc = fc.filter(ee.Filter.notEquals("change", ""))
# fc = ee.FeatureCollection([fc.first()])

starty = 1984
endy = 2022
startdoy, enddoy = 274, 120 # Oct 1st, Apr 30 - non-leap year wet season
# startdoy, enddoy = 121, 273 # May 1st, Sept 30 - non-leap year dry season
orig_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
ixbands = ['ndvi', 'nbr', 'ndmi','tcb', 'tcg', 'tcw']
coll_kwargs = {'bands':orig_bands, 'rescale':True, 'cloud_cover':50, 'tdom':False}

comps = time_series.annual_composites(fc, starty, endy, startdoy, enddoy,
                                      lxtools.sr_collection, time_series.medoid,
                                      coll_kwargs, fill=False)

comps = comps.map(lambda i: (i.addBands(lxtools.specixs(i, ixlist=ixbands))))

DEFAULT_PROPERTIES = ee.Dictionary(
    {band: -32768 for band in orig_bands + ixbands}
)


def ensure_schema(feature):
    feature_props = feature.toDictionary()
    merged_props = feature_props.combine(DEFAULT_PROPERTIES, overwrite=False)
    return ee.Feature(feature.geometry(), merged_props)


def extract_point_data(image):
    sampled_fc = image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.first(),
        scale=30,
        tileScale=16
    )
    
    sampled_fc = sampled_fc.map(ensure_schema)
    
    return sampled_fc.map(lambda feature: feature.set('year', image.get("year")))

fc = comps.map(extract_point_data).flatten()

# fc.getInfo()

task = ee.batch.Export.table.toDrive(
    collection=fc,
    description='vca_unburned09to17_snapped_pi_landsat_wet2',
    folder='gee',
    fileFormat='CSV'
)
task.start()

# Veld Condition Assessment
Analyze VCA data for height and density of woody vegetation

## Common species
Get a list of common species and their abbreviations to help in constucting field guide.

### Woody

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\Woody VCA data - all years.xlsx"
df = pd.read_excel(path, sheet_name='2005 spp density per circle', header=2)#, index_col=[0, 1, 2])

In [ ]:
# Redo index
df['site'] = df['SITEPNTCIR'].str[:4]
df['pnt'] = df['SITEPNTCIR'].str[4:7]
df['cir'] = df['SITEPNTCIR'].str[-1]
df = df.set_index(['site', 'pnt', 'cir'])
df = df.drop(['SITEPNTCIR', 'POINT', 'CIRCLE'], axis=1)

#### C circle - trees > 3 m  
Working with C separately because erroneous values needed to be fixed

In [ ]:
# Isolate the C circle (trees > 3 m)
dfc = df.query('cir=="C"').droplevel(-1)

In [ ]:
# fix erroneously high values (would be better to recalculate density correctly later)
def fix_high_C(x):
    """Function to move the decimal place so a value is less than 1."""
    return x / (10*(10**np.floor(np.log10(np.abs(x)))))

fixedhighC = dfc[dfc>1].apply(np.vectorize(fix_high_C))
dfc[dfc>=1] = fixedhighC

In [ ]:
# convert to number of individuals per hectare
m2_per_ha = 100*100
dfc = dfc*m2_per_ha

In [ ]:
# average across all points
cdense = dfc.mean().sort_values(ascending=False)
cdense.name='density'

In [ ]:
# get acronyms
path = r"J:\projects\ECOFOR\ancillary_data\VCA\Woody VCA data - all years.xlsx"
abv = pd.read_excel(path, sheet_name='2005 original VCA data', header=2)
abv = abv[['ABBREV', 'NAME']]
abv = abv.drop_duplicates()
abv = abv.set_index('ABBREV')['NAME']
abv = abv.str.capitalize()

In [ ]:
# Show average density and name across all points
cdense = pd.merge(cdense, abv, how='left', left_index=True, right_index=True)
cdense = cdense.sort_values('density',ascending=False)

with pd.option_context("display.max_rows", 1000):
    display(cdense)

In [ ]:
# get the mean across all points on a site for each circle type
dfc_site = dfc.groupby(level=['site']).mean() 

In [ ]:
site_count = (dfc_site>0).sum(axis=0)
site_count.name = 'count'
site_count = pd.merge(site_count, abv, how='left', left_index=True, right_index=True)
site_count = site_count.sort_values('count', ascending=False)
with pd.option_context("display.max_rows", 1000):
    display(site_count)

#### B circle - trees 1 - 3 m

In [ ]:
# Isolate the B circle (trees 1 -3 m)
dfb = df.query('cir=="B"').droplevel(-1)

In [ ]:
# convert to number of individuals per hectare
m2_per_ha = 100*100
dfb = dfb*m2_per_ha

In [ ]:
# average across all points
bdense = dfb.mean().sort_values(ascending=False)
bdense.name='density'

In [ ]:
# Show average density and name across all points
bdense = pd.merge(bdense, abv, how='left', left_index=True, right_index=True)
bdense = bdense.sort_values('density',ascending=False)

with pd.option_context("display.max_rows", 1000):
    display(bdense)

In [ ]:
# get the mean across all points on a site for each circle type
dfb_site = dfb.groupby(level=['site']).mean() 

In [ ]:
site_count = (dfb_site>0).sum(axis=0)
site_count.name = 'count'
site_count = pd.merge(site_count, abv, how='left', left_index=True, right_index=True)
site_count = site_count.sort_values('count', ascending=False)
with pd.option_context("display.max_rows", 1000):
    display(site_count)

In [ ]:
bdense

## Location data  
Summarize number of sites visited by year which have GPS locations. Many sites are missing GPS points.

In [ ]:
# path = r"J:\projects\ECOFOR\ancillary_data\VCA\derived\VCA_SitesByYear_numSpecies_Draft.shp"
# vca = gpd.read_file(path)

# # # summarize species data
# # col_mask = vca.columns.str.startswith('F')
# # ((vca.loc[:,col_mask]!='ND') & (vca.loc[:,col_mask].notna())).sum()

In [ ]:
# Load from excel and recreate geometry
path = r"J:\projects\ECOFOR\ancillary_data\VCA\derived\VCA clean up.xlsx"
cdf = pd.read_excel(path, sheet_name="With Location", usecols=range(5), names = ['site', 'section', 'description', 'x', 'y'], index_col='site')
cdf = gpd.GeoDataFrame(cdf, geometry=gpd.points_from_xy(cdf['x'], cdf['y'], crs='EPSG:32736'))

In [ ]:
# Load site details for comparison
sdf = pd.read_excel(r"J:\projects\ECOFOR\ancillary_data\VCA\VCA Site details 1998-2015.xlsx",
                    usecols=[0,1,2], names = ["site", "lat", "lon"], index_col='site')

# Clean up the lat lon data
sdf = sdf.dropna()
sdf = sdf[~sdf['lon'].astype(str).str.startswith('S')]
sdf['lat_orig'], sdf['lon_orig'] = sdf['lat'].copy(), sdf['lon'].copy()

# Correct coordinates   (NOTE: ASSUMING THE FIRST SPACE IS A PERIOD LEADS TO COORDS OUTSIDE KRUGER)
for col in ['lat', 'lon']:
    # remove S and E, and remove leading and trailing whitespace
    sdf[col] = sdf[col].astype('str').str.replace('S|E|\?','', regex=True).str.strip()

    # assume first space is a period 
    sdf[col] = sdf[col].apply(lambda x: x.replace(' ', '.', 1).replace(' ', ''))

    # remove leading zeros
    sdf[col] = sdf[col].apply(lambda x: x[1:] if x.startswith('0') else x)
    
    sdf[col] = sdf[col].astype(float)
sdf['lat'] = sdf['lat'] * -1

sdf = gpd.GeoDataFrame(sdf, geometry=gpd.points_from_xy(sdf['lon'], sdf['lat'], crs='EPSG:4326'))
sdf = sdf.to_crs(cdf.crs)
sdf = sdf[~sdf.index.duplicated()]

sdf.to_file(r"G:\temp\site_details.gpkg", driver='GPKG')

In [ ]:
# merge and get distance between coordinates
gdf = pd.merge(cdf, sdf, 'outer', on='site', suffixes=('_clean', '_site'))
gdf['coords_dif'] = gdf['geometry_clean'].distance(gdf['geometry_site'])

In [ ]:
# Sites with coordinates that are separated by more than X
gdf['coords_dif_gt100'] = gdf['coords_dif'] > 100

with pd.option_context('display.max_rows', 200):
    display(gdf[gdf['coords_dif']>100].sort_values('coords_dif', ascending=False))

In [ ]:
# Clean up for export
gdf['site_lat'], gdf['site_lon'] = gdf['lat'], gdf['lon']
gdf['clean_wgs84'] = gdf['geometry_clean'].to_crs(epsg=4326)

gdf['clean_lat'], gdf['clean_lon'] = gdf['clean_wgs84'].y, gdf['clean_wgs84'].x

gdf.geometry = gdf['geometry_clean'].copy()
gdf = gdf.set_crs(gdf['geometry_clean'].crs)

gdf = gdf[['section', 'description', 'clean_lat', 'clean_lon', 'site_lat', 'site_lon', 'coords_dif', 'geometry']]

In [ ]:
# # project to 36 N to match predictor layers
# gdf = gdf.to_crs(epsg=32636)

# gdf.reset_index().to_file(r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_sites.gpkg", driver='GPKG')

### Point shapfiles
Check the point shapefiles for 2008 and 2009 sent by Chenay.

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\revcadataversions\veld_condition_assessment_sites_2008.shp"
df = gpd.read_file(path)

# Coordinates in attribute table match those in the geometry
df['X_COORD_dif'] = df['X_COORD'] - df.geometry.x
df['Y_COORD_dif'] = df['Y_COORD'] - df.geometry.y
df['POINT_X_dif'] = df['POINT_X'] - df.geometry.x
df['POINT_Y_dif'] = df['POINT_Y'] - df.geometry.y

df8 = df.copy()
df.describe().loc[['min', 'max'],] # look at min and max for the _dif columns

In [ ]:
# 2009
path = r"J:\projects\ECOFOR\ancillary_data\VCA\revcadataversions\veld_condition_assessment_sites_2009.shp"
df = gpd.read_file(path)

# Coordinates in attribute table match those in the geometry
df['X_Coord_dif'] = df['X_Coord'] - df.geometry.x
df['Y_Coord_dif'] = df['Y_Coord'] - df.geometry.y

df9 = df.copy()
df.describe().loc[['min', 'max'],] # look at min and max for the _dif columns

In [ ]:
df9 = df9.rename(columns={'SITENO':'SITE_NO'})
df9 = df9.to_crs(df8.crs)
df = pd.merge(df8, df9[['SITE_NO', 'geometry']], 'outer', on='SITE_NO', suffixes=['_8', '_9'])
df['geo_dif'] = df['geometry_8'].distance(df['geometry_9'])
df['site_int'] = pd.to_numeric(df['SITE_NO'])

df[['SITE_NO', 'geo_dif', 'geometry_8', 'geometry_9']].sort_values('geo_dif', ascending=False)

In [ ]:
# Check list of sites in 2008 woody against 08/09 points
path = r"J:\projects\ECOFOR\ancillary_data\VCA\Woody VCA data - all years.xlsx"
year = "2008"

ydict = {"2002":{"header":5, "sitecol":"SITE_NO"},
         "2005":{"header":2, "sitecol":"SITE_NO"},
         "2008":{"header":2, "sitecol":"SITE NUMBER"}}
yvals = ydict[year]
dfw = pd.read_excel(path, sheet_name=year+" original VCA data", header=yvals["header"])

# dfw = gpd.GeoDataFrame(dfw, geometry=gpd.points_from_xy(dfw['Longitude'], dfw['Latitude'], crs=4326))
# dfw = dfw.to_crs(df.crs)

In [ ]:
# Compare lists of unique site numbers to find sites with woody data but no point coordinates
site_nums = dfw[yvals["sitecol"]].unique()
site_nums.sort()

mask = ~np.isin(site_nums, df['site_int'].unique())
missing = site_nums[mask]

display(dfw[dfw[yvals['sitecol']].isin(missing)])

In [ ]:
dfw[yvals['sitecol']].unique().shape

In [ ]:
# Create point file of plots with woody data in each year
# (just doing 2008 for now)
gdf = df[['SITE_NO', 'site_int', 'geometry_9']]

In [ ]:
gdf = gdf.rename(columns={'geometry_9':'geometry'})

In [ ]:
mdf = pd.merge(gdf, dfw[['.drop_duplicates(yvals['sitecol']), how='left', left_on='site_int', right_on=yvals['sitecol'])

In [ ]:
gdfw = gdf[gdf['site_int'].isin(dfw[yvals['sitecol']].unique())]

In [ ]:
gdfw = gpd.GeoDataFrame(gdfw, geometry='geometry', crs=gdfw.crs)

In [ ]:
gdfw.to_file(r"G:\temp\vca_woody2008_withgeo.gpkg", driver="GPKG")

In [ ]:
gdfw

### Site polygons
Create site polygons based on GPS data and location description & protocol. Maybe orient or match site based on visible trees and woody data?

## Woody basal area
Calculate live woody basal area using trees measured in circles B & C along with their basal diameters and total stems. Use the B & C expansion factors to convert this to a per area quantity.

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\Woody VCA data - all years.xlsx"
dfw = pd.read_excel(path, sheet_name='2005 original VCA data', header=2)

dfw.columns = dfw.columns.str.lower()
dfw = dfw.rename(columns={'site_no':'site'})
dfw = dfw.sort_values('sitepntcir')

In [ ]:
# Circle A usually doesn't have basal diameter measured, so it's excluded
dfa = dfw[dfw['circle']=='A'].copy()
dfa['basal_dia'].value_counts(dropna=False)

In [ ]:
dfw = dfw[dfw['circle']!='A'].copy()

### QC
Just a quick QC to note some of the logical inconsistencies in the data

In [ ]:
# Some sites don't have a record for every point
point_cnt = dfw.drop_duplicates(['site', 'point_no']).groupby('site')['point_no'].size()
point_cnt[point_cnt<8]

In [ ]:
print((dfw['sitepntcir'].str[-1]!=dfw['circle']).sum(), "record with the circle listed in SITEPNTCIR doesn't match CIRCLE")
print((dfw['sitepntcir'].str[-3:-1]!=dfw['point_no'].astype(str)).sum(), "records with the point_no listed in SITEPNTCIR not matching POINT_NO")
print((dfw['sitepntcir'].str[:4]!=dfw['site'].astype(str).str.zfill(4)).sum(), "records with the site listed in SITEPNTCIR not matching site")

In [ ]:
# recreate sitenptcir
# Note: it appears the sitepntcir is correct in some cases because using circle groups a NIL with other species
dfw['sitepntcir2'] = dfw['site'].astype(str).str.zfill(4) + " " + dfw['point_no'].astype(str) + dfw['circle']

In cases where the circle in sitepntcir differs from circle, it is usually that the sitepntcir is 'C' but the tree is <3 so listed as 'B' in circle. Meaning B is correct with regard to height, but was the tree actually measured in circle B or C? This is necessary for determining which plot expansion factor to use.

In [ ]:
# Max height when sitepntcir is C and circle is B
mask = (dfw['sitepntcir'].str[-1]=='C') & (dfw['circle']=='B')
sub = dfw.loc[mask, ['sitepntcir', 'circle', 'name', 'a_ttl_hgt']]
print(sub['a_ttl_hgt'].max(), 'm max height when sitepntcir says C and circle says B')

# Likewise, where B in sitepntcir and C in circle, the height is above 3
mask = (dfw['sitepntcir'].str[-1]=='B') & (dfw['circle']=='C') & (dfw['a_ttl_hgt']!=0)
sub = dfw.loc[mask, ['sitepntcir', 'circle', 'name', 'a_ttl_hgt']]
print(sub['a_ttl_hgt'].min(), 'm min height when sitepntcir says B and circle says C')

In [ ]:
# Trying to find out which is correct with respect to actual circle measured for the sake of expansion factors - sitepntcir or circle
mask = (dfw['sitepntcir'] != dfw['sitepntcir2']) & (dfw['name']=='NIL')
dfw[mask]

In [ ]:
# Some rows without a listed species have a valid diameter
# In some cases this looks like the diameter was written in the wrong row
mask = (dfw['basal_dia']>0) & (dfw['name']=='NIL')
dfw.loc[mask, ['year', 'sitepntcir', 'site', 'point_no', 'circle', 'basal_dia']]

In [ ]:
# More often a named tree doesn't have a diameter
mask = (dfw['basal_dia']==0) & (dfw['name']!='NIL')
dfw.loc[mask]

In [ ]:
# Stem pattern may be null and total stems may be 0, but there is a diameter. These are likely single stem
dfw.loc[(dfw['stem_ptn'].isnull() & dfw['basal_dia'].notna()), ['name', 'stem_ptn', 'ttl_stems', 'basal_dia']]

In [ ]:
# Stem pattern may be Multi, but total stems is 0 or null
dfw.loc[(dfw['stem_ptn']=='M') & (dfw['ttl_stems'].isnull() | (dfw['ttl_stems']==0))]

In [ ]:
# Basal diameter > 0 but no stems listed
mask = ((dfw['ttl_stems']==0) | (dfw['ttl_stems'].isnull())) & (~dfw['stem_ptn'].isin(['S', 'L', 'T'])) & (dfw['basal_dia']>0)
dfw.loc[mask, 'stem_ptn'].value_counts(dropna=False)

In [ ]:
# Total stems greater than 0 but no basal diameter
mask = (dfw['ttl_stems']>0) & ((dfw['basal_dia']==0) | (dfw['basal_dia'].isna()))
dfw.loc[mask, ['year', 'sitepntcir', 'stem_ptn', 'ttl_stems', 'basal_dia']]

In [ ]:
# TODO: Ridiculous # of stems and/or basal diameter

In [ ]:
# Some basal diameters are absurdly large (e.g. comapi 3 m wide)
dfw.groupby('name')['basal_dia'].max().sort_values(ascending=False)

In [ ]:
dfw.loc[dfw['name'] == 'COMBRETUM APICULATUM', 'basal_dia'].hist(bins=30)

In [ ]:
# Some stem counts are absurdly high (e.g. 81 stems for single colmop)
dfw.groupby('name')['ttl_stems'].max().sort_values(ascending=False)

### Prep & Calc

Make some assumptions in correcting data for now since fixing all the errors would take too long.

In [ ]:
# Drop all incomplete sites for now (some of these are site typos)
point_cnt = dfw.drop_duplicates(['site', 'point_no']).groupby('site')['point_no'].size()
incomplete_sites = point_cnt[point_cnt<8].index.values
dfw = dfw[~dfw['site'].isin(incomplete_sites)]

In [ ]:
# Fix hgt though not used
dfw['a_ttl_hgt'] = dfw['a_ttl_hgt'].replace({-99.9:np.nan, -9:np.nan})

In [ ]:
# Assume NIL is wrong for now(likely basal_dia or name in wrong row)
mask = (dfw['name']=='NIL') & (dfw['basal_dia']!=0)
dfw.loc[mask, 'name'] = 'UNK'

In [ ]:
# Dead
# Age has +,-,=,_ which are not in the manual
dfw['dead'] = False
mask = (dfw['imp_ele'].isin(['A', 'B', 'C'])) | (dfw['imp_age']=='D') | (dfw['fire_type'].isin(['A', 'B']))
dfw.loc[mask, 'dead'] = True                                              

In [ ]:
# Assume empty total # of stems is actually 1 when basal_dia is filled; regardless of stem_ptn
mask = ((dfw['ttl_stems'].isnull()) | (dfw['ttl_stems']==0)) & (dfw['basal_dia']>0)
dfw.loc[mask, 'ttl_stems'] = 1

In [ ]:
# Basal area per tree in m2
dfw['ba_tree'] = dfw['ttl_stems'] * (np.pi * (dfw['basal_dia']/2)**2) / 100**2  # converting cm2 to m2

In [ ]:
# sum basal area for all B and C circles on the site
ba_circle = dfw.groupby(['site', 'circle'])['ba_tree'].sum().reset_index()

In [ ]:
# Pivot and calculate site basal areas
ba = ba_circle.pivot(index='site', columns='circle', values='ba_tree')
ba.columns = ['ba_Bm2', 'ba_Cm2']

# Get area of site circles in hectares
b_circle_m2 = np.pi * 2**2 # 2 m radius
b_site_ha = b_circle_m2 * 8 / (100**2)
c_circle_m2 = np.pi * 5**2 # 5 m radius
c_site_ha = c_circle_m2 * 8 / (100**2)

# Apply expansion factor to get m2 / ha
ba['ba_B'] = ba['ba_Bm2'] / b_site_ha
ba['ba_C'] = ba['ba_Cm2'] / c_site_ha
ba['ba'] = ba['ba_B'] + ba['ba_C']
ba['ba_pct'] = ba['ba'] / 100**2 * 100

In [ ]:
# # Get rid of sites with basal area > measured area
# ba = ba[ba['ba_pct']<100]

In [ ]:
# Hist
ba['ba_pct'].hist(bins=20, figsize=(3,2))

In [ ]:
# Much of the data has a basal area less than 0.4%
mask = ba['ba_pct'] <= .5
ba.loc[mask, 'ba_pct'].hist(bins=30, figsize=(3,2))

In [ ]:
mask = ba['ba_pct'] >= 0.5
ba.loc[mask, 'ba_pct'].hist(bins=20, figsize=(3,2))

In [ ]:
# Some basal area is over 80 m2/ha (typical of rainforest) and into the 1000's (absurd)
fig, ax = plt.subplots(figsize=(3, 2))
ba.loc[ba['ba']<=60, 'ba'].hist(bins=30, ax=ax)#, xlims=(0,60))
# ax.set(xlim=(0,60))
ax.vlines([5, 20, 60], 0, 70, colors='k', linestyles='--')
ax.set(ylabel='count', xlabel='basal area m2/ha')

## Herbaceous  
Load herbaceous standing crop. Check data for anomalies. Output clean version.  
Most data is only disc height and standing crop per site. Only 2015 has individual measurements.

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\VCA data 1998-2015.xlsx"
dfh1 = pd.read_excel(path, sheet_name='1989-2006 Standing crop',
                  names = ['year', 'site', 'herb_disc', 'herb_agb'])

dfh1.loc[dfh1['year']<2000, 'year'] += 1900 # Fix year

# add date and assume its end of wet season
dfh1['date'] = dfh1['year'].apply(lambda y: pd.to_datetime(str(y)+'-4-30'))

path = r"J:\projects\ECOFOR\ancillary_data\VCA\VCA data 1998-2015.xlsx"
dfh2 = pd.read_excel(path, sheet_name='2007-2012 Stanidng Crop', usecols=list(range(1,6)),
                  names = ['date', 'section', 'site', 'herb_disc', 'herb_agb'], 
                  parse_dates=[0])

# Filter out bad site names
dfh2 = dfh2[dfh2['site'].apply(lambda x: type(x) is int)]

# Drop missing AGB
dfh2 = dfh2[dfh2['herb_agb']>=0]

# drop missing dates
dfh2 = dfh2[dfh2['date'].notna()]

# replace 2071 with 2010
dfh2.loc[dfh2['date']=='2071-05-28', 'date'] = pd.to_datetime('2010-05-28')

# get year
dfh2['year'] = dfh2['date'].dt.year

# Merge
dfh = pd.concat([dfh1, dfh2], ignore_index=True)

In [ ]:
# check that standing crop is calculated based on latest allometrics, and fix errors.
sns.scatterplot('herb_disc', 'herb_agb', hue='year', data=dfh1.loc[dfh['year']==2005])

In [ ]:
fig, ax = plt.subplots(figsize=(3,2))
sns.histplot(x='herb_agb', data=dfh1[dfh1['year']==2005], ax=ax)
ax.set(xlabel="Herb AGB (kg/ha)")
ax.vlines([2500, 5000], 0, 80, colors='k', linestyle='--')

**TODO: Compare herbaceous aerial cover and biomass**

## Burn
It looks like 'burn effectiveness' is partially encoded in 'burnt' and 'burn_type' through a mix of codes. Some of these can be decifered with safe assumptions, but others are an educated guess or just missing.  

There is no burn table for 2007-2012.

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\VCA data 1998-2015.xlsx"
dfb = pd.read_excel(path, sheet_name='1989-2006 Burn data', 
                    names = ['year', 'site', 'burnt', 'burn_type'])

dfb.loc[dfb['year']<2000, 'year'] += 1900  # Fix year
dfb = dfb.dropna(subset=['year', 'site'])  # drop nulls
dfb['year'] = dfb['year'].astype(int)

In [ ]:
# Create a new burn effectiveness column by making some
# assumptions about the mixed codes
burn_dict = {0:'unburnt',
             -9:'unburnt',
             '-':'unburnt',
             '?':'unburnt',  # this may be a bad assumption
             'No':'unburnt',
             'N':'unburnt',
             'Z':'unburnt',
             np.nan:'unburnt',
             'Y':'burnt',   # 'burnt' means no effectiveness code
             1:'burnt',
             'A':'very_poor',
             'B':'poor',
             'P':'poor',
             'Poor':'poor',
             'C':'moderate',
             'Moderate':'moderate',
             'M':'moderate', # this should be a safe assumption
             'D':'clean',       
             'Clean':'clean'
            }

dfb = dfb.replace({'burnt':burn_dict, 'burn_type':burn_dict})
dfb['burn_eff'] = dfb['burn_type'].replace({'none':None})
dfb['burn_eff'] = dfb['burn_eff'].fillna(dfb['burnt'])

dfb[['burnt', 'burn_type', 'burn_eff']].value_counts(dropna=False).sort_index()

In [ ]:
dfb['year'].value_counts(dropna=False)

## 2005 VCA structure modeling

### Combine 2005 for export
Run the location, basal area, herb, and burn sections above and then combine 2005 data from those dataframes to output a 2005 geodataframe.

In [ ]:
dfb05 = dfb[dfb['year']==2005].set_index('site')[['burn_eff']]
dfh05 = dfh[dfh['year']==2005].set_index('site')[['herb_disc', 'herb_agb']]
df = pd.concat([gdf, ba, dfh05, dfb05], axis=1)

# project to 36 N to match predictor layers
df = df.to_crs(epsg=32636)

df.reset_index().to_file(r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_2005_merged.gpkg", driver="GPKG")

### Extract predictors

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_2005_merged.gpkg"
df = gpd.read_file(path)
df = df.dropna(subset='geometry')

# predictor layers to extract from as prefix:path
layers = {
          "ltdry":r"K:\ECOFOR\lt\dry\lt_dry_2005.vrt",
          "ltwet":r"K:\ECOFOR\lt\wet\lt_wet_2005.vrt",
          "palsar":r"K:\ECOFOR\palsar\palsar_2007.vrt"
          "ccdc":r"C:\scratch\ECOFOR\ccdc\20050430\ccdc_coefs_segpre20050430_greaterkruger.vrt"
         }

for name, path in layers.items():
    with rasterio.open(path) as src:
        bands = src.descriptions
        crs = src.crs
    
    if df.crs!=crs:
        print("CRS mismatch. Skipping.")
        continue
    
    def extract_vals(band, band_name):
        vals = list(gen_point_query(df['geometry'], path, band=band+1, interpolate='nearest'))
        return pd.Series(vals, index=df.index, name=name+'_'+band_name)

    val_series = Parallel(n_jobs=4)(delayed(extract_vals)(band, band_name) for band, band_name in enumerate(bands))
    valdf = pd.concat(val_series, axis=1)
    df = pd.merge(df, valdf, 'left', left_index=True, right_index=True)

# df.to_file(r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_2005_merged_rs.gpkg", driver="GPKG")

### Load existing

In [ ]:
path = r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_2005_merged_rs.gpkg"
df = gpd.read_file(path)

### Filtering

In [ ]:
# filter out unrealistic basal area. Savanna is 0-20 m2/ha, PNW rainforest reaches like 80 m2/ha max.
# This removes like 200 plots
df = df[df['ba'] < 60]

In [ ]:
# drop null herbaceous
df = df.dropna(subset='herb_agb')

In [ ]:
# Remove plots with a large discrepancy in coordinates between the site details and vca data sheets
# This removes like 23 plots
df = df[(df['coords_dif'].isna()) | (df['coords_dif'] < 100)]

In [ ]:
# TODO: Remove plots with B/C discrepancy or containing unrealistic stem counts or tree basal diameters?

### Create classes

Basal area and herbaceous biomass appear to not be related and there is no obvious clustering; even when log transforming either or both variables.

In [ ]:
# Examine histogram; maybe better to decide based on understanding of ecology there though
g = sns.jointplot(x='herb_agb', y='ba', data=df, height=3, kind='hex')
g.ax_joint.set(xlabel='Herb AGB (kg/ha)', ylabel='BA (Mg/ha)')
g.ax_joint.vlines([2500, 5000], 0, 60, colors='k', linestyle='--')
g.ax_joint.hlines([5, 20], 0, 8400, colors='k', linestyles='--')

In [ ]:
# Create classes
df['ba_class'] = pd.cut(df['ba'], [0,5,20,60], labels=['Wlo', 'Wmed', 'Whi'], include_lowest=True).astype(str)
df['herb_class'] = pd.cut(df['herb_agb'], [0,2500, 5000, 9000], labels = ['Hlo', 'Hmed', 'Hhi'], include_lowest=True).astype(str)

df['mixed_class'] = df['ba_class'] + '_' + df['herb_class']

In [ ]:
# recode to ints for mapping
code_dict = {
    'Wlo_Hlo':1,
    'Wlo_Hmed':2,
    'Wlo_Hhi':3,
    'Wmed_Hlo':4,
    'Wmed_Hmed':5,
    'Wmed_Hhi':6,
    'Whi_Hlo':7,
    'Whi_Hmed':8,
    'Whi_Hhi':9
}
df['mixed_code'] = df['mixed_class'].replace(code_dict).astype(np.uint8)

In [ ]:
# Drop LT from columns to align with rasters used for prediction
df.columns = df.columns.str.replace("^lt", "", regex=True)

**TODO: Compare herbaceous aerial cover and biomass**

### Models

In [ ]:
# Mixed class model
Xcols = df.columns[df.columns.str.startswith('dry') | df.columns.str.startswith('wet')].tolist() #, 'ccdc', , 'palsar'
ycol = 'mixed_code'
mdf = df[~df[Xcols+[ycol]].isnull().any(axis=1)] # drop nulls (should be none)
X, y = mdf[Xcols], mdf[ycol]

# Run model and show results
rf = RandomForestClassifier(n_estimators=500, max_features='sqrt', oob_score=True, random_state=0, n_jobs=1)
rf = rf.fit(X, y)

# get OOB predictions and importance values
ypred = pd.Series(rf.classes_[rf.oob_decision_function_.argmax(axis=1)], index=y.index)
imps = pd.Series(rf.feature_importances_, index=X.columns)

# confusion matrix of OOB predictions for all samples
fig = pretty_matrix(y, ypred, normalize=False, outline_diag=True);
display(fig)
print(classification_report(y, ypred))
print(imps.sort_values(ascending=False))

In [ ]:
model_path = r"C:\scratch\ECOFOR\habitat\vca2005_mixed_v1.joblib"
joblib.dump(rf, model_path)

In [ ]:
# Herb AGB regression 
Xcols = df.columns[df.columns.str.startswith(('lt', 'palsar'))].tolist() #'ccdc'
ycol = 'herb_agb'
mdf = df[~df[Xcols+[ycol]].isnull().any(axis=1)] # drop nulls (should be none)
X, y = mdf[Xcols], mdf[ycol]
rf = RandomForestRegressor(n_estimators=200, max_features='sqrt', oob_score=True, random_state=0, n_jobs=4)
rf = rf.fit(X, y)

ypred = pd.Series(rf.oob_prediction_, index=y.index)
imps = pd.Series(rf.feature_importances_, index=X.columns)
fig = obs_pred_hexbin(y, ypred, vmax=10);
fig.suptitle('Herb AGB (kg/ha)')
print("N=", len(y))
print(imps.sort_values(ascending=False))

In [ ]:
# Basal area regression 
Xcols = df.columns[df.columns.str.startswith(('lt', 'palsar', 'ccdc'))].tolist()
ycol = 'ba'
mdf = df[~df[Xcols+[ycol]].isnull().any(axis=1)] # drop nulls (should be none)
X, y = mdf[Xcols], mdf[ycol]
rf = RandomForestRegressor(n_estimators=200, max_features='sqrt', oob_score=True, random_state=0, n_jobs=4)
rf = rf.fit(X, y)

ypred = pd.Series(rf.oob_prediction_, index=y.index)
imps = pd.Series(rf.feature_importances_, index=X.columns)
fig = obs_pred_hexbin(y, ypred, vmax=10);
fig.suptitle('Basal Area (Mg/ha)')
print("N=", len(y))
print(imps.sort_values(ascending=False))

### Post-classification Clean-up

In [ ]:
path = r"J:\projects\ECOFOR\veg_type\mixed_2005.tif"
outpath = path[:-4]+"_sieve5.tif" 
sieve_path = r"C:\OSGeo4W\apps\Python39\Scripts\gdal_sieve.py"
cmd = "python " + sieve_path + " -st 5 -8 " + path + " " + outpath[:-4]+"_temp.tif"
stdout = subprocess.check_output(cmd)

# Can't seem to use creation options in gdal_sieve so compressing the gdal_translate
cmd  = "gdal_translate -co COMPRESS=LZW -co TILED=YES -co PREDICTOR=2 " + outpath[:-4]+"_temp.tif" + " " + outpath
stdout = subprocess.check_output(cmd)
os.remove(outpath[:-4]+"_temp.tif")

## Herbaceous biomass modeling
First run herbaceous section above to get datasheets assembled.  
Then extract predictor values and run models.

### Prep data

In [ ]:
dfg = gpd.read_file(r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_sites.gpkg")
df = pd.merge(dfh, dfg[['site', 'coords_dif', 'geometry']], 'left', on='site')

df['has_geo'] = df['geometry'].notna()
# df.groupby(['year', 'has_geo'], dropna=False).size()

# Site numbers may not have been standardized till 2000
df = df[df['year']>=2000]

# drop rows without geometry
df = df[df['has_geo']]

# drop null agb
df = df[df['herb_agb']>0]

# Get geo back
df = gpd.GeoDataFrame(df, geometry='geometry', crs=dfg.crs)
df['x'], df['y'] = df.geometry.x, df.geometry.y

In [ ]:
# Use Dask and process images by row   
def get_ls_data(r, year_col, krads=[0,0], ls_dir="", basename="", bands=[]):
    """
    r: row in a dataframe
    krads: list of kernel radii (rows, cols) for extraction. [0,0] is point extraction of single pixel.
    """
    if (np.isnan(r[year_col])) or (r[year_col]==-9999):
        return np.full(len(bands), np.nan)
    
    path = os.path.join(ls_dir, basename+'{:4.0f}'.format(r[year_col])+".vrt")
    if not os.path.exists(path):
        return np.full(len(bands), np.nan)
        
    with rasterio.open(path) as src:
        row, col = rasterio.transform.rowcol(src.transform, r['x'], r['y']) #r.geometry.x, r.geometry.y)# 
        window = rasterio.windows.Window.from_slices((row-krads[0], row+krads[0]+1), (col-krads[1], col+krads[1]+1)) # +1 b/c slice end exclusive
        arr = src.read(window=window).astype(np.float32)
        arr[arr==src.nodata] = np.nan
        vals = np.nanmean(arr, axis=(1,2))
    return vals

In [ ]:
# Extract LandTrendr data at each site (Takes ~16 minutes)
## It might be faster to just read the entire image and extract for every point at once (imgs only ~2-3 gb)
## Or run point_query on each image
ls_dir = r"K:\ECOFOR\lt\dry"
basename="lt_dry_"
ncores = 6
krads = [0,0] # use mean of 3x3 window (i.e. kernel radius of 1)

with rasterio.open(os.path.join(ls_dir, basename+"2010.vrt")) as src:
    bands = list(src.descriptions)
    if src.crs!=df.crs:
        raise Exception("Coordinate systems of the raster and dataframe don't match")
        
ddf = dd.from_pandas(df, npartitions=ncores)
empty = pd.DataFrame(columns=list(range(len(bands))))
proc = ddf.map_partitions(lambda df: df.apply(get_ls_data, axis=1, result_type='expand', year_col="year", krads=krads, basename=basename, ls_dir=ls_dir, bands=bands), meta=empty)
result = proc.compute(scheduler="processes")
cols = [basename+b for b in bands]
result.columns = cols
df.loc[:,cols] = result

In [ ]:
# Extract LandTrendr data at each site
ls_dir = r"K:\ECOFOR\lt\wet"
basename="lt_wet_"
ncores = 6
krads = [0,0] # use mean of 3x3 window (i.e. kernel radius of 1)

with rasterio.open(os.path.join(ls_dir, basename+"2010.vrt")) as src:
    bands = list(src.descriptions)
    if src.crs!=df.crs:
        raise Exception("Coordinate systems of the raster and dataframe don't match")
        
ddf = dd.from_pandas(df, npartitions=ncores)
empty = pd.DataFrame(columns=list(range(len(bands))))
proc = ddf.map_partitions(lambda df: df.apply(get_ls_data, axis=1, result_type='expand', year_col="year", krads=krads, basename=basename, ls_dir=ls_dir, bands=bands), meta=empty)
result = proc.compute(scheduler="processes")
cols = [basename+b for b in bands]
result.columns = cols
df.loc[:,cols] = result

In [ ]:
# df.to_file(r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_herb_lt.gpkg", driver="GPKG")

### Model

In [ ]:
# Load existing data
df = gpd.read_file(r"J:\projects\ECOFOR\ancillary_data\VCA\derived\vca_herb_lt.gpkg")

In [ ]:
# Filtering

# Remove plots with a large discrepancy in coordinates between the site details and vca data sheets
# This removes like 300 plots
df = df[(df['coords_dif'].isna()) | (df['coords_dif'] < 100)]

# Remove plots with really high agb
df = df[df['herb_agb']<10000]

# TODO: Remove burns?


In [ ]:
def obs_pred_hexbin(y, x, folds=None, vmax=100):
    """y=true, x=pred, 
       k = Series of fold index in x and y used in K-fold cross-validation. Calculate mean error across folds if given.
    """
    fig, ax = plt.subplots(figsize=(4.5,3.75))   #(3, 2.5)
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=vmax)
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('counts')
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')
    
    if folds is not None:
        fdf = pd.DataFrame({'true':y, 'pred':x, 'fold':folds})
        folded = fdf.groupby('fold')
        r2 = folded.apply(lambda g: r2_score(g['true'], g['pred'])).mean()
        bias = folded.apply(lambda g: (g['pred'] - g['true']).mean()).mean()
        rmse = folded.apply(lambda g: mean_squared_error(g['true'], g['pred'])**0.5).mean()
    else:
        r2 = r2_score(y, x)
        bias = (x-y).mean()
        rmse = mean_squared_error(y, x)**0.5
    
    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(r2.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')
    ax.set(xlabel='Predicted', ylabel='Observed')
    return fig

In [ ]:
# Herb AGB regression 
Xcols = df.columns[df.columns.str.startswith('lt')].tolist()
ycol = 'herb_agb'
mdf = df[~df[Xcols+[ycol]].isnull().any(axis=1)] # drop nulls (should be none)
X, y = mdf[Xcols], mdf[ycol]
rf = RandomForestRegressor(n_estimators=500, max_features='sqrt', oob_score=True, random_state=0, n_jobs=4)
rf = rf.fit(X, y)

ypred = pd.Series(rf.oob_prediction_, index=y.index)
imps = pd.Series(rf.feature_importances_, index=X.columns)
fig = obs_pred_hexbin(y, ypred, vmax=100);
fig.suptitle('Herb AGB 2000-2012 (kg/ha)')
print("N=", len(y))
# print(imps.sort_values(ascending=False))

# Burn perimeters

In [ ]:
# Merge fire peremiters
indir = r"J:\projects\ECOFOR\ancillary_data\knp_fires"
paths = glob(os.path.join(indir, "*.shp"))

In [ ]:
# drop duplicates and keep UTM version
drop_paths = ['J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2014fires.shp',
             'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2015fires.shp',
             'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2016fires.shp',
             'J:\\projects\\ECOFOR\\ancillary_data\\knp_fires\\2017fires.shp']
paths = [p for p in paths if p not in drop_paths]

# CCDC Veg Exploration
Explore CCDC metrics for 2015 segment to see if they correspond to vegetation composition or structure

## Unsupervised classification

In [ ]:
path = r"C:\scratch\ecofor\ccdc\ccdc_coefs_segpre20150301_kruger.vrt"
aoi_path = r"J:\projects\ECOFOR\boundaries\kruger_utm36n.tif"

with rasterio.open(aoi_path) as src:
    aoi = src.read(1).astype(bool)

with rasterio.open(path) as src:
    arr = src.read()
    bands = src.descriptions

In [ ]:
# Arr to dataframe and sample
tab = arr.transpose([1,2,0]).reshape(arr.shape[1]*arr.shape[2], arr.shape[0], order='C')
df = pd.DataFrame(tab, columns=[s.lower() for s in bands])

# Use sample
# df = df.sample(10000, weights = aoi.ravel(), random_state=0)

# Or drop no data and data outside the park
df = df[aoi.ravel()]
df = df.dropna()

# clear some ram (might be less ram intensive to sample array first)
del arr, tab

In [ ]:
Xcols = df.columns.copy()

### Build models

#### K-means

In [ ]:
# mini-batch K-means
from sklearn.cluster import MiniBatchKMeans
from joblib import cpu_count
cores = cpu_count() - 2

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=8,
                         batch_size=256*cores,
                         init="k-means++",
                         max_iter=10,
                         n_init=5,
                         random_state=0
                        )
kmeans.fit(df[Xcols])

df['kmeans'] = kmeans.labels_

#### Mean shift

#### Ward

#### Gaussian mixture model

### Evaluate clusters

In [ ]:
# TODO: Hexbin pairplot without clusters

In [ ]:
# Plot clusters in pairwise comparison of variables
cols = [col for col in Xcols for band in ['swir1', 'red', 'nir'] if band in col]
# cols = cols[:10] #+ ['kmeans']

g = sns.PairGrid(df.sample(500, random_state=0),vars=cols, hue='kmeans', palette='Set1', despine=False, height=3, diag_sharey=False)
g.map_upper(sns.scatterplot, alpha=0.4)
g.map_lower(sns.kdeplot, alpha=0.6)
g.map_diag(sns.kdeplot)

for ax in g.axes.flat:
    ax.tick_params(axis='both', labelleft=True, labelbottom=True)
    ax.set_xlabel(ax.xaxis.get_label_text(), visible=True)
    ax.set_ylabel(ax.yaxis.get_label_text(), visible=True)
    
plt.subplots_adjust(wspace=0.3, hspace=0.3)

In [ ]:
g.savefig(r'C:\scratch\ecofor\kmeans_all_cl8.pdf')

In [ ]:
# Metrics to indicate most important variables?

In [ ]:
# Metrics to show distance between clusters and spread within clusters

Pairwise plots show almost no clustering for most variables. NIR and RED mag show the most clear clustering.

### Apply model to image

In [ ]:
path = r"C:\scratch\ecofor\ccdc\ccdc_coefs_segpre20150301_kruger.vrt"
aoi_path = r"J:\projects\ECOFOR\boundaries\kruger_utm36n.tif"

with rasterio.open(aoi_path) as src:
    aoi = src.read(1).astype(bool)
    prof = src.profile

with rasterio.open(path) as src:
    arr = src.read()
    bands = src.descriptions

In [ ]:
# Arr to dataframe for prediction
tab = arr.transpose([1,2,0]).reshape(arr.shape[1]*arr.shape[2], arr.shape[0], order='C')
df = pd.DataFrame(tab, columns=[s.lower() for s in bands])

In [ ]:
# Apply model
pred = kmeans.predict(df).astype(np.uint8)
pred = pred.reshape(arr.shape[1], arr.shape[2])
pred[~aoi] = 255

In [ ]:
# Export to image
outpath = r"C:\scratch\ecofor\kmeans_all_cl8.tif"

prof['nodata'] = 255
with rasterio.open(outpath, 'w', **prof) as dst:
        dst.write(pred, 1)

### Compare to veg data

In [ ]:
# Count of each class in the pre-existing veg maps
vent_path = r"J:\projects\ECOFOR\ancillary_data\Venter\landtypes_venter1990.shp"
gert_path = r"J:\projects\ECOFOR\ancillary_data\Gertenbach\landscapes_gertenbach1983.shp"

vent = gpd.read_file(vent_path)
gert = gpd.read_file(gert_path)

# reproject
rast_path = r"C:\scratch\ecofor\kmeans_all_cl8.tif"
with rasterio.open(rast_path) as src:
    crs = src.crs
vent = vent.to_crs(crs)
gert = gert.to_crs(crs)

In [ ]:
# Add zonal stats as area
ventz = pd.DataFrame(zonal_stats(vent, rast_path, categorical=True), index=vent.index)
ventz = ventz.add_prefix('c')
vent = pd.concat([vent, ventz], axis=1)
class_cols = ventz.columns.tolist()
vent[class_cols] = vent[class_cols] * (30**2 / 1000**2) # convert to km2

In [ ]:
# group and sum areas for land types with multiple geometrys
vent = vent.drop(['geometry', 'SOIL'], axis=1)
vgrp = vent.groupby(vent.columns[vent.dtypes=='object'].tolist(), as_index=False).sum()

In [ ]:
# Get percent area of classes
vgrp['sum_km2'] = vgrp[class_cols].sum(axis=1)
pct_cols = ventz.add_suffix('_pct').columns.to_list()
vgrp[pct_cols] = vgrp[class_cols].divide(vgrp['sum_km2'], axis=0)*100

In [ ]:
# melt for plotting
id_vars = vgrp.columns[vgrp.dtypes=='object'].tolist()
vmelt = pd.melt(vgrp[pct_cols+id_vars], id_vars=id_vars, var_name='class', value_vars=pct_cols, value_name = 'pct')

In [ ]:
fig, ax = plt.subplots(figsize=(7, 25))
sns.scatterplot(y='DOMWOODY', x='pct', hue='class', data=vmelt, ax=ax, palette='Accent', y_jitter=True, x_jitter=True, s=100, alpha=0.8)

# Figures

## Paper figures

In [ ]:
# presentation set up
figdir = r"E:\My Drive\Work\ecofor\manuscript\figs"
os.makedirs(figdir, exist_ok=True)

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial']
mpl.rcParams['font.size'] = 8

sns.set_style('ticks',
               {'font.family':'sans-serif', 'font.sans-serif':['Arial'], 'font.size':8})

# temporary style to use on dark backgrounds
# white_axes = {'axes.labelcolor': '.99', 'text.color': '.99', 'xtick.color': '.99', 'ytick.color': '.99', 'axes.edgecolor': '.99', 'figure.facecolor': 'black'}

### Distribution of GEDI  
Show distribution of GEDI values for entire study area and by vegetation type.  

Maybe do as KDE lines or ECDF lines with "All" and then by veg type or land cover class as a hue.
Alternatively, also show the pairwise relationship with pairgrid by having a contour plot or hexgrid for bivariate distributions and a kde for univariate.

In [ ]:
path = r"J:\projects\ECOFOR\gedi\models\v08\GEDI_2AB_2019to2023_leafon_sampy500m_all_oob_v08.parquet"
df = gpd.read_parquet(path)

df['cover'] *= 100 # convert cover to %
ycol_dict = {'cover':'Cover (%)', 'rh98':'RH98 (m)', 'fhd_normal':'FHD'} #'pai':'PAI', 
df = df.rename(columns=ycol_dict)


# ldf = pd.melt(df[ycol_dict.values()])

In [ ]:
# Load land cover or veg type for hue
path = r"J:\projects\ECOFOR\gedi\extracted\GEDI_2AB_2019to2023_leafon_sampy500m_sanlc20.csv"
cdf = pd.read_csv(path).set_index('shot_number')

# Create a modification of salcc1 to separate out open woodland and group others
rat_path = r"J:\projects\ECOFOR\lcluc\SANLC\2020\SA_NLC_2020_GEO.tif.vat.dbf"
rat = gpd.read_file(rat_path).drop('geometry', axis=1)
rat['SALCC_1'] = rat['SALCC_1'].replace('Forested Land', 'Forested land')
mod_dict = rat.set_index('Value')['SALCC_1'].to_dict()
mod_dict[4] = 'Open Woodland'
cdf['sanlc20_salcc1_mod'] = cdf['sanlc20_val'].map(mod_dict)
other_mask = cdf['sanlc20_salcc1_mod'].isin([None, 'Built-up', 'Wetlands', 'Barren Land', 'Waterbodies', 'Mines & Quarries', 'Shrubland'])
cdf.loc[other_mask, 'sanlc20_salcc1_mod'] = 'Other'

# Merge GEDI with land cover classes
df = pd.merge(df, cdf, how='left', left_index=True,right_index=True)

In [ ]:
hue_order = df['sanlc20_salcc1_mod'].value_counts().index
palette =  ['#CDAA66', '#728944', '#FFAA00', '#CD6666', '#E9FFBE'] #'deep'#['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e']
# palette.reverse()

g = sns.PairGrid(df, vars=ycol_dict.values(), hue='sanlc20_salcc1_mod',
                 corner=True, height=1.2, diag_sharey=False,
                 hue_order=hue_order)
g.map_diag(sns.kdeplot, palette=palette)#, common_norm=False)
# g.map_diag(sns.ecdfplot, stat="proportion", alpha=.6, linewidth=1.5)
g.map_lower(sns.histplot, hue=None, bins=50, cmap = 'YlGn', vmin=50, vmax=2000)
lines = g.fig.axes[-1].get_lines()
lines.reverse()
g.fig.legend(lines, hue_order, loc=(0.63,0.75))

ax = g.fig.add_axes((0.635, 0.64, 0.3, 0.06))
norm = mpl.colors.Normalize(vmin=50, vmax=2000)
g.fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.YlGn),
             cax=ax, orientation='horizontal')

figname = "gedi_metrics_pairplot"
for ext in [".pdf", ".svg"]:
    figpath = os.path.join(figdir, figname + ext)
    g.fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

### Optical data availability  
Show count of L30 vs HLS scenes available over time to show how adding S2 fills gaps.

In [ ]:
# paired t-test of RMSE of CCDC fit using HLS vs L30 for the sample
from scipy.stats import ttest_rel
path = r"J:\projects\ECOFOR\gedi\extracted\GEDI_2AB_2019to2023_leafon_sampy500m_all.parquet"
df = gpd.read_parquet(path)

bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'] #'ca', 
cols = [col for col in df.columns for src in ['hls', 'l30'] for band in bands if col.startswith(src) and (band in col) and col.endswith('rmse')]

df = df[cols].dropna()
df.columns = pd.MultiIndex.from_tuples([col.split('_')[:2] for col in df.columns], names=["Source", "Band"])

for band in bands:
    print(band)
    print(ttest_rel(df[('hls', band)], df[('l30', band)]))

In [ ]:
# Get confidence interval for the differences
import scipy.stats as st
confidence_level = 0.95

sdf = df.stack()
sdf['dif'] = sdf['hls'] - sdf['l30']
ddf = sdf['dif'].unstack()

cis = ddf.apply(lambda x: st.t.interval(0.95, len(x)-1, loc=x.mean(), scale=st.sem(x)))
cis.index = ['low', 'high']
cis = cis.T
cis['mean'] = ddf.mean()
cis['half_width'] = (cis['high'] - cis['low']) / 2

# cis
with pd.option_context('display.precision', 2):
    display(cis)

### Model accuracy

#### All models

In [ ]:
# Load and setup data
path = r"J:\projects\ECOFOR\gedi\models\v08\GEDI_2AB_2019to2023_leafon_sampy500m_all_oob_v08.parquet"
df = gpd.read_parquet(path)

source_dict = {
    'lt-p-s-t': 'LandTrendr + PALSAR + Soils + Topo',
    'ccdcl30-p-s-t': '$CCDC^{L30}$ + PALSAR + Soils + Topo',
    'ccdchls-p-s-t': '$CCDC^{HLS}$ + PALSAR + Soils + Topo',
    'lt-p': 'LandTrendr + PALSAR',
    'ccdcl30-p': '$CCDC^{L30}$ + PALSAR',
    'ccdchls-p':'$CCDC^{HLS}$ + PALSAR',
    'p-s-t': 'PALSAR + Soils + Topo',
    'lt': 'LandTrendr',
    'ccdcl30': '$CCDC^{L30}$',
    'ccdchls': '$CCDC^{HLS}$',
    'p': 'PALSAR',
    's-t': 'Soils + Topography',
}


ydict = {'cover': 'Cover',
#          'pai': 'PAI',
         'rh98': 'RH98',
         'fhd_normal': 'FHD'}
Xsets = list(source_dict.keys())
Ycols = list(ydict.keys())

# make cover cols as percent
df[df.columns[df.columns.str.contains('cover')]] *= 100

In [ ]:
# Get stats for all models
sdf = pd.DataFrame(columns = pd.MultiIndex.from_product([list(ydict.values()), ['R2', 'RMSE', 'Bias', 'N']], names=("Metric", "Stat")))
sdf.index.name = 'Xset'

for Xset in Xsets:
    for ycol in Ycols:
        x, y = df['pred_'+Xset+'_'+ycol], df[ycol]
        
        # Save stats
        sdf.loc[Xset, (ydict[ycol], 'R2')] = r2_score(y, x)
        sdf.loc[Xset, (ydict[ycol], 'RMSE')] = mean_squared_error(y, x)**0.5
        sdf.loc[Xset, (ydict[ycol], 'Bias')] = (x-y).mean()
        sdf.loc[Xset, (ydict[ycol], 'N')] = len(y)
sdf = sdf.apply(pd.to_numeric, errors='coerce', axis=1)

sdf

In [ ]:
# Load and setup TCV data
tcv_path = r"J:\projects\ECOFOR\gedi\models\v08\GEDI_2AB_2019to2023_leafon_sampy500m_all_tcv_v08_stats.csv"
tdf = pd.read_csv(tcv_path)

tdf = tdf[tdf['metric']!='pai'] # Drop PAI

tdf['Metric'] = tdf['metric'].replace(ydict)
tdf = tdf.rename(columns={'n':'N', 'r2':'R2', 'rmse':'RMSE', 'bias':'Bias'})
tdf.loc[tdf['metric']=='cover', ['RMSE', 'Bias']] *= 100

tcv = tdf.groupby(['Xset', 'Metric']).mean(numeric_only=True)
tcv = tcv.drop(columns=['year'])
tcv.columns.name = 'Stat'

tldf = pd.melt(tcv, ignore_index=False, value_name='TCV').reset_index()
# tldf['Source'] = tldf['Xset'].map(source_dict)

In [ ]:
# Bar chart of accuracy stats

# Make long form and add TCV stats
ldf = pd.melt(sdf, ignore_index=False).reset_index()
ldf = pd.merge(ldf, tldf, how='left', on=['Xset', 'Metric', 'Stat'])
ldf['Source'] = ldf['Xset'].map(source_dict)

mask = ldf['Stat'].isin(['R2', 'RMSE', 'Bias'])

# palette = ['#a6cee3','#b2df8a','#cab2d6', '#1f78b4','#33a02c']
p = sns.color_palette(palette='tab20c')
palette = p[0:9:4]+p[1:10:4]+[p[12]]+p[2:11:4]+[p[13], p[16]]

g = sns.catplot(data=ldf[mask], x="value", y="Source", row="Stat", col="Metric", kind='bar',
                sharex=False, sharey=True, height=3, aspect=1.3, margin_titles=True, palette=palette)

# Overlay point plot with TCV stats
g.map(sns.pointplot, "TCV", "Source", marker="o", join=False, color="k")

# Clean up plot
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.set_ylabels("")

figname = "gedi_acc_all"
for ext in [".pdf", ".svg"]:
    figpath = os.path.join(figdir, figname + ext)
    g.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Percent difference of RMSE for each optical only model from the mean RMSE of those models
opt_rmse = sdf.loc[['lt', 'ccdcl30', 'ccdchls'], (slice(None), 'RMSE')]
100 * opt_rmse.subtract(opt_rmse.mean()).divide(opt_rmse)

In [ ]:
# Percent difference of chosen model RMSE from best model
opt_rmse = sdf.loc[['lt-p-s-t', 'ccdcl30-p-s-t', 'ccdchls-p-s-t'], (slice(None), 'RMSE')]
100 * opt_rmse.subtract(opt_rmse.mean()).divide(opt_rmse)

In [ ]:
# Comparison of LandTrendr only to PALSAR only
lt_minus_p_r2 = sdf.loc['lt', (slice(None), 'R2')] - sdf.loc['p', (slice(None), 'R2')]
display(lt_minus_p_r2)
print("LandTrendr explained", (lt_minus_p_r2.mean()*100).round(), "% more variance on average than PALSAR.")

In [ ]:
# Change in RMSE when adding PALSAR and Soils and Topo
opt_sar_rmse = sdf.loc[['lt-p', 'ccdcl30-p', 'ccdchls-p'], (slice(None), 'RMSE')]
opt_sar_rmse = opt_sar_rmse.set_index(opt_rmse.index)
mean_opt_sar_chg = np.nanmean(100* opt_sar_rmse.subtract(opt_rmse).divide(opt_rmse)).round(1)
print("RMSE changed", mean_opt_sar_chg, "% on average when adding PALSAR predictors to optical predictors")

# Change in RMSE when adding PALSAR and Soils and Topo
opt_pst_rmse = sdf.loc[['lt-p-s-t', 'ccdcl30-p-s-t', 'ccdchls-p-s-t'], (slice(None), 'RMSE')]
opt_pst_rmse = opt_pst_rmse.set_index(opt_rmse.index)
mean_opt_pst_chg = np.nanmean(100* opt_pst_rmse.subtract(opt_rmse).divide(opt_rmse)).round(1)
print("RMSE changed", mean_opt_pst_chg, "% on average when adding PALSAR and soil and topography predictors to optical predictors")

In [ ]:
# Get difference between TCV stats and OOB stats

# Reshape TCV stats to match OOB stats (sdf)
tcv_wide = tcv.unstack(level=1)
tcv_wide.columns = tcv_wide.columns.swaplevel()
tcv_wide.sort_index(axis=1, level=0, inplace=True)

sdif = tcv_wide.subtract(sdf)
spct = sdif.divide(sdf) * 100

r2_dif_mean = np.nanmean(sdif.loc[:,(slice(None), 'R2')]).round(2)
print("TCV R2 different from OOB R2 by", r2_dif_mean, "on average.")

rmse_pct_dif_mean = np.nanmean(spct.loc[:,(slice(None), 'RMSE')]).round(2)
print("TCV RMSE different from OOB RMSE by", rmse_pct_dif_mean, "% on average.")

In [ ]:
# Bias as % of mean observed value
# Mean observed value
mean_obs = df[['cover', 'rh98', 'fhd_normal']].mean()
mean_obs.index = ['Cover', 'RH98', 'FHD']

# OOB absolute bias
oob_bias = sdf.loc[:,(slice(None), 'Bias')].droplevel(1, axis=1).abs()
oob_bias_pct = oob_bias / mean_obs * 100

# TCV absolute bias
tcv_bias = tcv_wide.loc[:,(slice(None), 'Bias')].droplevel(1, axis=1).abs()
tcv_bias_pct = tcv_bias / mean_obs * 100

def get_df_max(df):
    col_max = df.max().idxmax()
    row_max = df[col_max].idxmax()
    max_val = df.loc[row_max, col_max]
    return max_val, (row_max, col_max)

val, (model, metric) = get_df_max(oob_bias_pct)
print("OOB absolute bias maximum as a percent of the mean observed value was", val.round(1), "% for", model, metric)

val, (model, metric) = get_df_max(tcv_bias_pct)
print("TCV absolute bias maximum  as a percent of the mean observed value was", val.round(1), "% for", model, metric)

#### Chosen model

In [ ]:
# Plot Obs vs pred for only the best model
# 1x4
Xset = 'lt-p-s-t' #'hlsp'
ycol_dict = {'cover':'Cover (%)','pai':'PAI', 'rh98':'RH98 (m)', 'fhd_normal':'FHD'}

fig, axes = plt.subplots(1, 3, figsize=(2.15*3, 1.5))

for i, (ycol, ax) in enumerate(zip(Ycols, axes.flat)):
    x, y = df['pred_'+Xset+'_'+ycol], df[ycol]
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

    r2 = r2_score(y, x)
    bias = (x-y).mean()
    rmse = mean_squared_error(y, x)**0.5

    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(np.round(r2, 2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+"{:.2f}".format(np.round(bias, 2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(np.round(rmse, 2)), transform=ax.transAxes, ha='right')

    ax.set(title=ycol_dict[ycol])
    if i==0:
        ax.set(ylabel='Observed')
    if i==1:
        ax.set(xlabel='Predicted')

fig.subplots_adjust(wspace=0.3)
cb = fig.colorbar(hb, ax=axes, location='right', orientation='vertical', pad=0.02)#, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
cb.set_label('Count')

figname = "gedi_acc_" + Xset
for ext in [".pdf", ".svg"]:
    figpath = os.path.join(figdir, figname + ext)
    fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Chosen model verseus model with lowest RMSE
chosen = "lt-p-s-t"
best_rmse = pd.concat([sdf.loc[:,(slice(None), "RMSE")].min(), 
                       sdf.loc[:,(slice(None), "RMSE")].idxmin()], axis=1, keys=['RMSE', 'Xset'])

print("Model with lowest RMSE")
display(best_rmse)

print("Difference of chosen model from best model")
display(sdf.loc[chosen, (slice(None), "RMSE")] - best_rmse["RMSE"])

print("Percent difference of chosen model from best model")
display((sdf.loc[chosen, (slice(None), "RMSE")] - best_rmse["RMSE"]) / best_rmse["RMSE"] * 100)

In [ ]:
# Number of CCDC and LandTrendr variables
lt_path = r"J:\projects\ECOFOR\gedi\extracted\GEDI_2AB_2019to2023_leafon_sampy500m_lt.csv"
ccdc_path = r"J:\projects\ECOFOR\gedi\extracted\GEDI_2AB_2019to2023_leafon_sampy500m_l30s2_ccdc.csv"

lt = pd.read_csv(lt_path, nrows=0).columns
ccdc = pd.read_csv(ccdc_path, nrows=0).columns

# Filter to same columns used in modeling
lt = lt.drop('shot_number')
bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'] #'ca', 
ccdc = [col for col in ccdc for band in bands if col.startswith(band)] # using CCDC

print("LandTrendr used", len(lt), "variables.")
print("CCDC used", len(ccdc), "variables.")

In [ ]:
# Summary of predicted and observed for chosen model
pred_cov_lt_1pct = df.loc[df['cover']<1, 'pred_'+chosen+'_cover'].mean()
print("Observed cover <1% was predicted as", pred_cov_lt_1pct, "% on average.")

mask = df['cover']>50
pred_cov_err_mask = (df.loc[mask, 'pred_'+chosen+'_cover'] - df.loc[mask, 'cover']).mean()
print("Observed cover >50% was overestimated by", pred_cov_err_mask, "% on average.")

In [ ]:
mask = df['rh98']>15
(df.loc[mask, 'pred_'+chosen+'_rh98'] - df.loc[mask, 'rh98']).mean()

In [ ]:
# TCV stats for chosen model
tldf[tldf['Xset']=='lt-p-s-t'].pivot(index=['Xset', 'Metric'], columns='Stat', values='TCV').round(2)

#### Bias correction
Bias correction results for chosen model. May need to update stats used above in paper to use the bias-corrected version.

In [ ]:
path = r"J:\projects\ECOFOR\gedi\models\v08BC\GEDI_2AB_2019to2023_leafon_sampy500m_all_bc_v08BC.parquet"
# path = r"J:\projects\ECOFOR\gedi\models\v08EDM\GEDI_2AB_2019to2023_leafon_sampy500m_all_bc_v08EDM.parquet"
df = pd.read_parquet(path)

In [ ]:
ycol_dict = {'cover':'Cover (%)', 'rh98':'RH98 (m)', 'fhd_normal':'FHD'}
Ycols=list(ycol_dict.keys())

fig, axes = plt.subplots(2, 3, figsize=(2.15*3, 3.1))

for pred_type, axrow in zip(['pred', 'pred_bc'], axes):
    for i, (ycol, ax) in enumerate(zip(Ycols, axrow)):
        x, y = df[pred_type+'_'+ycol], df[ycol]
        hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=1000)
        ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

        r2 = r2_score(y, x)
        bias = (x-y).mean()
        rmse = mean_squared_error(y, x)**0.5

        # add text
        ax.text(0.99, 0.22, "R$^2$= " + str(np.round(r2, 2)), transform=ax.transAxes, ha='right')
        ax.text(0.99, 0.13, "Bias= "+"{:.2f}".format(np.round(bias, 2)), transform=ax.transAxes, ha='right')
        ax.text(0.99, 0.02,  "RMSE= " + str(np.round(rmse, 2)), transform=ax.transAxes, ha='right')
        ytext = "Original" if pred_type=="pred" else "Bias-corrected"
        
        if pred_type=='pred':
            ax.set(title=ycol_dict[ycol])
            ax.set_xticklabels([])
        if i==0:
            ax.set(ylabel='Observed')
            ax.text(-0.5, 0.5, ytext, transform=ax.transAxes, ha='center', va='center', rotation='vertical', fontsize=10, fontweight='bold')
        if i==1 and pred_type=='pred_bc':
            ax.set(xlabel='Predicted')
            

fig.subplots_adjust(wspace=0.3)
cb = fig.colorbar(hb, ax=axes, location='right', orientation='vertical', pad=0.02)#, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
cb.set_label('Count')

# figname = "gedi_acc_biascorrection"
# for ext in [".pdf", ".svg"]:
#     figpath = os.path.join(figdir, figname + ext)
#     fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Compare distributions of the predictions and observations
from scipy.stats import ks_2samp, t

fig, axes = plt.subplots(1, 3, figsize=(6.5, 2))

for i, (ycol, ax) in enumerate(zip(Ycols, axes)):
    pred_col = 'pred_'+ycol
    bc_col = 'pred_bc_'+ycol
    xydf = pd.melt(df[[pred_col, bc_col, ycol]])
    sns.ecdfplot(xydf, x='value', hue='variable', ax=ax, legend=False)

    # ks_2samp silently gives wrong values if nan's included, so make sure they're removed
    mask = df[[pred_col, ycol]].notna().all(axis=1)
    ks, pval = ks_2samp(df.loc[mask, pred_col], df.loc[mask, ycol])
    ax.text(0.95, 0.16, f"Orig & Obs KS= {np.round(ks,2)}", ha='right', transform=ax.transAxes)
    
    mask = df[[bc_col, ycol]].notna().all(axis=1)
    ks, pval = ks_2samp(df.loc[mask, bc_col], df.loc[mask, ycol])
    ax.text(0.95, 0.1, f"BC & Obs KS= {np.round(ks,2)}", ha='right', transform=ax.transAxes)
    
    mask = df[[bc_col, pred_col]].notna().all(axis=1)
    ks, pval = ks_2samp(df.loc[mask, bc_col], df.loc[mask, pred_col])
    ax.text(0.95, 0.03, f"BC & Orig KS= {np.round(ks,2)}", ha='right', transform=ax.transAxes)

    ax.set(ylabel=None, title=ycol_dict[ycol])
        
orange_line = mpl.lines.Line2D([0], [0], color='orange', lw=2)
blue_line = mpl.lines.Line2D([0], [0], color='blue', lw=2)
green_line = mpl.lines.Line2D([0], [0], color='green', lw=2)
fig.legend([orange_line, blue_line, green_line], ['Bias-corrected Prediction', 'Original Prediction', 'Observed'], loc='center', bbox_to_anchor=(0.5,-0.15), ncol=3)

# figname = "gedi_bc_ecdf"
# for ext in [".pdf", ".svg"]:
#     figpath = os.path.join(figdir, figname + ext)
#     fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

### Field evaluation
Compare the field measurements to GEDI footprints and predicted maps.

In [ ]:
def present_regplot(x, y, ax, lims=None, reg=True, oneone=True, **kwargs):
    from scipy.stats import linregress
    
    sns.regplot(x=x, y=y, ax=ax, **kwargs)

    if reg:
        # Regression
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        rmse = mean_squared_error(y, x*slope+intercept)**0.5
        eq = "y = " + str(np.round(slope,2)) + "x + " + str(np.round(intercept, 2))
        ax.text(0.97, 0.26, "Regression:", transform=ax.transAxes, ha='right')
        ax.text(0.98, 0.17, eq, transform=ax.transAxes, ha='right')
        ax.text(0.98, 0.09, "R$^2$= " + str(np.round(r_value**2, 2)), transform=ax.transAxes, ha='right')
        ax.text(0.98, 0.01, "RMSE= "+str(np.round(rmse, 2)), transform=ax.transAxes, ha='right')

#         # check that line is the same as from sns.reglot
#         samp = np.arange(x.min(), x.max(), 1)
#         ax.plot(samp, intercept + slope * samp, 'r')

    if oneone:
        if lims is None:
            lims = (0, np.nanmax(x.append(y))) #np.nanmin(x.append(y))
        ax.plot(lims, lims, '--k')
        ax.set(ylim=lims, xlim=lims)

        # add text for R2 and RMSE
        r2 = r2_score(y, x)
        rmse = mean_squared_error(y, x)**0.5
        bias = (x-y).mean()
        ax.text(0.03, 0.93, "1:1 stats:", transform=ax.transAxes)
        ax.text(0.03, 0.85,"R$^2$= "+str(np.round(r2, 2)), transform=ax.transAxes)
        ax.text(0.03, 0.77, "RMSE= "+str(np.round(rmse, 2)), transform=ax.transAxes)
        ax.text(0.03, 0.69, "Bias= "+str(np.round(bias, 2)), transform=ax.transAxes)

In [ ]:
# Plot max tree height compared to GEDI's RH98
path = r"J:\projects\ECOFOR\field\merged\gedi_trees_cover_simp.csv"
df = pd.read_csv(path, index_col='plot_ix')

# Load plot notes to drop bad plots
path = r"J:\projects\ECOFOR\field\merged\plot_notes.csv"
pdf = pd.read_csv(path, index_col='plot_ix')

df[['exclude_plot', 'exclude_reason']] = pdf[['exclude_plot', 'exclude_reason']]
df = df[~df['exclude_plot']]

In [ ]:
# Get predicted RH98 for 2022 for the location of the tallest measured tree for comparison
trees = gpd.read_file(r"J:\projects\ECOFOR\field\merged\gedi_all_merged.gpkg", layer="trees")
rast_path = r"J:\projects\ECOFOR\gedi\maps\v08\lt-p-s-t\rh98\rh98_2022.tif"

with rasterio.open(rast_path) as src:
    trees = trees.to_crs(src.crs)
trees = trees.dropna(subset=['hgt', 'geometry'])

trees['pred_rh98'] = list(gen_point_query(trees, rast_path, interpolate='nearest'))

# Get tallest tree of kept plots
trees = trees[trees['plot_ix'].isin(df.index.values)]
tallest_ix = trees.groupby('plot_ix')['hgt'].idxmax().dropna()
trees = trees.loc[tallest_ix]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(4,2))
present_regplot(df['rh98'], df['hgt'], lims=(0,30), ax=ax1, scatter_kws={'s': 15, 'zorder':2, 'alpha':0.5}, line_kws={'color':'k', 'alpha':0.5, 'zorder':1})
present_regplot(trees['pred_rh98'], trees['hgt'], lims=(0,30), ax=ax2, scatter_kws={'s': 15, 'zorder':2, 'alpha':0.5}, line_kws={'color':'k', 'alpha':0.5, 'zorder':1})
ax1.set(xlabel='RH98 (m)', ylabel='Max tree height (m)')
ax2.set(xlabel='Predicted RH98 (m)', ylabel='Max tree height (m)')
fig.tight_layout()

figname = "field_height"
for ext in [".pdf", ".svg"]: 
    figpath = os.path.join(figdir, figname + ext)
    fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Plot without the tall tree observation
fig, ax = plt.subplots(figsize=(2,2))
mask = trees['plot_ix']!=14
present_regplot(trees.loc[mask, 'pred_rh98'], trees.loc[mask, 'hgt'], lims=(0,15), ax=ax, scatter_kws={'s': 15, 'zorder':2, 'alpha':0.5}, line_kws={'color':'k', 'alpha':0.5, 'zorder':1})

### Climate sensitivity

In [ ]:
# Load LandTrendr and predictions and make long form for plotting
path = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi_extract2.gpkg"

df = gpd.read_file(path)

df = df[df["change"]=="none"] # only analyze no change sites

srcs = list(indirs.keys())
value_vars = [col for col in df.columns for src in srcs if col.startswith(src)]
id_vars = [col for col in df.columns if col not in value_vars]

ldf = (
    df.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='original_column_name', # Create a temporary column for old names
        value_name='value' # The required 'value' column
    )
    # Split the temporary column into the required new columns
    .assign(
        source = lambda x: x['original_column_name'].str.split('_').str[0],
        year = lambda x: x['original_column_name'].str.split('_').str[1].astype(int), # Convert to int
        band = lambda x: x['original_column_name'].str.split('_').str[2]
    )
    # Drop the temporary column
    .drop(columns=['original_column_name'])
)

In [ ]:
# Line plot all sites for cover
bdf = ldf[(ldf["band"]=="cover")]

fig, ax = plt.subplots(figsize=(3,1.5))
sns.lineplot(data=bdf, x="year", y="value", hue="site", linewidth=0.5, legend=False, palette="tab20", ax=ax)
ax.axvspan(xmin=2010.5, xmax=2014.5, facecolor='gray', alpha=0.3, label='No predictions')
ax.axvline(2015, linestyle='--', c='k')
ax.text(2014, 0.32, '2015 drought')
ax.set(xlabel="Cover (%)", ylabel="Year")

figname = "climate_sensitivity_cover"
for ext in [".svg"]:
    figpath = os.path.join(figdir, figname + ext)
    fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Get important predictors to decide which ones to plot
path = r"J:\projects\ECOFOR\gedi\models\v08\GEDI_2AB_2019to2023_leafon_sampy500m_all_imps_v08.csv"
idf = pd.read_csv(path, header=[0,1])
idf = idf['lt-p-s-t_cover']
idf.mean().sort_values(ascending=False)[:20]

In [ ]:
# # Line plot all sites for NDVI

# # Load base landsat composite values for comparison
# path = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi_landsat_wet.csv"
# odf = pd.read_csv(path)

# fig, ax = plt.subplots(figsize=(3,3))
# bdf = ldf[(ldf["source"]=="ltwet") & (ldf["band"]=="ndvi") & (ldf["change"]=="none")]
# bdf["value"] = bdf["value"] / 1000 # rescale ndvi to real value
# sns.lineplot(data=bdf, x="year", y="value", hue="site", linewidth=0.5, legend=False, palette="tab20", ax=ax)

# # sns.scatterplot(data=odf, x="year", y="ndvi", hue="site", linewidth=0.5, legend=False, palette="tab20", ax=ax, alpha=0.7)

In [ ]:
# Select LandTrendr fits for demonstration
combos = [
    (521, "wet", "ndvi", 2007), 
    (1704, "wet", "green", 2007),
    (1704, "wet", "green", 1984),
#     (809, "wet", "green", 2007),
#     (809, "wet", "green", 1984),
]

for (site, season, band, starty) in combos:
    # Load base landsat composite values for comparison
    path = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi_landsat_"+season+"2.csv"
    odf = pd.read_csv(path)
    odf = odf[(odf["site"] == site) & (odf[band]!=-32768) & (odf["year"]>starty)]
    
    mask = (ldf["site"]==site) & (ldf["source"]=="lt"+season) & (ldf["band"]==band) & (ldf["year"]>starty)
    bdf = ldf[mask].copy()
    bdf["value"] = bdf["value"] / 1000 # rescale to real value
    
    fig, ax = plt.subplots(figsize=(3,1.5))
    sns.lineplot(data=bdf, x="year", y="value", linewidth=1, legend=False, ax=ax)
    sns.scatterplot(data=odf, x="year", y=band, size=1, legend=False, ax=ax)
    ax.set(xlabel="Year", ylabel=band)
    
    figname = "lt_vs_landsat_"+str(site)+"_"+season+"_"+band+"_"+str(starty)
    for ext in [".svg"]:
        figpath = os.path.join(figdir, figname + ext)
        fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Line plot LT against Landsat for a given band for each site
band = "green"
season = "wet"

# Load base landsat composite values for comparison
path = r"J:\projects\ECOFOR\climate_sensitivity\vca_unburned09to17_snapped_pi_landsat_"+season+"2.csv"
odf = pd.read_csv(path)

for site in ldf["site"].unique():
    fig, ax = plt.subplots(figsize=(3,2))
    mask = (ldf["site"]==site) & (ldf["source"]=="lt"+season) & (ldf["band"]==band) & (ldf["year"]>starty)
    bdf = ldf[mask].copy()
    bdf["value"] = bdf["value"] / 1000 # rescale to real value

    sns.lineplot(data=bdf, x="year", y="value", linewidth=1, legend=False, ax=ax)

    sodf = odf[(odf["site"] == site) & (odf[band]!=-32768)]
    sns.scatterplot(data=sodf, x="year", y=band, legend=False, ax=ax)
    
    ax.set(xlabel="Year", ylabel=band)

    print(site)
    display(fig)

In [ ]:
# Take difference between all pairs of subsequent years
bdf_s = bdf.sort_values(by=['site', 'year']).reset_index(drop=True)
bdf_s['valdif'] = bdf_s.groupby('site')['value'].diff()

In [ ]:
# Average change across all year pairs
bdf_s.groupby('year')['valdif'].mean().mean()

In [ ]:
# Average change for 2010 to 2015
bdf_s[bdf_s["year"]==2015].set_index("site")["valdif"].mean()

In [ ]:
# Average change for 2015 to 2016
bdf_s[bdf_s["year"]==2016].set_index("site")["valdif"].mean()

### Small Area Estimation

In [ ]:
# Get model-based estimators exported from R
# path = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20240823_l6b6.csv" #r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20240823_l6b6.csv"
path =  r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20251103.csv" #r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20250312.csv" #
package = "emdi" #"sae" # 

df = pd.read_csv(path)

df = df[df['metric']!='pai'] # drop use of PAI

if package=="sae":
    df = df.rename(columns={"mean":"Mean", "domain":"Domain", "mse":"Mean_MSE"})

df['metric_title'] = df['metric'].replace({'cover':'Cover (%)', 'rh98': 'RH98 (m)', 'fhd': 'FHD', 'pai':'PAI'})
df['aoi'] = df['Domain'].str[:-5]
df['year'] = df['Domain'].str[-4:].astype(int)

# Fix cover to be in percent for plotting and get confidence intervals
df['mean_rmse'] = df['Mean_MSE']**0.5
df.loc[df['metric']=='cover', ['Mean', 'mean_rmse']] *= 100
t_val = 1.645 # critical value for 90% CI from t-distribution with inf degrees of freedom
df['mean_ci90_half'] = (df['mean_rmse'] * 1.645) / 2

#### Model-based vs. Design-based

In [ ]:
# Get direct estimators exported from R
path = r"J:\projects\ECOFOR\gedi\sae\direct_gedi_estimates_20250415.csv"
ddf = pd.read_csv(path)

ddf = ddf[ddf['domain'].isin(df['Domain'].unique())]

ddf['aoi'] = ddf['domain'].str[:-5]
ddf['year'] = ddf['domain'].str[-4:].astype(int)

ddf = ddf[ddf['metric']!='pai'] # drop use of PAI

ddf = ddf.dropna(subset='mean_ci90_half') # drop rows without proper variance (nclusters<2)

# convert cover to percent
ddf.loc[ddf['metric']=='cover', ['mean', 'mean_ci90_half', 'mean_ci90_lower', 'mean_ci90_upper']] *= 100

# merge with model-based df in long form so all can be plotted together
df.columns = df.columns.str.lower()
df['mean_ci90_upper'] = df['mean'] + df['mean_ci90_half']
df['mean_ci90_lower'] = df['mean'] - df['mean_ci90_half']
df['samptype']='model-based'

mdf = pd.concat([df, ddf])

In [ ]:
# Merge design and model-based in wide form for direct comparison of CIs
cdf = ddf[ddf['samptype']=='cluster']
cols = ['domain', 'metric', 'aoi', 'year', 'mean', 'mean_ci90_lower', 'mean_ci90_upper', 'mean_ci90_half']
wdf = pd.merge(cdf[cols+['nclusters']], df[cols+['metric_title']], how='left', on=['domain', 'metric'], suffixes=['_d', '_m'])

# model-based estimates overlapping the CI of the design-based estimates
wdf['m_in_dci'] = wdf['mean_m'].between(wdf['mean_ci90_lower_d'], wdf['mean_ci90_upper_d'])

# model-based CI and design_based CI overlap
wdf['overlap'] = (wdf['mean_ci90_upper_m'] >= wdf['mean_ci90_lower_d']) & (wdf['mean_ci90_lower_m'] <= wdf['mean_ci90_upper_d'])

# difference and percent difference in means
wdf['m_diff'] = wdf['mean_m'] - wdf['mean_d']
wdf['m_pctdiff'] = wdf['m_diff'] / wdf['mean_d'] * 100

# Column of relationship between model-based and design-based
wdf['relate'] = 'No overlap'
wdf.loc[wdf['overlap'], 'relate'] = 'CIs overlap'
wdf.loc[wdf['m_in_dci'], 'relate'] = 'Model-based within\nDesign-based CI'

print(len(wdf['domain'].unique()), 'domains.')

In [ ]:
# TODO: look at difference or percent difference or simply overlap/not overlap as a function of
#       aoi size, gedi sample size, gedi # of clusters, and difference of m_mean from the training sample mean.
#       Consider modeling the difference as a function of these factors.

In [ ]:
def one2one(x, y, data, color=None, label=None):
    lims = (data[[x,y]].min().min(), data[[x,y]].max().max())
    plt.plot(lims, lims, '--k')
    
xcol, ycol = "mean_m", "mean_d"
g = sns.FacetGrid(wdf, col="metric_title", hue="relate", sharey=False, sharex=False, aspect=0.9, height=2)
g.map_dataframe(one2one, xcol, ycol)
g.map_dataframe(plt.errorbar, xcol, ycol, "mean_ci90_half_d", "mean_ci90_half_m", fmt='.', ecolor='0.8', alpha=0.8)
g.add_legend(title='')
g.set_titles(col_template="{col_name}")
g.set_axis_labels(x_var = 'Model-based Mean', y_var='Design-based Mean')

# Add RMSE, R2 and Bias for each metric
for col, ax in zip(g.col_names, g.axes[0]):
    mask = wdf['metric_title']==col
    x, y = wdf.loc[mask, xcol], wdf.loc[mask, ycol]
        
    # add text
    r2 = r2_score(y, x)
    ax.text(0.99, 0.22, "R$^2$= " + str(r2.round(2)), transform=ax.transAxes, ha='right')
    bias = (x-y).mean()
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    rmse = mean_squared_error(y, x)**0.5
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')

# figname = "modelbased_vs_designbased"
# for ext in [".png", ".svg"]: #, ".pdf"]:
#     figpath = os.path.join(figdir, figname + ext)
#     g.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Get the distribution of the training data
path = r"J:\projects\ECOFOR\gedi\models\v08\GEDI_2AB_2019to2023_leafon_sampy500m_all_oob_v08.parquet"
oob = pd.read_parquet(path)

oob.columns = oob.columns.str.replace('fhd_normal','fhd')
oob[oob.columns[oob.columns.str.endswith('cover')]] *=100

modstr = '' #'pred_lt-p-s-t_' #
ostats = oob[[modstr+'cover', modstr+'rh98', modstr+'fhd']].describe()
ostats.rename(columns={'cover':'Cover (%)', 'rh98': 'RH98 (m)', 'fhd': 'FHD', 'pai':'PAI'}, inplace=True)

xcol, ycol = "mean_d", "m_diff"
g = sns.relplot(xcol, ycol, col='metric_title', hue='relate', kind='scatter', size='nclusters', data=wdf,
                facet_kws={'sharex':False, 'sharey':False}, aspect=0.75, height=2.1, alpha=0.8, sizes=(8,32))
g.map(sns.regplot, xcol, ycol, scatter=False, ci=False, color='k', line_kws={'alpha':0.5})
g.set_titles(col_template="{col_name}")
g.set_axis_labels(y_var = 'Estimator difference', x_var='Design-based Mean')
g.legend.remove()
g.add_legend(label_order=['2', '4', '6', '8', '10', '12'], title='N clusters')
for col, ax in zip(g.col_names, g.axes[0]):
    ax.axhline(0, linestyle='--', c='k')
    ax.axvline(ostats.loc['50%', modstr+col], linestyle=':', c='k', alpha=0.5)
    
# figname = "modelbased_err_vs_designbased"
# for ext in [".png", ".svg"]: #, ".pdf"]:
#     figpath = os.path.join(figdir, figname + ext)
#     g.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Percent of overlap between design-based and model-based
wdf.groupby('metric')[['m_in_dci', 'overlap']].mean().round(2)*100

In [ ]:
# Average size of CI half-width as percent of the design-based estimator
wdf['ci_half_pct_d'] = wdf['mean_ci90_half_d'] / wdf['mean_d'] * 100
wdf.groupby('metric')['ci_half_pct_d'].mean().round(1)

In [ ]:
# Average percent difference of model-based from design-based
wdf['m_pctdiff_abs'] = wdf['m_pctdiff'].abs()
wdf.groupby('metric')['m_pctdiff_abs'].describe().round(1)

In [ ]:
sns.violinplot('metric', 'm_pctdiff_abs', data=wdf)

In [ ]:
# Plot mean and CI of all years and metrics for select aois
# aois = ["thornybush", "skukuza_se", "plantation_a", "bushbuckridge_a"]#, "justacia"] #"moditlo"]
aois = mdf['aoi'].unique()
for aoi in aois:
    pdf = mdf[mdf['aoi']==aoi]
    order = pdf['year'].unique()
    order.sort()
        
    def errplot(x, y, data, order, hue, yerr, palette='deep', color=None):
        xs = np.arange(len(order))
        hues = data[hue].unique()
        dodge_width = 0.8
        dodge_vals = np.linspace(-dodge_width / 2, dodge_width / 2, len(hues)*2+1)[1::2]
        colors = sns.color_palette(palette, len(hues))
        for hue_val, dodge_val, color in zip(hues, dodge_vals, colors):
            ys = []
            yerrs = []
            for xi in order:
                mask = (data[x] == xi) & (data[hue] == hue_val)
                if mask.sum()>0:
                    ys.append(data[mask][y].to_numpy()[0])
                    yerrs.append(data[mask][yerr].to_numpy()[0])
                else:
                    ys.append(np.nan)
                    yerrs.append(np.nan)
            plt.bar(x=xs + dodge_val, height=ys, yerr=yerrs, width=dodge_width / len(hues), color=color, label=hue_val)
        plt.xticks(xs, order)

    g = sns.FacetGrid(pdf, row="metric", sharey=False, sharex=False, aspect=2)
    g.map_dataframe(errplot, "year", "mean", hue="samptype", yerr="mean_ci90_half", order=order)
    g.set_xlabels("")
    g.set_xticklabels(rotation=45, ha='right', rotation_mode='anchor')
    g.set_titles(template=aoi+" {row_name}")
    g.tight_layout()

#### Change in AOIs

In [ ]:
# Plot mean and CI of all years and metrics for select aois
aois = ["thornybush", "skukuza_se", "plantation_a", "bushbuckridge_a", "justacia"] #"moditlo"]
for aoi in aois:
    pdf = df[df['aoi']==aoi]

    def errplot(x, y, yerr, **kwargs):
        ax = plt.gca()
        data = kwargs.pop("data")
        data.plot(x=x, y=y, yerr=yerr, kind="bar", ax=ax, capsize=4, **kwargs)

    g = sns.FacetGrid(pdf, row="metric_title", sharey=False, sharex=False, aspect=2)
    g.map_dataframe(errplot, "year", "mean", "mean_ci90_half")
    g.set_xlabels("")
    g.set_xticklabels(rotation=45, ha='right', rotation_mode='anchor')
    g.set_titles(template=aoi+" {row_name}")
    g.tight_layout()

In [ ]:
# Make one figure of all metrics of pre/post for each AOI
# aois = ["thornybush", "bushbuckridge_a", "moditlo"]
aoi_years = {
    'thornybush':{'pre':2017, 'post':2021},
    'bushbuckridge_a':{'pre':2007, 'post':2021},
    'plantation_a':{'pre':2017, 'post':2021},
    'skukuza_se':{'pre':2007, 'post':2022},
#              'justacia':{'pre':2016, 'post':2021}
            }
for aoi, ydict in aoi_years.items():
    print(aoi)
    pdf = df[df['aoi']==aoi]
    pdf = pdf[pdf['year'].isin([ydict['pre'], ydict['post']])]
    display(pdf)

    def errplot(x, y, yerr, **kwargs):
        ax = plt.gca()
        data = kwargs.pop("data")
        data.plot(x=x, y=y, yerr=yerr, kind="bar", ax=ax, capsize=3, **kwargs)

    g = sns.FacetGrid(pdf, col="metric_title", sharey=False, height=2, aspect=0.5)
    g.map_dataframe(errplot, "year", "mean", "mean_ci90_half")
    g.set_xlabels("")
    g.set_xticklabels(rotation=45, ha='right', rotation_mode='anchor')
    g.set_titles(template="{col_name}")
    g.tight_layout()
    
    figname = "gedi_sae_" + aoi + str(ydict['pre'])+"_"+str(ydict['post'])
    for ext in [".pdf"]: #, ".svg"]: #, ".pdf"]:
        figpath = os.path.join(figdir, figname + ext)
        g.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
mask = (df['aoi']=='bushbuckridge_a') & (df['metric']=='cover')
calc_chg = df.loc[mask & (df['year']==2021), 'mean'].iloc[0] - df.loc[mask & (df['year']==2007), 'mean'].iloc[0]
print('Cover in bushbuckridge changed', np.round(calc_chg, 1), '%')

mask = (df['aoi']=='thornybush') & (df['metric']=='cover')
calc_chg = df.loc[mask & (df['year']==2021), 'mean'].iloc[0] - df.loc[mask & (df['year']==2017), 'mean'].iloc[0]
print('Cover in thornybush changed', np.round(calc_chg, 1), '%')

mask = (df['aoi']=='thornybush') & (df['metric']=='rh98')
calc_chg = df.loc[mask & (df['year']==2021), 'mean'].iloc[0] - df.loc[mask & (df['year']==2017), 'mean'].iloc[0]
print('RH98 in thornybush changed', np.round(calc_chg, 1), 'm')

mask = (df['aoi']=='skukuza_se') & (df['metric']=='cover')
calc_chg = df.loc[mask & (df['year']==2022), 'mean'].iloc[0] - df.loc[mask & (df['year']==2007), 'mean'].iloc[0]
print('Cover in skukuza changed', np.round(calc_chg, 1), '%')

## Presentation figures

In [ ]:
# presentation set up
figdir = r"E:\My Drive\Work\ecofor\ssnm_25" #r"C:\Users\stevenf\Google Drive\Work\ecofor\forestsat24" #r"G:\My Drive\Work\ecofor\forestsat24"
os.makedirs(figdir, exist_ok=True)

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial']
mpl.rcParams['font.size'] = 16

sns.set_style('ticks',
               {'font.family':'sans-serif', 'font.sans-serif':['Arial'], 'font.size':16})

# temporary style to use on dark backgrounds
# white_axes = {'axes.labelcolor': '.99', 'text.color': '.99', 'xtick.color': '.99', 'ytick.color': '.99', 'axes.edgecolor': '.99', 'figure.facecolor': 'black'}

### Model accuracy

In [ ]:
# Load and setup data
pdf = gpd.read_parquet(r"D:\ECOFOR\gedi\models\v05\GEDI_2AB_2019to2023_leafon_sampy500m_all_oob_v05.parquet")
source_dict = {'l30':'Landsat',
               'hls':'HLS',
               'palsar':'PALSAR',
               'l30p': 'Landsat + PALSAR',
               'hlsp': 'HLS + PALSAR'}
ydict = {'cover': 'Cover',
         'pai': 'PAI',
         'rh98': 'RH98',
         'fhd_normal': 'FHD'}
Xsets = list(source_dict.keys())
Ycols = list(ydict.keys())

# make cover cols as percent
pdf[pdf.columns[pdf.columns.str.contains('cover')]] *= 100

In [ ]:
# Plot obs vs pred for all model results
nrows = len(Xsets)
fig, axes = plt.subplots(nrows, 4, figsize=(12, 2.3*nrows))
sdf = pd.DataFrame(columns = pd.MultiIndex.from_product([list(ydict.values()), ['R2', 'RMSE', 'Bias', 'N']], names=("Metric", "Stat")))

for i, (Xset, axrow) in enumerate(zip(Xsets, axes)):
    for j, (ycol, ax) in enumerate(zip(Ycols, axrow)):
        x, y = pdf['pred_'+Xset+'_'+ycol], pdf[ycol]
        hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
        ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

        r2 = r2_score(y, x)
        bias = (x-y).mean()
        rmse = mean_squared_error(y, x)**0.5
        
        # Save stats
        sdf.loc[Xset, (ydict[ycol], 'R2')] = r2
        sdf.loc[Xset, (ydict[ycol], 'RMSE')] = rmse
        sdf.loc[Xset, (ydict[ycol], 'Bias')] = bias
        sdf.loc[Xset, (ydict[ycol], 'N')] = len(y)

        # add text
        ax.text(0.99, 0.22, "R$^2$= " + str(r2.round(2)), transform=ax.transAxes, ha='right')
        ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
        ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')

        ax.set(title=Xset+'_'+ycol)
        if j==0:
            ax.set(ylabel='Observed')
        fig.supxlabel('Predicted', x=0.47, y=0, ha='center', fontsize=10)

fig.subplots_adjust(wspace=0.3, hspace=0.5)
cb = fig.colorbar(hb, ax=axes, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
cb.set_label('Count')

In [ ]:
# Plot Obs vs pred for only the best model
# 2x2
Xset = 'l30p' #'hlsp'
ycol_dict = {'cover':'Cover (%)', 'pai':'PAI', 'rh98':'RH98', 'fhd_normal':'FHD'}

fig, axes = plt.subplots(2, 2, figsize=(6.8, 6))

for i, (ycol, ax) in enumerate(zip(Ycols, axes.flat)):
    x, y = pdf['pred_'+Xset+'_'+ycol], pdf[ycol]
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

    r2 = r2_score(y, x)
    bias = (x-y).mean()
    rmse = mean_squared_error(y, x)**0.5

    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(round(r2, 2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')

    ax.set(title=ycol_dict[ycol])
#     if i==0 or i==2:
#         ax.set(ylabel='Observed')
    fig.supxlabel('Predicted', x=0.47, y=0.02, ha='center', fontsize=16)
    fig.supylabel('Observed', x=0.02, y=0.5, ha='center', fontsize=16)

fig.subplots_adjust(wspace=0.3, hspace=0.5)
cb = fig.colorbar(hb, ax=axes, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
cb.set_label('Count')

# figname = "gedi_acc_" + Xset
# for ext in [".png", ".svg"]: #, ".pdf"]:
#     figpath = os.path.join(figdir, figname + ext)
#     fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Plot Obs vs pred for only the best model
# 1x4
Xset = 'l30p' #'hlsp'
ycol_dict = {'cover':'Cover (%)', 'pai':'PAI', 'rh98':'RH98', 'fhd_normal':'FHD'}

fig, axes = plt.subplots(4, 1, figsize=(2.4, 2.8*4))

for i, (ycol, ax) in enumerate(zip(Ycols, axes.flat)):
    x, y = pdf['pred_'+Xset+'_'+ycol], pdf[ycol]
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')

    r2 = r2_score(y, x)
    bias = (x-y).mean()
    rmse = mean_squared_error(y, x)**0.5

    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(round(r2, 2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')

    ax.set(title=ycol_dict[ycol])
#     if i==0 or i==2:
#         ax.set(ylabel='Observed')


fig.supxlabel('Predicted', x=0.5, y=0, ha='center', fontsize=16)
fig.supylabel('Observed', x=0, y=0.5, ha='center', fontsize=16)

fig.subplots_adjust(left=0.15, bottom=0.05, hspace=0.4)
# cb = fig.colorbar(hb, ax=axes, location='bottom', orientation='horizontal')#, pad=0.05)#, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
# cb.set_label('Count')

# figname = "gedi_acc_" + Xset + "_vert"
# for ext in [".png", ".svg"]: #, ".pdf"]:
#     figpath = os.path.join(figdir, figname + ext)
#     fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Bar chart of accuracy stats
ldf = pd.melt(sdf, ignore_index=False).reset_index()
ldf['Source'] = ldf['index'].map(source_dict)

mask = ldf['Stat'].isin(['R2', 'RMSE'])
palette = ['#a6cee3','#b2df8a','#cab2d6', '#1f78b4','#33a02c']
g = sns.catplot(data=ldf[mask], x="value", y="Source", col="Stat", row="Metric", kind='bar', sharex=False, sharey=True, height=3, aspect=1.5, margin_titles=True, palette=palette)
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.set_ylabels("")

figname = "gedi_acc_all"
for ext in [".png", ".svg"]: #, ".pdf"]:
    figpath = os.path.join(figdir, figname + ext)
    g.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

#### Temporal cross-validation

In [ ]:
# Load and setup TCV predictions
pdf = pd.read_csv(r"J:\projects\ECOFOR\gedi\models\v05\tcv\GEDI_2AB_2019to2023_leafon_sampy500m_all_l30p_tcv_v5.csv")
source_dict = {'l30':'Landsat',
               'hls':'HLS',
               'palsar':'PALSAR',
               'l30p': 'Landsat + PALSAR',
               'hlsp': 'HLS + PALSAR'}
ydict = {'cover': 'Cover',
         'pai': 'PAI',
         'rh98': 'RH98',
         'fhd_normal': 'FHD'}
Xsets = list(source_dict.keys())
Ycols = list(ydict.keys())

# make cover cols as percent
pdf[pdf.columns[pdf.columns.str.contains('cover')]] *= 100

In [ ]:
# Load TCV stats
stats_path = r"J:\projects\ECOFOR\gedi\models\v05\tcv\GEDI_2AB_2019to2023_leafon_sampy500m_all_l30p_tcv_v5_stats.csv"
stats = pd.read_csv(stats_path)

tcv_stats = stats.groupby(['dset', 'ycol']).mean()

# Change stats back to percent from frac
tcv_stats.loc[('l30p', 'cover'), ['rmse', 'bias']] *= 100

In [ ]:
# Plot Obs vs pred for only the best model
# 1x4
Xset = 'l30p' #'hlsp'
ycol_dict = {'cover':'Cover (%)', 'pai':'PAI', 'rh98':'RH98', 'fhd_normal':'FHD'}

fig, axes = plt.subplots(4, 1, figsize=(2.4, 2.8*4))

for i, (ycol, ax) in enumerate(zip(Ycols, axes.flat)):
    x, y = pdf[Xset+'_'+ycol], pdf[ycol]
    hb = ax.hexbin(x, y, gridsize=20, mincnt=1, cmap='magma_r', linewidths=0, edgecolor='none', vmax=2000)
    ax.plot((y.min(), y.max()), (y.min(),y.max()), '--k')
    
    # # Overall error from TCV predictions
    # r2 = r2_score(y, x)
    # bias = (x-y).mean()
    # rmse = mean_squared_error(y, x)**0.5
    
    # Error from mean of TCV (correct way)
    r2 = tcv_stats.loc[(Xset, ycol), 'r2']
    bias = tcv_stats.loc[(Xset, ycol), 'bias']
    rmse = tcv_stats.loc[(Xset, ycol), 'rmse']
    
    # add text
    ax.text(0.99, 0.22, "R$^2$= " + str(round(r2, 2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.13, "Bias= "+str(bias.round(2)), transform=ax.transAxes, ha='right')
    ax.text(0.99, 0.02,  "RMSE= " + str(rmse.round(2)), transform=ax.transAxes, ha='right')

    ax.set(title=ycol_dict[ycol])
#     if i==0 or i==2:
#         ax.set(ylabel='Observed')


fig.supxlabel('Predicted', x=0.5, y=0, ha='center', fontsize=16)
fig.supylabel('Observed', x=0, y=0.5, ha='center', fontsize=16)

fig.subplots_adjust(left=0.15, bottom=0.05, hspace=0.4)
# cb = fig.colorbar(hb, ax=axes, location='bottom', orientation='horizontal')#, pad=0.05)#, shrink=True, aspect=16, pad=0.02) #cax=cax, aspect=)#
# cb.set_label('Count')

figname = "gedi_tcv_" + Xset + "_vert"

for ext in [".png", ".svg"]: #, ".pdf"]:
    figpath = os.path.join(figdir, figname + ext)
    fig.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# TCV model accuracy



### SAE figure

In [ ]:
path = r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20240823_l6b6.csv" #r"J:\projects\ECOFOR\gedi\sae\sae_gedi_estimates_20240823_l6b6.csv"
df = pd.read_csv(path)

df['aoi'] = df['Domain'].str[:-5]
df['year'] = df['Domain'].str[-4:].astype(int)

df['mean_rmse_half'] = df['Mean_MSE']**0.5 / 2
df['median_rmse_half'] = df['Median_MSE']**0.5 / 2

df['metric_title'] = df['metric'].replace({'cover':'Cover (%)', 'rh98': 'RH98 (m)', 'fhd': 'FHD', 'pai':'PAI'})

# Fix cover to be in percent for plotting
df.loc[df['metric']=='cover', ['Mean', 'Mean_MSE', 'mean_rmse_half']] *= 100

In [ ]:
# Make a line plot of all years
aois = ["thornybush", "skukuza_se", "plantation_a"] #"bushbuckridge_a", "moditlo"
for aoi in aois:
    pdf = df[df['aoi']==aoi]

    def errplot(x, y, yerr, **kwargs):
        ax = plt.gca()
        data = kwargs.pop("data")
        data.plot(x=x, y=y, yerr=yerr, kind="bar", ax=ax, capsize=4, **kwargs)

    g = sns.FacetGrid(pdf, row="metric_title", sharey=False, sharex=False, aspect=2)
    g.map_dataframe(errplot, "year", "Mean", "mean_rmse_half")
    g.set_xlabels("")
    g.set_xticklabels(rotation=45, ha='right', rotation_mode='anchor')
    g.set_titles(template=aoi+" {row_name}")
    g.tight_layout()

In [ ]:
# Make one figure of all metrics of pre/post for each AOI
# aois = ["thornybush", "bushbuckridge_a", "moditlo"]
aoi_years = {'thornybush':{'pre':2016, 'post':2022},
             'skukuza_se':{'pre':2009, 'post':2019},
             'plantation_a':{'pre':2018, 'post':2021}
#              'bushbuckridge_a':{'pre':2007, 'post':2022},
             # 'moditlo':{'pre':2016, 'post':2021}
            }
for aoi, ydict in aoi_years.items():
    pdf = df[df['aoi']==aoi]
    pdf = pdf[pdf['year'].isin([ydict['pre'], ydict['post']])]

    def errplot(x, y, yerr, **kwargs):
        ax = plt.gca()
        data = kwargs.pop("data")
        data.plot(x=x, y=y, yerr=yerr, kind="bar", ax=ax, capsize=4, **kwargs)

    g = sns.FacetGrid(pdf, col="metric_title", sharey=False, aspect=0.5)
    g.map_dataframe(errplot, "year", "Mean", "mean_rmse_half")
    g.set_xlabels("")
    g.set_xticklabels(rotation=45, ha='right', rotation_mode='anchor')
    g.set_titles(template="{col_name}")
    g.tight_layout()
    
    # figname = "gedi_sae_" + aoi + "18to21"
    # for ext in [".png", ".svg"]: #, ".pdf"]:
    #     figpath = os.path.join(figdir, figname + ext)
    #     g.savefig(figpath, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# # Drop thornybush rh98 in 2021 which has a weirdly high MSE
# row_mask = (pdf['year'].isin([2015, 2021])) & (pdf['metric']=='rh98')
# pdf.loc[row_mask, ['Mean', 'Mean_MSE']] = np.nan

In [ ]:
# def get_errorbars(y):
#     ax=plt.gca()
    
# g = sns.catplot(data=pdf, x='year', y='Mean', col='metric', kind='bar', sharey=False)